## Climate_Emissions_Target_Tracker

In [1]:
import lseg.data as ld
import logging
import warnings # This will be useful to warn users if the Scope they're after is not covered.
import re # We'll use the `re` library to separate numbers from words in strings.
import pandas as pd # this library is great for dataframe manipulations.
import plotly.graph_objects as go # `plotly` and its modules are useful for us to plot graphs.
import numpy as np # `numpy` in this case will be useful for us to plot graphs.
from datetime import datetime # this is useful for us to collect data from LSEG up to today.


class GHGEmissionsBaseVsTargetUsingLSEGWorkspaceClass:


    def __init__(
            self,
            printMessage: bool = True,
            keepMessageObject: bool = False,
            RIC: str = "LVMH.PA", # "LVMH.PA", "MSFT.O", "PEUG.PA", "AAPL.O", "GOOGL.O", "AMZN.O", "TSLA.O", "META.O", "NVDA.O", "JPM.N"
            scope: str = "Scope 1", # choice between "Scope 1", "Scope 2", "Scope 3", "Scope 1 and 2" or "Scope 1 and 2 and 3"
            term: str = "LT", # choice between "ST" and "LT"
            source: str = "LSEG", # choice between "LSEG" and "CDP"
            fiscalYear: int = 2021,
            suppressWarnings: bool = True):
        """
        Initialize the GHGEmissionsBaseVsTargetUsingLSEGWorkspaceClass.

        Parameters:
        - printMessage (bool, default=True): Whether to print messages.
        - keepMessageObject (bool, default=False): Whether to keep the message object.
        - RIC (str, default="LVMH.PA"): The RIC code for the company. E.g.: "LVMH.PA", "MSFT.O", "PEUG.PA", "AAPL.O", "GOOGL.O", "AMZN.O", "TSLA.O", "META.O", "NVDA.O", "JPM.N", ...
        - scope (str, default="Scope 1"): The scope of emissions ("Scope 1", "Scope 2", "Scope 3", "Scope 1 and 2", "Scope 1 and 2 and 3").
        - term (str, default="LT"): The term of interest ("ST" for short-term, "LT" for long-term).
        - source (str, default="LSEG"): The data source ("LSEG" or "CDP").
        - fiscalYear (int, default=2021): The fiscal year of interest.
        - suppressWarnings (bool, default=True): Whether to suppress warnings.
        """

        # Validate parameters
        valid_scopes = ["Scope 1", "Scope 2", "Scope 3", "Scope 1 and 2", "Scope 1 and 2 and 3"]
        valid_terms = ["ST", "LT"]
        valid_sources = ["LSEG", "CDP"]

        if scope not in valid_scopes:
            raise ValueError(f"Invalid scope: {scope}. Must be one of {valid_scopes}.")
        if term not in valid_terms:
            raise ValueError(f"Invalid term: {term}. Must be one of {valid_terms}.")
        if source not in valid_sources:
            raise ValueError(f"Invalid source: {source}. Must be one of {valid_sources}.")

        self.printMessage = printMessage
        self.keepMessageObject = keepMessageObject
        self.RIC = RIC
        self.scope = scope
        self.term = term
        self.source = source
        self.suppressWarnings = suppressWarnings
        self.fiscalYear = fiscalYear
        # First, let's put in place the primary parameters:
        self.termOfInterest = term
        self.source = source
        
        if self.termOfInterest == "ST":
            self.termNotOfInterest = "LT"
        else:
            self.termNotOfInterest = "ST"
        
        if self.source == "LSEG":
            self.sourceOfInterest = ""
        elif self.source == "CDP":
            self.sourceOfInterest = self.source
        
        # Logging initialization
        logging.basicConfig(level=logging.INFO)
        logging.info(f"Initialized GHGEmissionsBaseVsTargetUsingLSEGWorkspaceClass with RIC={RIC}, scope={scope}, term={term}, source={source}, fiscalYear={fiscalYear}")


    def _base_year_data_for_multiple_scopes(
            self,
            RIC: str,
            testField: list,
            _sourceOfInterest: str,
            termOfInterest: str,
            setOfInterest: int,
            fiscalYear: int,
            warning: bool = True) -> tuple:
        """
        Retrieve base year data for multiple scopes.

        Parameters:
        - RIC (str): The RIC code for the company.
        - testField (list): The list of fields to test.
        - _sourceOfInterest (str): The source of interest.
        - termOfInterest (str): The term of interest.
        - setOfInterest (int): The set of interest.
        - fiscalYear (int): The fiscal year of interest.
        - warning (bool, default=True): Whether to display warnings.

        Returns:
        - tuple: A tuple containing:
            - valueTestBaseYear (float): The base year value.
            - dfTestBaseYearData (pd.DataFrame): The DataFrame containing base year data.
            - dfTestScope (pd.DataFrame): The DataFrame containing scope data.
            - valueTestScope (float): The scope value.
        """

        try:
            dfTestBaseYearData = ld.get_history(
                universe=RIC,
                fields=[f"TR.{_sourceOfInterest}{termOfInterest}GHGEmissionBaseYearSet{setOfInterest}"],
                start="1990-01-01",
                end=datetime.now().strftime("%Y-%m-%d")).replace('', pd.NA).dropna()
            valueTestBaseYear = dfTestBaseYearData.loc[
                dfTestBaseYearData.index.year == fiscalYear].values[0][0]
            if pd.isna(valueTestBaseYear):
                valueTestBaseYear = dfTestBaseYearData.dropna().loc[
                    dfTestBaseYearData.dropna().index.year == fiscalYear].values[0][0]
            testCheckIf_valueTestBaseYear_exists = valueTestBaseYear  # if `valueTestBaseYea` doesn't exist, this line will fail and the except clause below will trigger.
        except:
            if warning:
                warnings.warn(
                    "\nNo results returned. The function used was \n" +
                    f"`ld.get_history(universe=['{RIC}'], " +
                    f"fields=['TR.{_sourceOfInterest}{termOfInterest}GHGEmissionBaseYearSet{setOfInterest}'], " +
                    "start='1990-01-01', end=datetime.now().strftime('%Y-%m-%d')).drapna()`. \n")
        dfTestScope = ld.get_data(
            universe=RIC,
            fields=testField)
        valueTestScope = dfTestScope.values[0][1]

        return valueTestBaseYear, dfTestBaseYearData, dfTestScope, valueTestScope


    def get_list_of_sets_that_contain_our_scope(
            self,
            term: str,
            RIC: str,
            scope: str,
            fiscalYear: int) -> tuple:
        """
        Retrieve a list of sets that contain the specified scope for a given RIC and fiscal year.

        Parameters:
        - term (str): The term of interest ("ST" for short-term, "LT" for long-term).
        - RIC (str): The RIC code for the company.
        - scope (str): The scope of emissions ("Scope 1", "Scope 2", "Scope 3", "Scope 1 and 2", "Scope 1 and 2 and 3").
        - fiscalYear (int): The fiscal year of interest.

        Returns:
        - tuple: A tuple containing:
            - scopeDictPerY (dict): A dictionary with the fiscal year as the key and a list of set numbers as the value.
            - dfOfSets (pd.DataFrame): The DataFrame containing filtered data for the specified year and scope.
            - _dfOfSets (pd.DataFrame): The original DataFrame containing data for the specified time window.
        """
        logging.info(f"Retrieving sets containing scope for RIC={RIC}, term={term}, scope={scope}, fiscalYear={fiscalYear}")

        # Unfortunutally, we may have several rows of the same year with different data.
        # E.g.: using `ld.get_history(universe="VODJ.J", fields=[f"TR.LTGHGEmissionScopeSet{i}" for i in range(1,7)], start="1990-01-01", end=datetime.now().strftime("%Y-%m-%d"))`,
        # you get two rows for "2024-03-31" even though you only have one data point per column (the other row is empty).
        # As a result, we need to ask for data through a time-window (i.e.: a date-range) and then filter the data by year and scope.
        # First: Get data through a time-window:
        _dfOfSets = ld.get_history(
            universe=RIC,
            fields=[f"TR.{term}GHGEmissionScopeSet{i}" for i in range(1,7)],
            header_type=ld.HeaderType.NAME_AND_TITLE,
            start="1990-01-01",
            end=datetime.now().strftime("%Y-%m-%d"))
        
        outOfSetsMessage = str(
                "Expected Error: " +
                f"No Set available for {scope} for {RIC} for the year {fiscalYear}. " +
                "You can check this with " +
                f"`ld.get_history(universe='{RIC}', " +
                f"fields=[f'TR.{term}GHGEmissionScopeSet{{i}}' for i in range(1,7)], " +
                "header_type=ld.HeaderType.NAME_AND_TITLE, start='1990-01-01', " +
                "end=datetime.now().strftime('%Y-%m-%d'))`")
        # Second: filter the data by year and scope:
        dfOfSets = _dfOfSets.replace('', pd.NA)  # We need to replace the empty strings with pd.NA to remove them with `dropna`
        dfOfSets = dfOfSets.loc[_dfOfSets.index.year == fiscalYear]  # Here we filter the data for the year we are interested in
        dfOfSets = dfOfSets.dropna(how='all').T.dropna(how='all')  # Now that we focused on tonly one year, we remove empty rows & columns
        dfOfSets = dfOfSets.bfill(axis=1)  # Use bfill to fill NA values with the next valid value in the row        
        try: # It may be that no data was returned, in which case we need a `try` loop.
            dfOfSets = dfOfSets.iloc[:, 0]  # Select the first column (the only one left)
            dfOfSets = dfOfSets.to_frame(name='Scope Array')  # Convert the Series to a DataFrame with a name for the column
            # Third: Get a dictionary of the sets available for our fiscal year of interest:
            scopeDictPerY = {fiscalYear:
                [i.split("|")[0][-1] for i in dfOfSets.loc[dfOfSets['Scope Array'] == scope].index]}  # Get the scope number for each scope in the DataFrame
        except:
            raise Exception(
                "Expected Error: " +
                f"No Set available for {scope} for {RIC} for the year {fiscalYear}. " +
                "You can check this with " +
                f"`ld.get_history(universe='{RIC}', " +
                f"fields=[f'TR.{term}GHGEmissionScopeSet{{i}}' for i in range(1,7)], " +
                "header_type=ld.HeaderType.NAME_AND_TITLE, start='1990-01-01', " +
                "end=datetime.now().strftime('%Y-%m-%d'))`")
        
        if len(scopeDictPerY[fiscalYear]) == 0:
            raise Exception(outOfSetsMessage)
        
        logging.info(f"Successfully retrieved sets containing scope for RIC={RIC}, term={term}, scope={scope}, fiscalYear={fiscalYear}")
        
        return scopeDictPerY, dfOfSets, _dfOfSets


    def get_data(
            self,
            set: int,
            fiscalYear: int,
            warning: bool = True,
            returnElements: bool = True) -> 'GHGEmissionsBaseVsTargetUsingLSEGWorkspaceClass':
        """
        Retrieve and process GHG emissions data for the specified set and fiscal year.

        Parameters:
        - set (int): The set number of interest.
        - fiscalYear (int): The fiscal year of interest.
        - warning (bool, default=True): Whether to display warnings.
        - returnElements (bool, default=True): Whether to return the instance of the class.

        Returns:
        - GHGEmissionsBaseVsTargetUsingLSEGWorkspaceClass: The instance of the class with updated data attributes.

        Example:
        >>> ghg = GHGEmissionsBaseVsTargetUsingLSEGWorkspaceClass(
        ...     RIC="PUREHEALTH.AD",  # "SHEL.L",
        ...     printMessage=False, keepMessageObject=True, source='LSEG',
        ...     scope="Scope 1 and 2", fiscalYear=2019)
        >>> setDictOfLists, dfOfSets, _dfOfSets = ghg.get_list_of_sets_that_contain_our_scope(
        ...     fiscalYear=ghg.fiscalYear, RIC=ghg.RIC, term=ghg.term, scope=ghg.scope)
        >>> result = ghg.get_data(fiscalYear=ghg.fiscalYear, returnElements=True, set=3)
        >>> result.get_fig().fig.show()
        """
        logging.info(f"Retrieving data for set={set}, fiscalYear={fiscalYear}")
        
        self.set = set

        # Now, we can create the list of fields we're after from LSEG's services:
        self.lstOfFlds = []

        scopeErrorMessage = str(
            "Expected Error: " +
            f"No data for {self.scope} is available for {self.RIC}, set {self.set}, " +
            f"term {self.term}, source {self.source} and fiscalYear {fiscalYear}. " +
            "You can see this with the command `ld.get_history(" +
            f"universe='{self.RIC}', header_type=ld.HeaderType.NAME_AND_TITLE, " +
            f"fields=[f'TR.{self.term}GHGEmissionScopeSet{{i}}' for i in range(1,7)])`\n" +
            f"for fiscalYear {fiscalYear}, this is also true for: " +
            f"`ld.get_history(universe=['{self.RIC}'], " +
            f"fields=['TR.{self.sourceOfInterest}{self.termOfInterest}GHGEmissionBaseYearSet{self.set}'], " +
            "start='1990-01-01', end=datetime.now().strftime('%Y-%m-%d'))`. \n")

        if self.scope == "Scope 1":
            self.baseFieldOfInterest = f"TR.{self.sourceOfInterest}CO2DirectScope1"
            self.lstOfFlds.append(self.baseFieldOfInterest)
            try:
                self.baseEmissionYear, self.dfTestBaseYearData, self.dfTestScope, self.valueTestScope = self._base_year_data_for_multiple_scopes(
                    RIC=self.RIC, _sourceOfInterest=self.sourceOfInterest, termOfInterest=self.termOfInterest, warning=warning,
                    setOfInterest=self.set, testField=[self.baseFieldOfInterest], fiscalYear=fiscalYear)
            except:
                raise Exception(scopeErrorMessage)

        elif self.scope == "Scope 2":
            self.baseFieldOfInterest = f"TR.{self.sourceOfInterest}CO2IndirectScope2"
            self.lstOfFlds.append(self.baseFieldOfInterest)
            try:
                self.baseEmissionYear, self.dfTestBaseYearData, self.dfTestScope, self.valueTestScope = self._base_year_data_for_multiple_scopes(
                    RIC=self.RIC, _sourceOfInterest=self.sourceOfInterest, termOfInterest=self.termOfInterest, warning=warning,
                    setOfInterest=self.set, testField=[self.baseFieldOfInterest], fiscalYear=fiscalYear)
            except:
                raise Exception(scopeErrorMessage)

        elif self.scope == "Scope 3":
            self.baseFieldOfInterest = f"TR.{self.sourceOfInterest}CO2IndirectScope3"
            self.lstOfFlds.append(self.baseFieldOfInterest)

            try:
                self.baseEmissionYear, self.dfTestBaseYearData, self.dfTestScope, self.valueTestScope = self._base_year_data_for_multiple_scopes(
                    RIC=self.RIC, _sourceOfInterest=self.sourceOfInterest, termOfInterest=self.termOfInterest, warning=warning,
                    setOfInterest=self.set, testField=[self.baseFieldOfInterest], fiscalYear=fiscalYear)
            except:
                raise Exception(scopeErrorMessage)

        elif self.scope == "Scope 1 and 2":
            baseFieldsOfInterest = [
                f"TR.{self.sourceOfInterest}CO2DirectScope1",
                f"TR.{self.sourceOfInterest}CO2IndirectScope2"]
            for i in baseFieldsOfInterest:
                self.lstOfFlds.append(i)
            try:
                self.baseEmissionYear, self.dfTestBaseYearData, self.dfTestScope, self.valueTestScope = self._base_year_data_for_multiple_scopes(
                    RIC=self.RIC, _sourceOfInterest=self.sourceOfInterest, termOfInterest=self.termOfInterest, warning=warning,
                    setOfInterest=self.set, testField=baseFieldsOfInterest, fiscalYear=fiscalYear)
            except:
                raise Exception(scopeErrorMessage)

        elif self.scope == "Scope 1 and 2 and 3":
            baseFieldsOfInterest = [
                f"TR.{self.sourceOfInterest}CO2DirectScope1",
                f"TR.{self.sourceOfInterest}CO2IndirectScope2",
                f"TR.{self.sourceOfInterest}CO2IndirectScope3"]
            for i in baseFieldsOfInterest:
                self.lstOfFlds.append(i)
            try:
                self.baseEmissionYear, self.dfTestBaseYearData, self.dfTestScope, self.valueTestScope = self._base_year_data_for_multiple_scopes(
                    RIC=self.RIC, _sourceOfInterest=self.sourceOfInterest, termOfInterest=self.termOfInterest, warning=warning,
                    setOfInterest=self.set, testField=baseFieldsOfInterest, fiscalYear=fiscalYear)
            except:
                raise Exception(scopeErrorMessage)

        else:
            raise Exception(
                "Expected Error: Please note that `scope` must be 'Scope 1', " +
                "'Scope 2', 'Scope 3', 'Scope 1 and 2' or 'Scope 1 and 2 and 3'.")

        # print(f"self.baseEmissionYear: {self.baseEmissionYear}")

        baseEmissionYearErrorMessage = str(
            "Expected Error: " +
            f"No data available for {self.scope} for {self.RIC} for set {self.set}. " +
            f"Indeed, `self.baseEmissionYear` is `{self.baseEmissionYear}`. " +
            f"You can check this with `ld.get_history(universe='{self.RIC}', " +
            f"fields=[f'TR.{self.sourceOfInterest}{self.termOfInterest}GHGEmissionBaseYearSet{self.set}']," +
            f"start='1990-01-01', end=datetime.now().strftime('%Y-%m-%d'))`. \n" +
            f"")
        try:
            if isinstance(self.baseEmissionYear, (int, float)) and np.isnan(self.baseEmissionYear):  # if np.isnan(self.baseEmissionYear):
                raise Exception(baseEmissionYearErrorMessage)
        except:
            raise Exception(baseEmissionYearErrorMessage)

        self.lstOfFlds.append(f"TR.{self.sourceOfInterest}{self.termOfInterest}GHGEmissionPercentageReductionTargetedSet{self.set}")
        self.lstOfFlds.append(f"TR.{self.sourceOfInterest}{self.termOfInterest}PercentageofGHGEmissionCoveredbyTargetSet{self.set}")
        self.lstOfFlds.append(f"TR.{self.sourceOfInterest}{self.termOfInterest}GHGEmissionScopeSet{self.set}")
        self.lstOfFlds.append(f"TR.{self.sourceOfInterest}{self.termOfInterest}GHGEmissionBaseYearSet{self.set}")
        self.lstOfFlds.append(f"TR.{self.sourceOfInterest}{self.termOfInterest}GHGEmissionTargetYearSet{self.set}")

        # print(f"self.termOfInterest: {self.termOfInterest}")
        # print(self.lstOfFlds)

        if self.suppressWarnings:
            warnings.simplefilter(action='ignore', category=FutureWarning)
            warnings.simplefilter(action='ignore', category=UserWarning)

        self.df1 = ld.get_history(
            universe=[self.RIC],
            fields=self.lstOfFlds,
            start="1990-01-01",
            end=datetime.now().strftime("%Y-%m-%d"),
            header_type=ld.HeaderType.NAME_AND_TITLE).dropna(how='all')
        self.df2 = self.df1.replace('', pd.NA).replace({pd.NA: np.nan})
        self.df3 = self.df2.dropna(how='all').replace({pd.NA: np.nan})

        if self.scope == "Scope 1 and 2":
            self.baseFieldOfInterest = str(
                f"CONSTRUCTED.{self.sourceOfInterest}TOTALCO2EQUIVALENTEMISSIONSCOPE1AND2")
            self.df3.loc[:, self.baseFieldOfInterest] = pd.to_numeric( # You need pd.to_numeric with errors='coerce' to ensure that the data is of numeric type and to handle any non-numeric values by converting them to NaN. This is necessary because the second series contains NaN values and is of type object, which can cause issues when performing arithmetic operations.
                self.df3.filter(like=f"TR.{self.sourceOfInterest}CO2DIRECTSCOPE1").iloc[:,0],
                errors='coerce').add(pd.to_numeric(
                    self.df3.filter(like=f"TR.{self.sourceOfInterest}CO2INDIRECTSCOPE2").iloc[:,0],
                    errors='coerce'), fill_value=None)
        elif self.scope == "Scope 1 and 2 and 3":
            self.baseFieldOfInterest = str(
                f"CONSTRUCTED.{self.sourceOfInterest}TOTALCO2EQUIVALENTEMISSIONSCOPE1AND2AND3")
            self.df3.loc[:, self.baseFieldOfInterest] = pd.to_numeric(
                self.df3.filter(like=f"TR.{self.sourceOfInterest}CO2DIRECTSCOPE1").iloc[:,0],
                errors='coerce').add(pd.to_numeric(
                    self.df3.filter(like=f"TR.{self.sourceOfInterest}CO2INDIRECTSCOPE2").iloc[:,0],
                    errors='coerce'), fill_value=None).add(pd.to_numeric(
                        self.df3.filter(like=f"TR.{self.sourceOfInterest}CO2INDIRECTSCOPE3").iloc[:,0],
                        errors='coerce'), fill_value=None)

        self.df4 = self.df3.filter(like='PERCENTAGEOFGHGEMISSIONCOVEREDBYTARGETSET')
        if self.df4.apply(pd.to_numeric, errors='coerce').dropna().gt(80).all().all():
            GHGEmissionCoveredbyTargetMessage = str(
                "'GHGEmissionCoveredbyTarget' figures are above 80%. " +
                "The higher this figure, the more reliable the results." +
                "In this case, we consider them to be high enough to be reliable.")
            if self.printMessage:
                print(GHGEmissionCoveredbyTargetMessage)
            if self.keepMessageObject:
                self.messageObject = str(GHGEmissionCoveredbyTargetMessage)
        else:
            GHGEmissionCoveredbyTargetMessage = str(
                "Be weary of the fact that 'GHGEmissionCoveredbyTarget' is below 80%. " +
                "The lower this figure, the less reliable the results.")
            if self.printMessage:
                print(GHGEmissionCoveredbyTargetMessage)
            if self.keepMessageObject:
                self.messageObject = str(GHGEmissionCoveredbyTargetMessage)
        if len(self.df3.filter(like='PERCENTAGEOFGHGEMISSIONCOVEREDBYTARGET').dropna()) == 0:
            raise Exception(
                "Expected Error: " +
                f"No data available for {self.scope} for {self.RIC} for set {self.set}. " +
                f"Indeed, `self.baseEmissionYear` is `{self.baseEmissionYear}`. " +
                f"You can check this with `ld.get_history(universe='{self.RIC}', " +
                f"fields=[f'TR.{self.sourceOfInterest}{self.termOfInterest}PERCENTAGEOFGHGEMISSIONCOVEREDBYTARGETSET{self.set}']," +
                f"start='1990-01-01', end=datetime.now().strftime('%Y-%m-%d'))`.")
        
        
        self.GHGEmissionTargetTypeDf = ld.get_history(
            universe=self.RIC,
            fields=[f"TR.{self.sourceOfInterest}{self.termOfInterest}GHGEmissionTargetTypeSet{self.set}"],
            start="1990-01-01",
            end=datetime.now().strftime("%Y-%m-%d"),
            header_type=ld.HeaderType.NAME_AND_TITLE)
        self.GHGEmissionTargetType = self.GHGEmissionTargetTypeDf.loc[
            self.GHGEmissionTargetTypeDf.index.year == fiscalYear].replace('', pd.NA).dropna().values[0][0]
        if self.GHGEmissionTargetType != 'Absolute':
            raise Exception(
                "Expected Error: " +
                "Note that the `GHGEmissionsBaseVsTargetUsingLSEGWorkspaceClass` Python Class is for Absolute Emission Types only." +
                f"Data available for {self.scope} for {self.RIC} for set {self.set} " +
                f"are for GHGEmissionTargetType {self.GHGEmissionTargetType}. " +
                f"You can check this with `ld.get_history(universe='{self.RIC}', " +
                f"fields=[f'TR.{self.sourceOfInterest}{self.termOfInterest}GHGEmissionTargetTypeSet{self.set}']," +
                "start='1990-01-01', end=datetime.now().strftime('%Y-%m-%d'))`.")

        self.df5 = self.df3.copy()
        self.df5.index = [
            np.int64(z) for z in pd.to_datetime(
                self.df5.index, errors='coerce').year.astype(object)] # keep only the years

        self.targetEmissionReductionYear = int(
            self.df3.filter(
                like='GHGEMISSIONTARGETYEARSET').loc[
                    self.df3.index.year == fiscalYear].dropna().dropna().values[0][0])
        self.baseEmission = self.df3[
            self.df3.index.year == int(
                self.baseEmissionYear)].filter(
                    like=self.baseFieldOfInterest.upper()).dropna().values[0][0]
        self.targetReductionEmission = self.baseEmission * (
            (self.df3.filter(like='GHGEMISSIONPERCENTAGEREDUCTIONTARGETED').loc[self.df3.index.year == fiscalYear].dropna().iloc[-1] / 100).iloc[0] * (
                self.df3.filter(like='PERCENTAGEOFGHGEMISSIONCOVEREDBYTARGET').loc[self.df3.index.year == fiscalYear].dropna().iloc[-1] / 100).iloc[0])
        self.targetEmission = self.baseEmission - self.targetReductionEmission
        if self.printMessage:
            print(f"baseEmissionYear = {self.baseEmissionYear} & " +
                  f"targetEmissionReductionYear = {self.targetEmissionReductionYear}")
        if self.printMessage:
            print(f"baseEmission = {self.baseEmission} & " +
                  f"targetReductionEmission = {self.targetReductionEmission} & " +
                  f"targetEmission = {self.targetEmission}")

        self.start_year = np.datetime64(
            self.df3.T.filter(like=str(self.baseEmissionYear)).columns.values[0], 'ns')
        self.end_year = np.datetime64(
            f"{self.targetEmissionReductionYear}-{str(self.start_year)[5:7]}-{str(self.start_year)[8:10]}",
            'ns')

        self.start_value, self.end_value = self.baseEmission, self.targetEmission

        if returnElements:
            return self


    def get_fig(
            self,
            figMode: str = 'markers') -> 'GHGEmissionsBaseVsTargetUsingLSEGWorkspaceClass':
        """
        Generate a plotly figure showing the GHG emissions data and the exponential decay to the target.

        Parameters:
        - figMode (str, default='markers'): The mode for the plotly scatter plot ('markers', 'lines', 'lines+markers').

        Returns:
        - GHGEmissionsBaseVsTargetUsingLSEGWorkspaceClass: The instance of the class with the generated figure.

        Example:
        >>> ghg = GHGEmissionsBaseVsTargetUsingLSEGWorkspaceClass(
        ...     RIC="PUREHEALTH.AD",  # "SHEL.L",
        ...     printMessage=False, keepMessageObject=True, source='LSEG',
        ...     scope="Scope 1 and 2", fiscalYear=2019)
        >>> setDictOfLists, dfOfSets, _dfOfSets = ghg.get_list_of_sets_that_contain_our_scope(
        ...     fiscalYear=ghg.fiscalYear, RIC=ghg.RIC, term=ghg.term, scope=ghg.scope)
        >>> result = ghg.get_data(fiscalYear=ghg.fiscalYear, returnElements=True, set=3)
        >>> result.get_fig().fig.show()
        """
        logging.info(f"Generating figure with figMode={figMode}")
        
        # Calculate the decay constant 'k' for exponential decay
        # We use the formula for exponential decay: y(t) = y0 * exp(-k * (t - t0))
        # Solve for 'k' using y(t) at the end point
        if self.end_value / self.start_value == 0:
            k = -np.log(0.01) / (
                int(str(self.end_year)[:4]) - int(str(self.start_year)[:4]))  # Use a small value instead of zero to avoid log(0) issue
        else:
            k = -np.log(self.end_value / self.start_value) / (
                (self.end_year - self.start_year).astype('timedelta64[Y]').astype(float))

        # Generate years and corresponding values using exponential decay
        self.years = np.arange(
            self.start_year.astype('datetime64[Y]').astype(int) + 1970,
            self.end_year.astype('datetime64[Y]').astype(int) + 1971)
        self.values = self.start_value * np.exp(
            -k * (self.years - (self.start_year.astype('datetime64[Y]').astype(int) + 1970)))

        self.decayDf = pd.DataFrame(
            data=self.values,
            index=self.years,
            columns=[
                f'Exponential Decay to target of {int(self.targetEmission)}'])
        self.df6 = pd.concat(
            [self.df5, self.decayDf])

        # Create the plot

        self.fig = go.Figure()

        # Add the exponential decay line trace
        self.fig.add_trace(
            go.Scatter(
                x=self.df6.index,
                y=self.df6.replace({pd.NA: np.nan})[f'Exponential Decay to target of {int(self.targetEmission)}'],  # You need to replace <NA> with np.nan in the entire DataFrame using `replace({pd.NA: np.nan})` because <NA> is a special missing value indicator used by pandas, which may not be compatible with some plotting libraries or other operations that expect np.nan for missing values. Converting <NA> to np.nan ensures consistency and compatibility.
                mode=figMode,
                name=f'Exponential Decay to target of {int(self.targetEmission)}'))

        # Set plot title and labels
        self.fig.update_layout(
            title=str(
                f"{self.RIC} GHG Emissions scope " +
                f"{self.scope}, set {self.set}, " +
                f"term {self.term} & source {self.source} " +
                f"({self.baseEmissionYear} Base & " +
                f"{self.targetEmissionReductionYear} Target)"),
            xaxis_title="Year",
            yaxis_title="CO2 Tons",
            template="plotly_dark"  # Optional: You can use different templates, such as 'plotly', 'ggplot2', etc.
        )

        # Add the dots representing the actual CO2 data from df3
        self.fig.add_trace(go.Scatter(
            x=self.df6.index, 
            y=self.df6.replace({pd.NA: np.nan})[self.df6.filter(like=self.baseFieldOfInterest.upper()).columns[0]],  # You need to replace <NA> with np.nan in the entire DataFrame using `replace({pd.NA: np.nan})` because <NA> is a special missing value indicator used by pandas, which may not be compatible with some plotting libraries or other operations that expect np.nan for missing values. Converting <NA> to np.nan ensures consistency and compatibility.
            mode=figMode, 
            name='CO2 Equivalents Emission Total',
            marker=dict(color='grey', size=8)  # Customize the dot color and size
        ))

        logging.info("Figure generated successfully")

        return self

In [2]:
# ghg = GHGEmissionsBaseVsTargetUsingLSEGWorkspaceClass(
#     RIC="TTEF.PA",  # "SHEL.L",
#     printMessage=False, keepMessageObject=True, source='LSEG',
#     scope="Scope 3", fiscalYear=2022)
# setDictOfLists, dfOfSets, _dfOfSets = ghg.get_list_of_sets_that_contain_our_scope(
#     fiscalYear=ghg.fiscalYear, RIC=ghg.RIC, term=ghg.term, scope=ghg.scope)
# result = ghg.get_data(fiscalYear=ghg.fiscalYear, returnElements=True, set=3)
# result.get_fig().fig.show()

In [3]:
import lseg.data as ld
import IPython
import ipywidgets as widgets
from refinitiv_widgets import Tab, TabBar, Select, Button, Loader, TextFieldAutosuggest
%matplotlib inline

In [4]:
europeanRICs = ['MBWS.PA', 'AAL.L', 'EVNV.VI', 'LENV.VI', 'OBER.VI', 'AU.N', 'ABGV.VI', 'MOED.MI', 'GASI.MI', 'RHIM.L', 'ISP.MI', 'WBSV.VI', 'CIRI.MI', 'ACKB.BR', 'BEKB.BR', 'CEMI.MI', 'SCHA.OL', 'WBD.MI', 'CRDI.MI', 'EMBI.MI', 'STLAM.MI', 'COLR.BR', 'LOTB.BR', 'ECONB.BR', 'ITMI.MI', 'RECT.BR', 'GBLB.BR', 'KBC.BR', 'DANI.MI', 'TLIT.MI', 'RECI.MI', 'SOF.BR', 'SOLB.BR', 'SPMI.MI', 'LDOF.MI', 'TESB.BR', 'UCB.BR', 'BZU.MI', 'AAHG.F', 'GGS.BE', 'ATCOa.ST', 'BAYGn.DE', 'HOLMb.ST', 'PBBG.DE', 'ELUXb.ST', 'SEBa.ST', 'BYWGnx.DE', 'ERICb.ST', 'BEIG.DE', 'ADCG.F', 'HMb.ST', 'INDUa.ST', 'JM.ST', 'GBFG.DE', 'LATOb.ST', 'LUNDb.ST', 'NCCb.ST', 'RATOb.ST', 'SAABb.ST', 'SAND.ST', 'BOSSn.DE', 'SKAb.ST', 'SKFb.ST', 'SHBa.ST', 'BLHG.F', 'TRELb.ST', 'VOLVb.ST', 'BIJG.DE', 'STERV.HE', 'HUH1V.HE', 'KESKOB.HE', 'CBKG.DE', 'METSB.HE', 'OUT1V.HE', 'CONG.DE', 'DAMG.DE', 'LINDEX.HE', 'DBKGn.DE', 'DBANn.DE', 'LHAG.DE', 'DUEG.DE', 'EDD3_p.F', 'EIN_p.DE', 'EUKG_p.F', 'KWGG.DE', 'FPE3_p.DE', 'GILG.DE', 'HABAn.DE', 'HEIG.DE', 'HNKG_p.DE', 'HNDG.F', 'HOTG.DE', 'IURG.DE', 'SDFGn.DE', 'AROG.H', 'KWSG.DE', 'DEZG.DE', 'SKBG.DE', 'KSBG.DE', 'LEIG.DE', 'MUVGn.DE', 'NBHG.MU', 'PSHG_p.DE', 'TUI1n.DE', 'PWO.DE', 'RLVG.D', 'RHMG.DE', 'SAPG.DE', 'JCI.N', 'SIXG.DE', 'SZGG.DE', 'SZUG.DE', 'TEGG.DE', 'VAR1.DE', 'EONGn.DE', 'VOWG.DE', '2HR.DE', 'WUWGn.DE', 'ZILGn.DE', 'ACX.MC', 'CIEA.MC', 'AZK.MC', 'BBVA.MC', 'BKT.MC', 'NTGY.MC', 'IDR.MC', 'NHH.MC', 'ECR.MC', 'ANA.MC', 'ENOR.MC', 'ENC.MC', 'TUBA.MC', 'ASMI.AS', 'FAE.MC', 'FCC.MC', 'IBE.MC', 'MDF.MC', 'ACCP.PA', 'AIRP.PA', 'SAF.PA', 'CHBE.PA', 'BICP.PA', 'BOIR.PA', 'SAVEN.PA', 'BOUY.PA', 'DANO.PA', 'CAPP.PA', 'CBLP.PA', 'CARR.PA', 'SCOR.PA', 'ALKS.OQ', 'AXAF.PA', 'VCTP.PA', 'AM.PA', 'ZURN.S', 'ICAD.PA', 'ODET.PA', 'ESLX.PA', 'ESSF.PA', 'EURA.PA', 'FRVIA.PA', 'COVH.PA', 'FLYP.PA', 'FOUG.PA', 'CVO.PA', 'VIRI.PA', 'SGEF.PA', 'VIV.PA', 'SOGN.PA', 'GENC.PA', 'ALGEV.PA', 'GFII.PA', 'GFCP.PA', 'GRBT.PA', 'IMTP.PA', 'RUBF.PA', 'JCQ.PA', 'LOIM.PA', 'LECS.PA', 'OREP.PA', 'MANP.PA', 'MWDP.PA', 'BOLL.PA', 'MAUP.PA', 'MICP.PA', 'LAGA.PA', 'ALATI.PA', 'ALEXA.PA', 'PERP.PA', 'OPM.PA', 'ALPM.PA', 'PUBP.PA', 'AREIT.PA', 'ROBF.PA', 'TEPRF.PA', 'SGOB.PA', 'SCHN.PA', 'SEBF.PA', 'SOON.S', 'EXHO.PA', 'ATOS.PA', 'IMAF.PA', 'CMCL.A', 'TFFP.PA', 'TTEF.PA', 'VLLP.PA', 'AIRF.PA', 'VLOF.PA', 'VIRB.PA', 'DSCV.L', 'ALWA.L', 'AIBG.I', 'ALUG.L', 'ICGT.L', 'ANEA.L', 'AHT.L', 'PNL.L', 'GL9.I', 'AVON.L', 'BAB.L', 'BNKR.L', 'BIRG.I', 'BARC.L', 'BTRW.L', 'BAG.L', 'BWY.L', 'FOUR.L', 'BKGH.L', 'BALF.L', 'COA.L', 'BOY.L', 'BOOT.L', 'SPT.L', 'AON.N', 'BAES.L', 'AGT.L', 'BLND.L', 'BP.L', 'BT.L', 'GBGP.L', 'SINT.ST', 'BNZL.L', 'CNE.L', 'CLDN.L', 'CAME.L', 'CPI.L', 'CGT.L', 'CARRC.L', 'CGS.L', 'CHG.L', 'CKN.L', 'CBRO.L', 'CFX.L', 'AV.L', 'NTAS.L', 'VSVS.L', 'COSG.L', 'CWK.L', 'RSW.L', 'CRDA.L', 'CRPR.L', 'SDG.L', 'ASL.L', 'DLN.L', 'DWHT.L', 'DPLM.L', 'EDIN.L', 'DGN.L', 'ELCO.L', 'RS1R.L', 'SXS.L', 'GSCT.L', 'TT.N', 'FSJ.L', 'JAM.L', 'JFJ.L', 'JGGI.L', 'FCIT.L', 'FSTA.L', 'GFRD.L', 'GSK.L', 'GLEG.L', 'KBT.L', 'ZIG.L', 'GDWN.L', 'GRI.L', 'ITV.L', 'GPEG.L', 'EXPN.L', 'GRG.L', 'HGT.L', 'PANI.L', 'DGE.L', 'HLMA.L', 'JHD.L', 'HMSO.L', 'ELM.L', 'HAYS.L', 'HEAD.L', 'HLCL.L', 'MTO.L', 'HILS.L', 'HTG.L', 'IMI.L', 'INCH.L', 'BGFD.L', 'JSG.L', 'JMAT.L', 'KMR.L', 'KYGa.I', 'KSP.I', 'LAND.L', 'LTHM.L', 'LWDB.L', 'EOTE.L', 'LGEN.L', 'LSC.L', 'RNWH.L', 'MACF.L', 'MSLH.L', 'MRCH.L', 'APTD.L', 'MPAC.L', 'MNKS.L', 'MGAMM.L', 'MUT.L', 'MYI.L', 'PAGPA.L', 'BWNG.L', 'NXT.L', 'NICL.L', 'BRSC.L', 'UU.L', 'OXIG.L', 'PHI.L', 'PZC.L', 'PSON.L', 'PINE.L', 'PSN.L', 'MEGPM.L', 'PMGR.L', 'PORV.L', 'VANQ.L', 'PRU.L', 'VOD.L', 'RAT.L', 'REAH.L', 'RKT.L', 'REL.L', 'RTO.L', 'RCDO.L', 'RCP.L', 'ROR.L', 'MPE.L', 'NWG.L', 'RIO.L', 'DLAR.L', 'MGNS.L', 'SGE.L', 'SBRY.L', 'SVS.L', 'SDR.L', 'SAIN.L', 'FGT.L', 'SMT.L', 'STVG.L', 'ARBB.L', 'SNR.L', 'SRP.L', 'SVT.L', 'SFR.L', 'RWI.L', 'SHI.L', 'BGS.L', 'SGRO.L', 'SMWH.L', 'SMDS.L', 'SMIN.L', 'PNN.L', 'SPX.L', 'STAN.L', 'MTC.L', 'SUS.L', 'TATE.L', 'TW.L', 'TMPL.L', 'TSCO.L', 'MRCM.L', 'TFW.L', 'THRG.L', 'TPK.L', 'TET.L', 'TRY.L', 'HSL.L', 'LLOY.L', 'TLW.L', 'ULVR.L', 'AVG.L', 'VP.L', 'VCP.L', 'VIDV.L', 'VLX.L', 'WEIR.L', 'WTB.L', 'MARS.L', 'WPP.L', 'YNGa.L', 'SYNTS.L', 'CTY.L', 'ARBNO.S', 'GAMH.S', 'BLKB.S', 'BELL.S', 'OERL.S', 'CALN.S', 'GAV.S', 'DAE.S', 'EMSN.S', 'CFR.S', 'CFT.S', 'GF.S', 'FHZN.S', 'FORN.S', 'APGN.S', 'GURN.S', 'HOLN.S', 'ISN.S', 'JFN.S', 'KARN.S', 'VATN.S', 'LEHN.S', 'LISN.S', 'METN.S', 'MIKN.S', 'NESN.S', 'OFN.S', 'PMN.S', 'RIEN.S', 'SCHP.S', 'SWTQ.S', 'SFZN.S', 'SIKA.S', 'SUN.S', 'TECN.S', 'REHN.S', 'VPBN.S', 'VETN.S', 'VONN.S', 'ZEHN.S', 'BYS.S', 'ROCKb.CO', 'MDT.N', 'FREG.DE', 'SVEG.OL', 'CCEP.OQ', 'PDL.L', 'BKOM.PR', 'CEZP.PR', 'GAZP.MM', 'ALBAV.HE', 'CTY1S.HE', 'KOG.OL', 'ALMB.CO', 'BO.CO', 'GABR.CO', 'COLOb.CO', 'MAERSKb.CO', 'DANSKE.CO', 'DFDS.CO', 'GN.CO', 'ALKb.CO', 'SPGP.CO', 'JYSK.CO', 'RBREW.CO', 'TIV.CO', 'NKT.CO', 'NOVOb.CO', 'PAALb.CO', 'RILBA.CO', 'FLS.CO', 'AUDK.LU', 'SGSN.S', 'AALB.AS', 'BAMN.AS', 'CORB.AS', 'ETN.N', 'HEIO.AS', 'HEIN.AS', 'SBMO.AS', 'PHG.AS', 'KRKG.LJ', 'SVEL.AS', 'TWKNc.AS', 'GBU.V', 'BERGb.ST', 'UIE.CO', 'BMWG.DE', 'SAN.MC', 'BOLJ.ST', 'EMII.MI', 'SQZ.L', 'ALB.MC', 'PNR.N', 'JET2.L', 'RM.L', 'FSV.L', 'TEF.MC', 'NRC.OL', 'BONHR.OL', 'ATEA.OL', 'DNO.OL', 'NHY.OL', 'ORK.OL', 'ODF.OL', 'OLT.OL', 'TOM.OL', 'STB.OL', 'VEI.OL', 'WWI.OL', 'CECG.DE', 'ECORE.L', 'AEGN.AS', 'FNMI.MI', 'JMG.L', 'FSKRS.HE', 'SOFb.ST', 'MOLB.BU', 'HBR.L', 'PEABb.ST', 'JLEC.L', 'MTHH.CO', 'WLSNc.AS', 'GNC.L', 'AMSU.L', 'RNK.L', 'IPAR.PA', 'MDBI.MI', 'RROS.ST', 'MCG.L', 'BALN.S', 'NOKIA.HE', 'OHLA.MC', 'ACS.MC', 'OEMb.ST', 'OLVAS.HE', 'ORES.ST', 'TRIA.PA', 'YIT.HE', 'AMRN.OQ', 'RWEG.DE', 'TRIB.OQ', 'AD.AS', 'SIEGn.DE', 'ALGIL.PA', 'SNBN.S', 'VIK1V.HE', 'INRN.S', 'TCFP.PA', 'KOMN.S', 'TKAG.DE', 'SNI.OL', 'UNPI.MI', 'VBGb.ST', 'WALLb.ST', 'WEHA.AS', 'JD.L', 'BOGr.AT', 'NORTHM.CO', 'RCSM.MI', 'COFB.BR', 'CGDM.PA', 'ENEA.ST', 'ENI.MI', 'TIGO.OQ', 'LPKG.DE', 'RAYb.ST', 'EAH.L', 'DTEGn.DE', 'FEV.L', 'INHG.DE', 'DEMANT.CO', 'VAIAS.HE', 'BOPr.AT', 'ASPO.HE', 'SCHO.CO', 'UBIP.PA', 'CAF.MC', 'MMKV.VI', 'GCO.MC', 'TIETO.HE', 'VIEV.VI', 'BORY.WA', 'ARYN.S', 'HKFOODS.HE', 'ZTF.L', 'ADEN.S', 'AKZO.AS', 'ALVG.DE', 'CPINV.BR', 'BEIAb.ST', 'AFRY.ST', 'MUL.L', 'BASFn.DE', 'INRr.AT', 'HRMr.AT', 'MFEB.MI', 'BCP.LS', 'CORA.LS', 'JMT.LS', 'SJP.L', 'AIRE.S', 'TRFT.L', 'AZN.L', 'VTYV.L', 'AMBUb.CO', 'JEDT.L', 'RCOP.PA', 'NTZ.N', 'SCAb.ST', 'ZUBN.S', 'PV.DE', 'KRNG.DE', 'VID.MC', 'BPSI.MI', 'CRIP.PA', 'UMI.BR', 'SSE.L', 'G1AG.DE', 'HBMG.H', 'HUFVa.ST', 'CB.N', 'SDY.L', 'CARLb.CO', 'ANTO.L', 'BAR.BR', 'ANSY.L', 'SATG.DE', 'SIM0.F', 'CFEB.BR', 'RAND.AS', 'JUNG_p.DE', 'MKS.L', 'GFTG.DE', 'FEML.L', 'RHKG.DE', 'STP.WA', 'MCB.L', 'VOSG.DE', 'TEM.L', 'VERB.VI', 'UNIQ.VI', 'OKEAC.CO', 'LAEP.PA', 'CASP.PA', 'URW.PA', 'COL.MC', 'SPNO.CO', 'MCM.MC', 'SOPR.PA', 'MLPG.DE', 'YSO.LS', 'BILIa.ST', 'MBHG_p.F', 'ACBr.AT', 'NBGr.AT', 'LVMH.PA', 'PEUG.PA', 'VIBG_p.DE', 'DIOR.PA', 'ALPJT.PA', 'SDGI.PA', 'AGRV.VI', 'TRNG.PA', 'WINEW.L', 'IMB.L', 'ABI.BR', 'CATb.ST', 'GFTU_u.L', 'PRGO.N', 'INVEb.ST', 'BIOG.DE', 'BDTG.DE', 'PCT.L', 'UZU.DE', 'BEAN.S', 'BBN.S', 'BION.S', 'NOVN.S', 'NETC.L', 'VIS.MC', 'KGF.L', 'CNA.L', 'CLASb.ST', 'VOPA.AS', 'ECMPA.AS', 'SN.L', 'BABr.AT', 'PXMP.WA', 'BATS.L', 'CPG.L', 'EXCr.AT', 'FAG.ST', 'ROTLV.BX', 'IMP.BX', 'EM.MI', 'HUBN.S', 'LECN.S', 'AFK.OL', 'DNB.OL', 'REDE.MC', 'IF.MI', 'STRO.OL', 'WKB.S', 'EKTAb.ST', 'KINVb.ST', 'GES.N', 'MSONb.ST', 'NTEA_p.L', 'HRI.L', 'FIA1S.HE', 'NG.L', 'MOZN.S', 'LAT1V.HE', 'WRT1V.HE', 'TEL2b.ST', 'FUGR.AS', 'IPX.L', 'VLRT.S', 'EDV.TO', 'BOS.S', 'SSABa.ST', 'KLR.L', 'FMEG.DE', 'DSV.CO', 'FLUGb.CO', 'HHDC.CO', 'ATT.L', 'VJBA.CO', 'SOLARb.CO', 'SYDB.CO', 'ATRAV.HE', 'OMVV.VI', 'DBG.PA', 'SWVk.H', 'EPEND.ST', 'HEXAb.ST', 'ELANb.ST', 'XANOb.ST', 'NOLAb.ST', 'SECUb.ST', 'BEIJb.ST', 'FABG.ST', 'ELE.MC', 'CONTX.OL', 'SCYR.MC', 'AFGA.OL', 'RTX.CO', 'QGEN.N', 'RR.L', 'TOWNT.L', 'RCH.L', 'ESCT.L', 'TTG.L', 'HWDN.L', 'COP1n.DE', 'STLN.S', 'HSBA.L', 'FGP.L', 'PUMG.DE', 'CAIV.VI', 'BRUN.AS', 'STOG_p.DE', 'SWEDa.ST', 'EURBr.AT', 'PSG.MC', 'ASCN.S', 'MAP.MC', 'REP.MC', 'LIO.L', 'JDW.L', 'OXB.L', 'PROFb.ST', 'AVANZ.ST', 'IWG.L', 'ASML.AS', 'GMMG.DE', 'DAL.MI', 'GETIb.ST', '1VUB02AE.BV', 'BRBI.MI', 'ALPN.S', 'BURE.ST', 'IGRI.L', 'SEM.LS', 'ADNGk.DE', 'SLNCF.PK', 'HLEE.S', 'EDP.LS', 'LKOH.MM', 'SOR.OL', 'BETSb.ST', 'COREa.ST', 'DURCb.ST', 'WKP.L', 'BRWM.L', 'NTGNT.CO', 'VCTX.L', 'RSL2.DE', 'ITPG.MI', 'DNORD.CO', 'MOWI.OL', 'COTNE.S', 'MEL.MC', 'DOKA.S', 'EXEP.PA', 'GIMV.BR', 'NVGR.LS', 'DAST.PA', 'TATN.MM', 'MTEL.BU', 'AFLT.MM', 'CWCG.DE', 'HRMS.PA', 'IFCN.S', 'MT.AS', 'CLN.S', 'JENGn.DE', 'MERG.L', 'HWG.L', 'FER.MC', 'IETB.BR', 'OLYr.AT', 'GAW.L', 'TELIA.ST', 'SUBC.OL', 'RAP1V.HE', 'VITR.ST', 'NETIb.ST', 'SGCG.DE', 'MOTA.LS', 'NOS.LS', 'LLBN.S', 'UHR.S', 'MBGn.DE', 'HELr.AT', 'NIBEb.ST', 'LOGN.S', 'MILP.WA', 'CCT.L', 'KIE.L', 'FORTUM.HE', 'EMG.L', 'ADVG.DE', 'PRTP.PA', 'MYCR.ST', 'DORO.ST', 'INGP.WA', 'BIOGb.ST', 'MBK.WA', 'WSU.DE', 'KNOW.ST', 'WWH.L', 'PHARP.L', 'ORAN.PA', 'BDXP.WA', 'SBER.MM', 'SPL1.WA', 'BESI.AS', 'MRFr.AT', 'BCGE.S', 'OTPB.BU', 'NWF.L', 'ARDS.AS', 'BNPP.PA', 'PHRA.LS', 'SMPV.VI', 'PON1V.HE', 'AKW.PA', 'HELN.S', 'SDP.L', 'SABT.PA', 'GHH.L', 'SYZG.DE', 'SBOE.VI', 'KNIN.S', 'OTEr.AT', 'MMTP.PA', 'ERMT.PA', 'CCP.L', 'UTDI.DE', 'DIAL.L', 'MING.OL', 'NONG.OL', 'SB1NO.OL', 'MEDIVIR.ST', 'ASSAb.ST', 'PNEGn.DE', 'ALSN.S', 'QDT.PA', 'ITERA.OL', 'SOIT.PA', 'TYRES.HE', 'AMPF.MI', 'NEKG.DE', '1TAT01DE.BV', 'MXHNn.DE', 'KPN.AS', 'DCC.L', 'BLPU.L', 'ICGIN.L', 'PSMGn.DE', 'KBHL.CO', 'BOND.PA', 'CEMB.MI', 'PACT.ST', 'YSNG.DE', 'SRSr.AT', 'PRICb.ST', 'PRS.MC', 'AXFO.ST', 'SKISb.ST', 'TLT1V.HE', 'IQTr.AT', 'ATRLJb.ST', 'CPRI.MI', 'JII.L', 'PHP.L', 'CLSH.L', 'MRCG.DE', 'CRH.N', 'CTTS.ST', 'III.L', 'GFRN.MI', 'MTGb.ST', 'RAUTE.HE', 'ABF.L', 'CITT.PA', 'SLTG.H', 'KEMIRA.HE', 'IVUG.DE', 'BMPS.MI', 'ERG.MI', 'CAST.ST', 'GDRB.BU', 'HLGZ.DE', 'AIXGn.DE', 'HDDG.DE', 'BAVA.CO', 'TTKG.DE', 'SAMPO.HE', 'FIEG.DE', 'STDM.PA', 'LMDr.AT', 'MYTr.AT', 'SECTb.ST', 'NEL.OL', 'PSAGn.DE', 'WIL.L', 'BITI.PA', 'BMA.OL', 'HEX.OL', 'AMDG.DE', 'AVQ.PA', 'SANOMA.HE', 'NOD.OL', 'FUTR.L', 'MSNG.MM', 'RTKM.MM', 'PKTM.S', 'UPM.HE', 'XAR.L', 'EVTG.DE', 'MUMG.DE', 'VWS.CO', 'TEMN.S', 'PTSB.I', 'VOES.VI', 'LTEN.PA', 'CHCH.L', 'HNRGn.DE', 'IMFI.VI', 'STMPA.PA', 'CDAF.PA', 'KCRA.HE', 'SNGS.MM', 'RENA.PA', 'ADSGn.DE', 'PRSO.OL', 'SOLS.MI', 'EUFI.PA', 'STGN.S', 'CCHE.PA', 'ALV.N', 'ADDVa.ST', 'ICP1V.HE', 'GLTN.PA', 'A2.MI', 'EUZG.DE', 'TTRGn.DE', 'BSLG.DE', 'PHAR.AS', 'A1OS.DE', 'BRMS.L', 'BHW.WA', 'TRACb.ST', 'PEO.WA', 'RYA.I', 'CHMF.MM', 'URKAI.RTS', 'FRO.N', 'KGH.WA', 'MEKKO.HE', 'PEHN.S', 'BITTI.HE', 'ERST.VI', 'ZIM.AX', 'KIT.OL', 'AGFB.BR', 'EXA.PA', 'SIBN.MM', 'INF.L', 'ACR.OL', 'CSHG.DE', 'TAGMb.ST', 'OPL.WA', 'OMG1.L', '1U1.DE', 'REY.MI', 'NEWAb.ST', 'ICLR.OQ', 'MLXS.BR', 'GREENY.BR', 'EBKG.DE', 'VIE.PA', 'ADVN.S', 'STRAX.ST', 'JLTM.ST', 'ALSO.PA', 'NSTEc.AS', 'IARb.ST', 'SAGAa.ST', 'TGS.OL', 'SWECb.ST', 'WITH.HE', 'VELD.BR', 'SVEDb.ST', 'EWI.L', 'CCC.L', 'COLUM.CO', 'GTN.WA', 'SMHNn.DE', 'PYHE.PA', 'ATSV.VI', 'KIPO.BR', 'LONN.S', 'KBCA.BR', 'BALDb.ST', 'RETE.BR', 'FTON.S', 'RWA.L', 'ENRO.ST', 'GNS.L', 'BARN.S', 'ACE.MI', 'MOMENT.ST', 'OPUSG.BU', 'GSC1n.DE', 'ASY.PA', 'ACWNn.DE', 'EXL1V.HE', 'CICN.S', 'HAWG.DE', 'ALSOG.PA', 'ECVG.DE', 'ISOS.PA', 'STMN.S', 'TPT.L', 'DATA.L', 'SBSG.DE', 'COKG.DE', 'BIOT.ST', 'EBRO.MC', 'TSL.MI', 'KUD.S', 'AUBT.PA', 'EIC.S', 'BANB.S', 'EMBLA.CO', 'BGBN.PA', 'MVVGn.DE', 'NAFG.DE', 'CURN.S', 'SESFd.PA', 'PTNL.AS', 'ANODb.ST', 'DEQGn.DE', 'RAAG.DE', 'SISG.DE', 'BRIMH.IC', 'HEPr.AT', 'ABBN.S', 'PREVb.ST', 'SASY.PA', 'ACPP.WA', 'IBAB.BR', 'FINGb.ST', 'DOCO.VI', 'PALF.VI', 'NFGN.L', 'ACN.N', 'IOMG.L', 'BCNT.MI', 'CPE.S', 'DE10k.H', 'SURG.DE', 'SCMN.S', 'NXUG.DE', 'CURY.L', 'AFXG.DE', 'RWS.L', 'GMKN.MM', 'OBEL.BR', 'BOL.ST', 'BTSb.ST', 'AQ.ST', 'TGTG.DE', 'MZXG.DE', 'AEVS.S', 'ELISA.HE', 'CAI.MI', 'ELGG.DE', 'AIR.PA', 'GEBN.S', 'IQE.L', 'RIG.N', 'UTG.L', 'CLLN.L', 'TPEG.DE', 'PAFR.L', 'KTN.DE', 'WDPP.BR', 'VITb.ST', 'ALMT.ST', 'ROG.S', 'VLAN.AS', 'ELEN.MI', 'EPH.S', 'CARM.PA', 'ENEI.MI', 'DIGIA.HE', 'EZJ.L', 'AUTr.AT', 'TEP.L', 'VANTI.PA', 'BC8G.DE', 'SOLTEQ.HE', 'BINV.ST', 'DOM.L', 'MSABb.ST', 'PKN.WA', 'TGHN.DE', 'NDAFI.HE', 'PREC.ST', 'ROBRD.BX', 'LPPP.WA', 'SABE.MC', 'IFXGn.DE', 'THW.ASE', 'WATRI.L', 'ACSO.L', 'SHEP.ASE', 'MSGL.H', 'STZEb.ST', 'WYWYN.L', 'IREE.MI', 'EVDG.DE', 'LSEG.L', 'CPHN.S', 'KOF.PA', 'EYDr.AT', 'SGKN.S', 'TLY.L', 'SQN.S', 'HNLG.DE', 'GMAB.CO', 'DHLn.DE', 'BYG.L', 'ZOO.L', 'EBQ.L', 'SGG.ST', 'ALLN.S', 'PSPN.S', 'SHFG.DE', 'QBYn.DE', 'SPSN.S', 'AEIGn.DE', 'MEKO.ST', 'SWEGn.DE', 'GIVN.S', 'SFPI.PA', 'GLJn.DE', 'ETTE.HE', 'G2MNO.OL', 'LSG.OL', 'IDOX.L', 'CCL.L', 'WDIG.H', 'NSISb.CO', 'ANDR.VI', 'TXGN.S', 'TCAPI.L', 'BVB.DE', 'PGNG.DE', 'DB1Gn.DE', 'EQNR.OL', 'BSS.MI', 'DLG.MI', 'TELA.VI', 'GRMN.N', 'PGH.L', 'MEDG.DE', 'TEL.OL', 'ITX.MC', 'TRBG.L', 'PAGE.L', 'OHBG.DE', 'SVIK.ST', 'GLA1V.HE', 'OPAr.AT', 'NEXS.PA', 'FRAG.DE', 'WTW.OQ', 'PPET.L', 'MCHN.S', 'MORr.AT', 'PRT.MI', 'JCDX.PA', 'IXXG.DE', 'LUKN.S', 'KOA.OL', 'DEHr.AT', 'AKRN.MM', 'BUCN.S', 'ROSNP.BX', 'ADDTb.ST', 'LAGRb.ST', 'ORRON.ST', 'ASOS.L', 'SUY1V.HE', 'BILL.ST', 'CAGR.PA', 'SRG.MI', 'ALAPH.L', 'JUVE.MI', 'TRNF_p.MM', 'AGES.BR', 'CDR.WA', 'IEM.L', 'FDPF.L', 'EMEIS.PA', 'ALFA.ST', 'ITRK.L', 'BAMI.MI', 'WG.L', 'EKF.L', 'INTRUM.ST', 'NOBI.ST', 'ENAG.MC', 'BRBY.L', 'INVP.L', 'JLP.L', 'SLHN.S', 'ENWE.L', 'KURN.S', 'BEZG.L', 'TENR.MI', 'EDNn.MI', 'BCVN.S', 'FIMI.MI', 'ANCR.L', 'JDG.L', 'MAB.L', 'IHG.L', 'BRNKn.DE', 'REJLb.ST', 'EXMR.BR', 'HRA.MI', 'OLPr.AT', 'JHX.AX', 'VOF.L', 'MTSS.MM', 'IPO.L', 'A3M.MC', 'NLMK.MM', 'NAS.OL', 'APIA.L', 'JHG.N', 'MTLR.MM', 'MAGN.MM', 'PROX.BR', 'CAU.L', 'OTEC.OL', 'BSLN.S', 'YAR.OL', 'AKAST.OL', 'GTCP.WA', 'AMS.S', 'GCC.L', 'CSN.L', 'MBH.L', 'ITAB.ST', 'HFD.L', 'TRN.MI', 'IVSO.ST', 'AFRNA.L', 'ITM.L', 'BIOX.PA', 'NOTE.ST', 'LOCAL.PA', 'AZMT.MI', 'RICA_p.L', 'SAA.L', 'NCCG.L', 'GRLS.MC', 'PGRm.BA', 'SREI.L', 'PFD.L', 'ACT1.DE', 'PBGG.WA', 'NANON.L', 'COTE.BX', 'OSPGk.H', 'GOODG.L', 'AKER.OL', 'YPSN.S', 'PAYP.L', 'ADML.L', 'BEG.L', 'AGY.L', 'NEXI.PA', 'NVTK.MM', 'IS7.DE', 'RSTP.L', 'PROB.ST', 'CGCBV.HE', 'EMPR.L', 'PKO.WA', 'GEO.MI', 'AFKS.MM', 'CWR.L', 'NEUP.WA', 'STAF.L', 'EMMN.S', 'CMBT.BR', 'CCCP.WA', 'MAIH.L', 'ENT.L', 'BORG.ST', 'ARTG.DE', 'LXSG.DE', 'FPIP.ST', 'ALMAC.HE', 'IGD.MI', 'EUR.WA', 'ALSS.LS', 'BRK.L', 'DSX.N', 'DESN.S', 'RBIV.VI', 'AURIM.ST', 'NESTE.HE', 'PCA.L', 'YOU.L', 'NDXG.DE', 'ATYM.L', 'GLPG.AS', 'SRB.L', 'EVE.S', 'IGG.L', 'FIVEDR.MM', 'SOU.L', 'MTXGn.DE', 'WIHL.ST', 'SPA.L', 'TOM2.AS', 'AAK.ST', 'KNEBV.HE', 'ESNT.L', 'CMPR.OQ', 'ELI.BR', 'VIGR.VI', 'MARR.MI', 'STORYb.ST', 'MOBN.S', 'ENGIE.PA', 'INDT.ST', 'ALFRE.PA', 'SHEL.L', 'SRC.L', 'WBS_p.L', 'TRMK.MM', 'GASS.OQ', 'QCEG.H', 'IMCD.AS', 'ELFV.MM', 'EFGN.S', 'AMUN.PA', 'MERY.PA', 'EVOK.L', 'TRYG.CO', 'PFC.L', 'ETL.PA', 'STEMS.L', 'HIK.L', 'ORX.ST', 'PCTN.L', 'HBMN.S', 'DKSH.S', 'TIP.MI', 'GENOb.ST', 'HASE.L', 'IPN.PA', 'E5T.MI', 'AVOL.S', 'S30.PA', 'DRX.L', 'VIVE.ST', 'TX.N', 'QQ.L', 'VU.PA', 'CEVI.ST', 'HUSQb.ST', 'CHRT.L', 'PGHN.S', 'POST.VI', 'RMV.L', 'IMPN.S', 'PATGn.DE', 'LEGD.PA', 'PIA.MI', 'PTEC.L', 'WCHG.DE', 'DWNG.DE', 'TMGT.L', 'CATE.ST', 'RECSI.OL', 'HICL.L', 'F3CG.DE', 'AKE.PA', 'ZUMV.VI', 'KCOGn.DE', 'CLIG.L', 'MBBG.DE', 'MGNT.MM', 'HTGR.L', 'DIOS.ST', 'V6CG.DE', 'ADP.PA', 'OKDBV.HE', 'IES.L', 'VLTSA.PA', 'CMCX.L', 'BCHN.S', 'WACGn.DE', 'B5A0n.H', 'PLZL.MM', 'D6HG.DE', 'LNA.PA', 'O5G.DE', 'CLTN.S', 'HOCN.S', 'METSO.HE', 'TRE.MC', 'AR40k.H', 'OXUR.BR', 'IPH.PA', 'ROSN.MM', 'ORNBV.HE', 'AUSS.OL', 'PROT.OL', 'GEA1.WA', 'ABDN.L', 'UPRO.MM', 'O4BG.DE', 'DMRE.DE', 'BGN.MI', 'SNWS.L', 'CE2G.H', 'SOBIV.ST', 'M5ZG.F', 'M4N.H', 'ASHM.L', 'RENE.LS', 'VBKG.DE', 'DAC.N', 'MONTE.BR', 'DNLM.L', 'TLSG.LJ', 'ESUN.S', 'G5EN.ST', 'AOO.BR', 'B7EG.DE', 'GALP.LS', 'AKVA.OL', 'HOCM.L', 'RASP.MM', 'SANN.S', 'MBTN.S', 'AER.N', 'SY1G.DE', 'INPP.L', 'NWOn.H', 'NTGG.DE', 'BEGR.ST', 'CLARI.PA', 'FPHG.DE', 'LSL.L', 'NWRN.S', 'ASCI.MI', 'CHEMM.CO', 'LIAB.ST', 'VTU.L', 'GNFT.PA', 'ICEAIR.IC', 'AKRBP.OL', 'ALCLS.PA', 'GEMD.L', 'ALISP.PA', 'AOXG.DE', 'EVKn.DE', 'SAFE.L', 'H9YG.H', 'COPN.S', 'VZN.S', 'FRAS.L', 'FNTGn.DE', '3IN.L', 'BHMG.L', 'JAZZ.OQ', 'SW.N', 'TEL.N', 'RXL.PA', 'AFEN.L', 'BOUV.OL', 'SALM.OL', 'VTBR.MM', 'GXIG.DE', 'B7C.MI', 'HRGV.L', 'PRY.MI', 'SRET.L', 'ADXN.S', 'GSFG.OL', 'NMAN.ST', 'NAXS.ST', 'FNOX.ST', 'HFG.L', 'ARGAN.PA', 'MOL.MI', 'ZV.MI', 'OGN.I', 'ST5G.DE', 'KTCG.VI', 'FXPO.L', 'VLS.PA', 'SLRS.MC', 'ALM.MC', 'MNDI.L', 'FOBANK.CO', 'IPF.L', 'AGNr.AT', 'HPOLb.ST', '70TD.L', 'ETGG.DE', 'DIAS.MI', 'SFQ.DE', 'MONY.L', 'GETP.PA', 'PPH.L', 'AEF.MI', 'HHFGn.DE', 'AMG.AS', 'BVI.PA', '2GBG.DE', 'NXR.L', 'CRADb.ST', 'NBPE.L', 'ENDUR.OL', 'HP3An.DE', 'HNSA.ST', 'CRW.L', 'HYQGn.DE', 'DOTD.L', 'CTNGk.F', 'RAILG.ST', 'FAGRO.BR', 'CABK.MC', 'CVSG.L', 'SYSR.ST', 'NYR.BR', 'UBXN.S', 'TRCS.L', 'LSRG.MM', 'MVID.MM', 'FLUI.MC', 'EAST9.ST', 'HMSN.ST', 'STRV.VI', 'DUNI.ST', 'MTCM.MI', 'TENr.AT', 'LMPL.L', 'FQT.DE', 'VGP1.BR', 'ROVI.MC', 'HVPEa.L', 'KER.WA', 'RECL.L', 'ORSTED.CO', 'HYDR.MM', 'PHNX.L', 'ODHN.S', 'WBI.L', 'SEZI.ST', 'EDPR.LS', 'EWRK.ST', 'CPS.WA', 'ZO1G.H', 'MHPCq.L', 'GKS0n.H', 'SB.N', 'BRKN.S', 'BREE.L', 'MUXG.DE', 'HTRO.ST', 'S92G.DE', 'SAGS.L', 'ATTP.WA', 'FEES.MM', 'IRAO.MM', 'GSL.N', 'GMLJ.J', 'BEZ.DE', 'ABLI.ST', 'LWBP.WA', 'REIT.LU', 'LOOMIS.ST', 'CLOEb.ST', 'SDIS.L', 'ENAE.WA', 'EXOR.AS', 'NRST.MI', 'ACROUD.ST', 'FTKn.DE', 'NRRT.L', 'GSJ.MC', 'ASPS.OQ', 'RLFB.S', 'BAER.S', 'AKTIA.HE', 'BURF.L', 'HACGn.DE', 'PGE.WA', 'RUAL.MM', 'PZU.WA', 'TPE.WA', 'ALAGR.PA', 'STNG.N', 'BAKKA.OL', 'YRM.MI', 'BNRGn.DE', 'ARISE.ST', 'CPPG.L', 'ABS.PA', 'ASRNL.AS', 'ORON.S', 'ENQ.L', 'FCSS.L', 'NXPI.OQ', 'AMA.MC', 'GIBG.H', 'LYB.N', 'BPOST.BR', 'SHCS.L', 'JSW.WA', 'BMAX.ST', 'MONC.MI', '1913.HK', 'EDEN.PA', 'JUP.L', 'DRWG.DE', 'SAXG.DE', 'WAWI.OL', 'OCDO.L', 'GCPI.L', 'MX.N', 'BGBSE.BB', 'PKPP.WA', 'PNDORA.CO', 'EPISb.ST', 'CAML.L', 'CMRE.N', 'PEAN.S', 'OKEYDR.MM', 'GPW.WA', 'VKCO.MM', 'KRU.WA', 'ZELA.CO', 'HLAG.DE', 'GJFG.OL', 'HUMR.L', 'NELLY.ST', 'TLXGn.DE', 'APAM.AS', 'AGRO.N', 'CDONAB.ST', 'ICAG.L', 'KANC.ST', 'DIDA.MC', 'NOEJ.DE', 'AGRODR.MM', 'DISTIT.ST', 'ENGP.WA', 'KDEV.ST', 'AMBEA.ST', 'ETLNDR.MM', 'SFER.MI', 'DEDIC.ST', 'IDLA.PA', 'NBIS.OQ', 'ODLO.OL', 'AUTON.S', 'BULTEN.ST', 'GLEN.L', 'SRENH.S', 'SIVEH.ST', 'APTV.N', '1910.HK', 'LTGL.L', 'GENL.L', 'ENZY.ST', 'SW10n.H', 'BOUL.ST', 'SES.MI', 'PHOR.MM', 'ALRS.MM', 'VERVE.ST', 'MOLN.S', 'CBOM.MM', 'BBGIB.L', 'HLE.DE', 'DOVA.MI', 'PST.MI', 'STBS.L', 'CVTEC.ST', 'FDJ.PA', 'EVRE.L', 'MFONI.RTS', 'INSEI.L', 'MAST.BU', 'CPRI.N', 'BKWB.S', 'CRWEN.NGM', 'HFR.AX', 'SPIE.PA', 'SCANFL.HE', 'SSPG.L', 'BCU.MI', 'ALRR.WA', 'BGEO.L', 'DBV.PA', 'ALLIX.PA', 'GYC.DE', 'DOCM.S', 'DLGD.L', 'SBOS.OL', 'MANU.N', 'ZUGN.S', 'MOEX.MM', 'WPPL.WA', 'AB9.DE', 'BRGB.OL', 'SYNCS.L', 'XVIVO.ST', 'O2Dn.H', 'SIILI.HE', 'MDMG.MM', 'LEON.S', 'EIMS.IC', 'IDSI.L', 'ATCO_pd.N', 'SYN.IC', 'HVO.L', 'PRTA.OQ', 'LEGn.DE', 'OCI.AS', 'UKWG.L', 'CRST.L', 'THRLT.L', 'TFIF.L', 'OSRn.H', 'RCN.L', 'NXQ.L', 'IMMUN.ST', 'PHXM.PA', 'SFSN.S', 'CCH.L', 'FNAC.PA', 'CSTM.N', 'DMYDb.ST', 'ABDP.L', 'BSIF.L', 'KBX.DE', 'MATAS.CO', 'KGX.DE', 'ALLE.N', 'VNAn.DE', 'TRIG.L', 'GTT.PA', 'GWI.L', 'FDEV.L', 'INGA.AS', 'TIG.L', 'ROSNN.BX', 'FSFL.L', 'TRWL.MI', 'FOXT.L', 'THG.L', 'CRTO.OQ', 'CNH.N', 'NOHOP.HE', 'TKTT.PA', 'CMBN.S', 'RSER.L', 'CTT.LS', 'SYNTH.TE', 'IRIST.ST', 'TCSG.MM', 'EMANE.L', 'VIOH.AT', 'JUSTJ.L', 'BBOXT.L', 'PLAZb.ST', 'ALCRB.PA', 'BMKB.L', 'NBS.L', 'GLOB.N', 'FESTI.IC', 'ZALG.DE', 'FILA.MI', 'FGA.PA', '42TE.L', 'VALMT.HE', 'QURE.OQ', 'BIMOB.ST', 'NESF.L', 'PETSP.L', 'NIOX.L', 'BUFAB.ST', 'WLN.PA', 'MCLSM.L', 'AO.L', 'LRES.MC', 'ISS.CO', 'FGENF.L', 'EDRE.MC', 'SANION.ST', 'BOOH.L', 'DHG.I', 'GENG.L', 'LMN.S', 'CREI.L', 'VERK.HE', 'ANORA.HE', 'MTLS.OQ', 'IRES.I', 'NOMD.N', 'ANIM.MI', 'KAMBI.ST', 'HIAG.S', 'FBK.MI', 'CARDC.L', 'SHOE.L', 'ATTOF.PK', 'SAGA.L', 'OSBO.L', 'FLOL.L', 'STM1.DE', 'ESP.L', 'CLAVI.ST', 'ZAL.OL', 'ELIOR.PA', 'FDM.L', 'ENX.PA', 'NN.AS', 'OEC.N', 'ALRPD.PA', 'NOGN.L', 'BMEB.L', 'FACC.VI', 'ONTEX.BR', 'FAN.L', 'MRL.MC', 'WIZZ.L', 'BACTIb.ST', 'HELIO.ST', 'ARGX.BR', 'HANZA.ST', 'SCST.ST', 'COFA.PA', 'OTB.L', 'FEOIb.ST', 'AFMD.OQ', 'FCT.MI', 'TRMED.OL', 'SPI.L', 'LOG.MC', 'EPWN.L', 'GROWE.MI', 'TLGO.MC', 'EQLS.L', 'TNXT.MI', 'GALE.S', 'PRQR.OQ', 'OVS.MI', 'SCATC.OL', 'ELIS.PA', 'ABSO.ST', 'XXL.OL', 'RKET.H', 'RWAY.MI', 'ICOS.MI', 'INWI.ST', 'AKSOA.OL', 'APN.BN', 'PSH.AS', 'SCTS.L', 'GRANG.ST', 'ENTRA.OL', 'GAMA.L', 'TLGG.H', 'SPOLS.OL', 'MULTI.OL', 'BERNERb.ST', 'HSS.L', 'MAB1.L', 'FNMA.ST', 'FEVR.L', 'SBBb.ST', 'BAWG.VI', 'MULT.DE', 'LIFCOb.ST', 'THULE.ST', 'UBSG.S', 'TUNE.L', 'UPR.I', 'KOJAMO.HE', 'B2I.OL', 'BRAV.ST', 'ELTEL.ST', 'PCELL.ST', 'ASND.OQ', 'TRNT.L', 'POXEL.PA', 'TC1n.H', 'IDHC.L', 'ECEL.L', 'DUST.ST', 'DFSD.L', 'AENA.MC', 'NNIT.CO', 'INOV.L', 'ORGC.ST', 'DETEC.HE', 'TRC.L', 'LNSX.F', 'SDIPpref.ST', 'HOFI.ST', 'SEQI.L', 'AUTOA.L', 'ENENTO.HE', 'TROAX.ST', 'OSE.PA', 'EVOG.ST', 'INWT.MI', 'MERIS.ST', 'ADB.MI', 'K2Apref.ST', 'REITIR.IC', 'AGRP.L', 'IGT.N', 'ADAP.OQ', 'TOBII.ST', 'CLNX.MC', 'BCART.BR', 'MTRO.L', 'TGYM.MI', 'GTLY.L', 'SCIB.ST', 'G4M.L', 'APAX.L', 'EPR.OL', 'AT1.DE', 'WAFGn.DE', 'PIHLIS.HE', 'CRN.L', 'COOR.ST', 'INISSb.ST', 'HEIMpref.ST', 'ALIG.ST', 'GREG.MC', 'NILb.ST', 'TNOM.HE', 'AMPLI.PA', 'PANDXb.ST', 'FLOW.AS', 'LUVE.MI', 'ADJ.DE', 'KNOS.L', 'MONNA.MI', 'GYM.L', 'ENAV.MI', 'TFBANK.ST', 'NVCR.OQ', 'ORSO.MI', 'BGSGH.BB', '1COVG.DE', 'G24n.DE', 'BMED.MI', 'ORY.MC', 'SINCH.ST', 'KID.OL', 'CYB1.ST', 'HBH.DE', 'CCAG.DE', 'IBST.L', 'SRPG.PA', 'LIVN.OQ', 'HSW.L', 'PHMR.MC', 'STE.N', 'DOMETIC.ST', 'MINEST.ST', 'RGLR.L', 'ABNd.AS', 'ACADE.ST', 'ATTE.ST', 'SHOTE.ST', 'CONSTI.HE', 'MRON.L', 'CAMX.ST', 'ED4.DE', 'SFRG.ST', 'XIOR.BR', 'IMMNOV.ST', 'SPKSJF.CO', 'GROW.L', 'SFIN.MM', 'VICOR.ST', 'VOLO.ST', 'GSM.OQ', 'RACE.MI', 'STOGR.CO', 'GNRO.PA', 'SIFG.AS', 'XBRANE.ST', 'DNR.MI', 'MDM.PA', 'CATME.ST', 'NETCG.CO', 'CER.L', 'POLYG.ST', 'PDXI.ST', 'GARO.ST', 'MRLM.L', 'WJG.L', 'ALIFb.ST', 'HUMAN.ST', 'VACN.S', 'MONET.PR', 'IBTb.ST', 'SHED.L', 'FORT.L', 'WIHN.S', 'LEHTO.HE', 'DOMI.MC', 'TOKMAN.HE', 'TRMDa.CO', 'RESURS.ST', 'LIGHT.AS', 'QTCOM.HE', 'MIDWM.L', 'TMOT.L', 'KAMUX.HE', 'MOTR.L', 'NLBR.LJ', 'FFARM.AS', 'TALKP.ST', 'ICESEA.IC', 'BFIT.AS', 'BONAVb.ST', 'IREN.S', 'SECARE.ST', 'GOMX.ST', 'DHH.MI', 'BOWL.L', 'MAHAa.ST', 'ACIU.OQ', 'FRAN.L', 'TBCG.L', 'EMBRACb.ST', 'ADNT.N', 'TKWY.AS', 'UN0k.DE', 'ALCA.ST', 'VQTG.H', 'RDC.DE', 'FF.MI', 'CRSP.OQ', 'PLYA.OQ', 'CTEC.L', 'FLERIE.ST', 'BICO.ST', 'LUCEL.L', 'VANL.L', 'BBH.L', 'IG.MI', 'TRVG.OQ', 'ATORX.ST', 'ACARIX.ST', 'SEYE.ST', 'GEST.MC', 'DNP.WA', 'VARN.S', 'CASHP.MC', 'ROM.BX', 'AINO.ST', 'GPI.MI', 'AACM.ST', 'PCZ.DE', 'SFOR.L', 'WIIT.MI', 'FTI.N', 'BOCH.CY', 'TKOO.PA', 'RBWR.L', 'RFXR.L', 'DSKY.MM', 'AYV.PA', 'XPS.L', 'ONCO.ST', 'BFF.MI', 'AAGG.DE', 'HOME.MC', 'MIPS.ST', 'XFAB.PA', 'UNI.MC', 'BIOVICb.ST', 'LDG.L', 'IVACC.ST', 'ECNL.MI', 'GREENM.CO', 'XMR.ST', 'DHER.DE', 'INSTAL.ST', 'ALFAAL.L', 'BOOZT.ST', 'DIGI.BX', 'MTRS.ST', 'PWG.PA', 'MCOVb.ST', 'NITRO.ST', 'PAX.ST', 'ESSITYb.ST', 'IDIA.S', 'BONEX.ST', 'ALLIGOb.ST', 'ADMr.AT', 'SEDANA.ST', 'GRPG.I', 'OXEMA.ST', 'B4B.DE', 'SITT.MI', 'NEXSN.L', 'SUPR.L', 'PHNU.MI', 'NRPJ.J', 'ANG.L', 'JST.DE', 'LANDI.S', 'VNTRF.PK', 'KETL.L', 'VOLCARb.ST', 'WHRW.L', 'SPRSP.L', 'AEDAS.MC', 'NEXA.N', 'XSPRAY.ST', 'PIRC.MI', 'TTALO.HE', 'BALCO.ST', 'GLV.I', 'WSTEP.OL', 'TENEO.ST', 'NLFSK.CO', 'SMCP.PA', 'BIOAb.ST', 'CLIMEb.ST', 'CRAYN.OL', 'BFSA.DE', 'HFGG.DE', 'TIFS.L', 'LIN.OQ', 'ROSFG.BX', 'BAKK.L', 'EQUI.MI', 'TCM.CO', 'SBRE.L', 'GOFORE.HE', 'TENG.L', '247.ST', 'ARJOb.ST', 'MAGIN.ST', 'CAAP.N', 'ACCON.ST', 'EG7.ST', 'QLT.L', 'INS2.DE', 'SHLG.DE', 'ROWINE.BX', 'MVC.MC', 'DMPG.DE', 'NEOEN.PA', 'ENOG.L', 'IHPI.L', 'CRLI.MI', 'ELMRA.OL', 'ALFEN.AS', 'BSGR.AS', 'SPOT.N', 'BHGF.ST', 'CIBUS.ST', 'GREENL.ST', 'JTC.L', 'HARVIA.HE', 'DWSG.DE', 'KVIKA.IC', 'SENSI.S', 'ASWN.S', 'FNTL.L', 'ELK.OL', 'MEDA.S', 'USAB.L', 'KP2.L', 'SJJG.DE', 'SPEXN.S', 'NVT.N', 'SF3.DE', 'SOME.MI', 'NCAB.ST', 'OVZON.ST', 'ARION.IC', 'KNSA.OQ', 'ITECH.ST', 'ADYEN.AS', 'CGEO.L', 'EPIRa.ST', 'AL2SI.PA', 'BETCO.CO', 'EEZY.HE', 'KGHK.L', 'ANXA.L', 'AMGO.L', 'H24.H', 'KLIN.S', 'PENGb.ST', 'RAI.L', 'MIDSU.ST', 'GB.N', 'RAKE.ST', 'MIND.L', 'WOSG.L', 'ASAI.L', 'OPRA.OQ', 'DAVA.N', 'METAC.ST', 'WRKS.L', 'PDD.OQ', 'CWK.N', 'FTCHQ.PK', 'FCH.L', 'SSON.L', 'ESTC.N', 'GTX.OQ', 'AML.L', 'SIGNC.S', '8TRA.DE', 'WEW.DE', 'SHUR.BR', 'PVNA.MI', 'CLVT.N', 'AAF.L', 'TPXT.L', 'GHC.MI', 'AJBA.L', 'NYFO.ST', 'SEIT.L', 'LIMET.ST', 'QLINEA.ST', 'COME.MI', 'ILTY.MI', 'SRAIL.S', 'MNTN.L', 'VPLAYb.ST', 'MOVE.S', 'LGRS.L', 'JMIA.N', 'ALCC.S', 'KARNO.ST', 'ANV.MI', 'NEXII.MI', 'VRLA.PA', 'JOMA.ST', 'MBHB.BU', 'AMCR.N', 'AFPD.S', 'MNTC.ST', 'GFG.DE', 'TITC.BR', 'FCL.AX', 'BRCK.L', 'PRX.AS', 'EQTAB.ST', 'ADCT.N', 'TMV.DE', 'BNTX.OQ', 'SNL.MI', 'AKELd.ST', 'SATSS.OL', 'HTWS.L', 'SWON.S', 'NSKOG.OL', 'MNG.L', 'NWLF.MI', 'CNTGF.PK', 'PEBB.L', 'ITH.L', 'KFASTb.ST', 'ORIT.L', 'ADT1.L', 'DOXE.MI', 'MUSTI.HE', 'SPECI.L', 'ENPG.MM', 'UD.MI', 'NLSP.OQ', 'FRP.L', 'N91.L', 'EPROb.ST', 'GVS.MI', 'JDEP.AS', 'CZG.PR', 'VZUG.S', 'VNV.ST', 'GPGR.ST', 'AKBM.OL', 'CVAC.OQ', 'KTAG.DE', 'ACCA.OL', 'BEWI.OL', 'READ.ST', 'VIAOY.PK', 'HOMEH.L', 'ENR1n.DE', 'VAREV.L', 'HAGG.DE', 'WBGRb.ST', 'AMBP.N', 'GDEV.OQ', 'NPAPER.ST', 'PSFE.N', 'ALEP.WA', 'LINK.OL', 'HYPRO.OL', 'HUSCO.CO', 'BYIT.L', 'SAVE.ST', 'OZONDR.MM', 'SGHC.N', 'CADLR.OL', 'FG.ST', 'NWHD.ST', '80CX.L', 'THUNDR.ST', 'ROIV.OQ', 'NA9n.DE', 'FSGF.L', 'MOONM.L', 'INPST.AS', 'AG1G.DE', 'RNW.OQ', 'DOCS.L', 'AFRI.OQ', 'DTGGe.DE', 'TE.PA', 'ATG.L', 'VTWRn.H', 'CTPNV.AS', 'ROO.L', 'TRST.L', 'ONT.L', 'HEM.ST', 'KTEn.H', 'ALLFG.AS', 'PBEE.L', 'WIX.L', 'PPGN.S', 'TRIFOR.CO', 'MILDEF.ST', 'PCOP.WA', 'ISB.IC', 'ACAST.ST', 'YOUG.DE', 'ANE.MC', 'GAMB.OQ', 'BCG.L', 'LLYC.MC', 'WISEa.L', 'ROONE.BX', 'BPTB.L', 'DOLE.N', 'WPS.L', 'ANTIN.PA', 'AZE.BR', 'UMG.AS', 'IHS.N', 'OVH.PA', 'PHLL.L', 'STORb.ST', 'AUTO.OL', 'HEIT.L', 'PODP.L', 'PROC.L', 'ARIS.MI', 'LENT.MM', 'DYVOX.ST', 'ZGN.N', 'IVG.MI', 'CPH2.L', 'VAR.OL', 'AOJb.CO', 'HLUNa.CO', 'HLN.L', 'DSFIR.AS', 'NCH2.DE', 'DWL.L', 'ARM.OQ', 'ETS.VI', '1SXP.DE', 'BIRK.N', 'SDZ.S', 'SYENS.BR', 'SHA0.DE']
africanRICs = ['AFEJ.J', 'AELJ.J', 'AVIJ.J', 'BAWJ.J', 'GFIJ.J', 'HARJ.J', 'IMPJ.J', 'SUIJ.J', 'MURJ.J', 'NEDJ.J', 'PIKJ.J', 'SAPJ.J', 'SBKJ.J', 'TONJ.J', 'ATW.CS', 'DTCJ.J', 'CLSJ.J', 'RMHJ.J', 'CSBJ.J', 'KAPJ.J', 'BVTJ.J', 'GNDJ.J', 'GRTJ.J', 'HYPJ.J', 'HCIJ.J', 'HDCJ.J', 'MTAJ.J', 'MTMJ.J', 'FSRJ.J', 'ADRJ.J', 'OUL.CS', 'SOLJ.J', 'ABGJ.J', 'REB.CS', 'RCLJ.J', 'MRPJ.J', 'BATJ.J', 'SID.CS', 'DLTA.ZI', 'ETEL.CA', 'COMI.CA', 'PPCJ.J', 'ADIB.CA', 'EXPA.CA', 'FAIT.CA', 'WAA.CS', 'ORWE.CA', 'PHAR.CA', 'FBRJ.J', 'LCSW.CA', 'TREJ.J', 'TSGJ.J', 'RLOJ.J', 'AMSJ.J', 'SNTJ.J', 'OMNJ.J', 'SHPJ.J', 'TRUJ.J', 'EAST.CA', 'AEGJ.J', 'HDBK.CA', 'LSAK.OQ', 'ARIJ.J', 'ACLJ.J', 'APNJ.J', 'OCEJ.J', 'DRDJ.J', 'SPGJ.J', 'INLJ.J', 'IVTJ.J', 'NPHJ.J', 'OCTJ.J', 'WBOJ.J', 'MRFJ.J', 'ADHJ.J', 'LHM.CS', 'SAUD.CA', 'QNBE.CA', 'UBA.LG', 'MAB.CS', 'SMI.CS', 'HELI.CA', 'NPNJn.J', 'CLHJ.J', 'ATH.CS', 'BOA.CS', 'BCI.CS', 'CDM.CS', 'CIH.CS', 'BELJ.J', 'CSR.CS', 'LES.CS', 'CMA.CS', 'MTNJ.J', 'TFGJ.J', 'TBSJ.J', 'NPKJ.J', 'EKHO.CA', 'MNG.CS', 'SACJ.J', 'MCBG.MZ', 'MASR.CA', 'WHLJ.J', 'NTCJ.J', 'BRTJ.J', 'OMUJ.J', 'DSYJ.J', 'IOCJ.J', 'HRHO.CA', 'SLMJ.J', 'OCDI.CA', 'MOX.CS', 'GAZ.CS', 'NEJ.CS', 'ARLJ.J', 'ESRS.CA', 'SURJ.J', 'RDFJ.J', 'GTHE.CA', 'REMJ.J', 'UMR.CS', 'SBM.CS', 'EXXJ.J', 'CPIJ.J', 'SUDATEL.AD', 'RESJ.J', 'TKGJ.J', 'CMLJ.J', 'EMIJ.J', 'VKEJ.J', 'BCP.CS', 'DFCU.UG', 'LEWJ.J', 'SPPJ.J', 'ZENITHB.LG', 'IAM.CS', 'FIDELIT.LG', 'AMOC.CA', 'WEZJ.J', 'SWDY.CA', 'JSEJ.J', 'RIS.CS', 'ADH.CS', 'EQTY.NR', 'ETI.LG', 'KIOJ.J', 'ZEDJ.J', 'DRI.CS', 'FBR.CS', 'HPS.CS', 'PHDC.CA', 'SBU.UG', 'DWY.CS', 'INV.CS', 'RBXJ.J', 'GBCO.CA', 'HLMJ.J', 'SSKJ.J', 'SOT.CS', 'M2M.CS', 'ATL.CS', 'BLUJ.J', 'TMGH.CA', 'SNA.CS', 'SLF.CS', 'SCOM.NR', 'DHO.CS', 'CMT.CS', 'GPLJ.J', 'LBV.CS', 'AIPJ.J', 'VODJ.J', 'FFBJ.J', 'CCAP.CA', 'JUFO.CA', 'NAKL.TN', 'LHCJ.J', 'DANGCEM.LG', 'SAH.CS', 'OUTJ.J', 'BTNJ.J', 'REBJ.J', 'MPTJ.J', 'BMNB.L', 'SEPL.L', 'DOMT.CA', 'TCPJ.J', 'UMEME.UG', 'TQM.CS', 'ATTJ.J', 'TMA.CS', 'EFID.CA', 'EQUJ.J', 'EMFD.CA', 'MSA.CS', 'BINV.CA', 'DCPJ.J', 'AFM.CS', 'CLHO.CA', 'BIDJ.J', 'PPHJ.J', 'MCGJ.J', 'MTHJ.J', 'MUT.CS', 'SSUJ.J', 'FWRY.CA', 'RMDA.CA', 'SSWJ.J', 'TSETNQJ.J', 'ARD.CS', 'MCRO.CA', 'TALM.CA', 'TGAJ.J', 'GTCO.LG', 'TGC.CS', 'AKT.CS', 'CFG.CS']
oceanianRICs = ['RSG.AX', 'QBE.AX', '1MC.AX', 'PFI.NZ', 'ALK.AX', 'BOQ.AX', 'BXB.AX', 'BKW.AX', 'BHP.AX', 'ALD.AX', 'ALQ.AX', 'RIO.AX', 'SHV.AX', 'ERA.AX', 'NUF.AX', 'FAR.AX', 'FPH.NZ', 'GPT.AX', 'MCM.AX', 'AOV.AX', 'ORI.AX', 'LLC.AX', 'LYC.AX', 'MAH.AX', 'CVN.AX', 'SUN.AX', 'MXI.AX', 'MCP.AX', 'REH.AX', 'STO.AX', 'SOL.AX', 'SGP.AX', 'WES.AX', 'ILU.AX', 'JIN.AX', 'WDS.AX', 'CBA.AX', 'APE.AX', 'EQT.AX', 'GWA.AX', 'HVN.AX', '1208.HK', 'RRL.AX', 'MAQ.AX', 'ASB.AX', 'CQR.AX', 'ALL.AX', 'RKN.AX', 'SKC.NZ', 'LIC.AX', 'PMV.AX', 'A1N.AX', 'RIC.AX', 'TAM.AX', 'STU.NZ', 'APZ.AX', 'FWD.AX', 'PRN.AX', 'SPK.NZ', 'NOU.AX', 'CCV.AX', 'SGM.AX', 'KPG.NZ', 'JRV.AX', 'PRX.AX', 'BPT.AX', 'FLT.AX', 'PPT.AX', 'FBU.NZ', 'ELD.AX', 'SWM.AX', 'DYL.AX', 'RBD.NZ', 'TNE.AX', 'WBC.AX', 'CEN.NZ', 'AIR.NZ', 'FMG.AX', 'ARB.AX', 'ORG.AX', 'OPT.AX', 'CMW.AX', 'EBO.NZ', 'EWC.AX', 'HLG.NZ', 'DOW.AX', 'KCN.AX', 'MND.AX', 'CHI.NZ', 'PAC.AX', 'IMM.AX', 'SAN.NZ', 'SHL.AX', 'THL.NZ', 'VAU.AX', 'CCX.AX', 'GRR.AX', 'POT.NZ', 'SLX.AX', 'TRA.NZ', 'EVT.AX', 'HZN.AX', 'ANN.AX', 'NAB.AX', 'TCL.AX', 'SBM.AX', 'PPK.AX', 'MGX.AX', 'AVE.AX', 'CUV.AX', 'ANZ.AX', 'BEN.AX', 'IMD.AX', 'MQG.AX', 'APA.AX', 'FGX.AX', 'MFT.NZ', 'WOW.AX', 'BCB.AX', 'BUB.AX', 'IMU.AX', 'HGO.AX', 'A1M.AX', 'MTS.AX', 'IFM.AX', 'CWP.AX', 'IFT.NZ', 'CSL.AX', 'PDN.AX', 'QAN.AX', 'EMR.AX', 'CPU.AX', 'PME.AX', 'BNL.AX', 'COH.AX', 'TLS.AX', 'TAH.AX', 'IRI.AX', 'RYM.NZ', 'WHS.NZ', 'RHC.AX', 'ATH.AX', 'PCT.NZ', 'WEB.AX', 'BWP.AX', 'ASX.AX', 'DTL.AX', 'DUB.AX', 'PNV.AX', 'MAY.AX', 'GNC.AX', 'AMP.AX', 'AAC.AX', 'BRG.AX', 'FML.AX', 'HLS.AX', 'MGR.AX', 'IAG.AX', '5GN.AX', 'AIA.NZ', 'GMT.NZ', 'OCL.AX', 'WEC.AX', 'TWR.NZ', 'VHP.NZ', 'AJX.AX', 'CLE.AX', 'HSN.AX', 'REA.AX', 'SRV.AX', 'OBL.AX', 'CGF.AX', 'CCP.AX', 'EOS.AX', 'IRE.AX', 'SPL.AX', 'TRS.AX', 'LKE.AX', 'AGI.AX', 'IGO.AX', 'AGY.AX', 'RHK.AX', 'AEL.AX', 'CNI.AX', 'BSL.AX', 'SKL.NZ', 'DEG.AX', 'NMT.AX', 'PET.AX', 'EVN.AX', 'RNE.AX', 'SIG.AX', 'ABG.AX', 'WOR.AX', 'ARG.NZ', 'AEF.AX', 'PEB.NZ', 'BKY.AX', 'RMS.AX', 'CQE.AX', 'NZX.NZ', 'IPL.AX', 'NHC.AX', 'FRW.NZ', 'DNK.NH', 'JBH.AX', 'ARU.AX', 'SNT.AX', 'CVT.NZ', 'CDA.AX', 'IFL.AX', 'MVP.AX', 'NST.AX', 'X64.AX', 'SFR.AX', 'ANG.AX', 'SYA.AX', 'MMS.AX', 'ATM.NZ', 'NCK.AX', 'MFG.AX', 'KAR.AX', 'INA.AX', 'AX1.AX', 'SUL.AX', 'STX.AX', 'MLX.AX', 'PRU.AX', 'ACR.AX', 'DXS.AX', 'MSB.AX', 'SSM.AX', 'DNA.AX', 'NEU.AX', 'GMG.AX', 'SEK.AX', 'CWY.AX', 'SIV.AX', 'DMP.AX', 'VCT.NZ', 'SKT.NZ', 'TGP.AX', 'M2Z.L', 'MNS.AX', 'CTP.AX', 'BRK.AX', 'AUB.AX', 'PFT.AX', 'SXL.AX', 'EML.AX', 'RUL.AX', 'BGL.AX', 'GOR.AX', 'SOLG.L', 'CHN.AX', 'NGI.AX', 'PPS.AX', 'RFG.AX', 'ETM.AX', 'CHC.AX', 'EHL.AX', 'MIN.AX', 'AMA.AX', 'SVR.AX', 'AGL.AX', 'LOM.AX', 'BCI.AX', 'LTR.AX', 'HUM.AX', 'QUB.AX', 'DVP.AX', 'NAN.AX', 'AMI.AX', 'CIA.AX', 'PTM.AX', 'WHC.AX', 'GOZ.AX', 'BOE.AX', 'XRO.AX', 'PNI.AX', 'MYX.AX', 'HUB.AX', 'A4N.AX', 'SYR.AX', 'PLS.AX', 'GMD.AX', 'NWH.AX', 'LCL.AX', 'IPD.AX', 'SRL.AX', 'NHF.AX', 'INR.AX', 'KRM.AX', 'GEM.AX', 'DTZ.AX', 'HAS.AX', 'ACF.AX', 'CMM.AX', 'BRU.AX', 'LOT.AX', 'MYS.AX', 'CAR.AX', 'MYR.AX', 'MCE.AX', 'KMD.NZ', 'ZIP.AX', 'SMR.AX', 'SPG.NZ', 'ALX.AX', 'SGH.AX', 'TLG.AX', 'WAF.AX', 'TER.AX', 'AZJ.AX', 'SML.NZ', 'WBT.AX', 'PDI.AX', 'NEC.AX', 'CTD.AX', 'NXT.AX', 'MPL.AX', 'CXO.AX', 'RNU.AX', 'YAL.AX', 'DDR.AX', 'HGH.NZ', 'AIS.AX', 'BGA.AX', 'TWE.AX', 'SGR.AX', 'CKF.AX', 'BRN.AX', 'SUM.NZ', 'CNU.NZ', 'VCX.AX', 'MCY.NZ', 'FCG.NZ', 'OFX.AX', 'GNE.NZ', 'MEL.NZ', 'RGN.AX', 'SDF.AX', 'FSF.NZ', 'CIP.AX', 'SHJ.AX', 'SKO.NZ', 'ARF.AX', 'BRL.AX', 'KLS.AX', 'DXI.AX', 'GDI.AX', 'NSR.AX', 'HPI.AX', 'ORA.AX', 'IEL.AX', 'PGH.AX', 'SGF.AX', 'RFF.AX', 'BAP.AX', 'HLI.AX', 'SCL.NZ', 'MVF.AX', 'GTK.NZ', 'SIQ.AX', 'SCG.AX', '3PL.AX', 'ERD.NZ', 'MPG.NZ', 'VGL.NZ', 'WTC.AX', 'REG.AX', 'COF.AX', 'IPH.AX', 'OML.AX', 'LOV.AX', 'PAR.AX', 'APX.AX', 'FPR.AX', 'SLC.AX', 'S32.AX', 'AFG.AX', 'ADH.AX', 'IDX.AX', 'BBN.AX', 'TEAM.OQ', 'PWH.AX', 'NVX.AX', 'TPW.AX', 'WLD.AX', 'BET.AX', 'IGL.AX', 'MP1.AX', 'RWC.AX', 'ATG.AX', 'CLW.AX', 'IPL.NZ', 'NZM.NZ', 'MHJ.AX', 'KGN.AX', 'WPR.AX', 'MNW.NZ', 'ING.AX', 'BVS.AX', 'WGX.AX', 'MAF.AX', 'OCA.NZ', 'AD8.AX', 'JLG.AX', 'NWL.AX', 'TLX.AX', 'DHG.AX', 'WGN.AX', 'JMS.AX', 'NIC.AX', 'VUL.AX', 'CXL.AX', 'VEA.AX', 'CRN.AX', 'COL.AX', 'NPH.NZ', 'CEH.AX', 'MGH.AX', 'PBH.AX', 'HMC.AX', 'TYR.AX', 'MBLPC.AX', 'ASM.AX', 'TUA.AX', 'TPG.AX', 'NXL.AX', 'ABB.AX', 'DRR.AX', 'HDN.AX', 'DBI.AX', 'CTT.AX', 'ACL.AX', 'EDV.AX', '29M.AX', 'PXA.AX', 'MI6.AX', 'HCW.AX', 'VSL.AX', 'SDR.AX', 'JDO.AX', 'VNT.AX', 'LLL.AX', 'TLC.AX']
americanRICs = ['KE.OQ', 'PATAm.BA', 'COMEm.BA', 'INDUm.BA', 'SEMIm.BA', 'LEDEm.BA', 'HARGm.BA', 'ALUAm.BA', 'GAROm.BA', 'MOLIm.BA', 'FERRm.BA', 'BBARm.BA', 'CEPUm.BA', 'TRANm.BA', 'TECO2m.BA', 'YPFDm.BA', 'BPATm.BA', 'CELU5m.BA', 'CAPXm.BA', 'TXARm.BA', 'CARCm.BA', 'GBANm.BA', 'CRESm.BA', 'AGROm.BA', 'SAMIm.BA', 'BOLTm.BA', 'IEBm.BA', 'CADOm.BA', 'BHIPm.BA', 'BMAm.BA', 'GGALm.BA', 'CECO2m.BA', 'GCLAm.BA', 'MIRGm.BA', 'METRm.BA', 'EDNm.BA', 'DGCU2m.BA', 'MORIm.BA', 'OESTm.BA', 'MELI.OQ', 'TGSU2m.BA', 'LONGm.BA', 'PAMPm.BA', 'CGPA2m.BA', 'TGNO2m.BA', 'EYPT.OQ', 'OGC.TO', 'LRE.L', 'NAT.N', 'AGO.N', 'GPRK.N', 'GKP.L', 'SFL.N', 'ESGR.OQ', 'GOGL.OQ', 'AXS.N', 'BBDC4.SA', 'ALPA4.SA', 'LIGT3.SA', 'BRKM5.SA', 'ITUB4.SA', 'LEVE3.SA', 'CMIG4.SA', 'ELET6.SA', 'BRFS3.SA', 'VALE3.SA', 'PETR4.SA', 'UNIP6.SA', 'ODPV3.SA', 'GOAU4.SA', 'WEGE3.SA', 'TUPY3.SA', 'SLCE3.SA', 'ENGI4.SA', 'DXCO3.SA', 'MRFG3.SA', 'OIBR4.SA', 'POMO4.SA', 'GOLL4.SA', 'EMBR3.SA', 'LREN3.SA', 'RAPT4.SA', 'USIM5.SA', 'CPLE6.SA', 'BHIA3.SA', 'RSID3.SA', 'SBSP3.SA', 'BBAS3.SA', 'MYPK3.SA', 'PCAR3.SA', 'BRAP4.SA', 'ITSA4.SA', 'TEND3.SA', 'CPFE3.SA', 'YDUQ3.SA', 'IGTAY.PK', 'BEEF3.SA', 'VIVT3.SA', 'MRVE3.SA', 'NEOE3.SA', 'CYRE3.SA', 'EGIE3.SA', 'MDIA3.SA', 'KLBN4.SA', 'VBBR3.SA', 'CCRO3.SA', 'JHSF3.SA', 'COGN3.SA', 'RAIL3.SA', 'ISAE4.SA', 'BRSR6.SA', 'GUAR3.SA', 'RADL3.SA', 'UGPA3.SA', 'IGTI3.SA', 'GFSA3.SA', 'VLID3.SA', 'OCN.L', 'TOTS3.SA', 'SMTO3.SA', 'RENT3.SA', 'PSSA3.SA', 'STBP3.SA', 'SAPR4.SA', 'EVEN3.SA', 'GRND3.SA', 'PDGR3.SA', 'EZTC3.SA', 'ENEV3.SA', 'CSAN3.SA', 'DASA3.SA', 'JBSS3.SA', 'ABCB4.SA', 'AMER3.SA', 'EQTL3.SA', 'POSI3.SA', 'MULT3.SA', 'TAEE11.SA', 'CSMG3.SA', 'CGY.TO', 'SUZB3.SA', 'AAV.TO', 'CCJ.N', 'BOS.TO', 'ARE.TO', 'URE.TO', 'LNR.TO', 'CRR_u.TO', 'WM.TO', 'RCG.TO', 'EQB.TO', 'OPY.N', 'DCa.TO', 'BCE.TO', 'BNS.TO', 'CNR.TO', 'BMO.TO', 'CM.TO', 'CFP.TO', 'FTT.TO', 'CU.TO', 'GWO.TO', 'LB.TO', 'L.TO', 'CCLb.TO', 'ENB.TO', 'CTCa.TO', 'MX.TO', 'MRU.TO', 'WTE.TO', 'RUS.TO', 'ENGH.TO', 'FRU.TO', 'IIP_u.TO', 'SPB.TO', 'ACOx.TO', 'SAP.TO', 'BBDb.TO', 'HLF.TO', 'RETa.V', 'POW.TO', 'DIIb.TO', 'BLX.TO', 'TCLa.TO', 'MEQ.TO', 'AP_u.TO', 'ELVA.TO', 'VRN.TO', 'ATD.TO', 'XTC.TO', 'CGO.TO', 'STN.TO', 'BNE.TO', 'BDGI.TO', 'NOVT.OQ', 'CFX.TO', 'JAG.TO', 'NFI.TO', 'BIR.TO', 'BEI_u.TO', 'ATS.TO', 'BN.N', 'CJT.TO', 'ATCU.TO', 'EDR.TO', 'ESI.TO', 'MFI.TO', 'AGFb.TO', 'PAAS.TO', 'ELD.TO', 'FSZ.TO', 'IMO.TO', 'TRZ.TO', 'REI_u.TO', 'TRI.TO', 'MAL.TO', 'MTL.TO', 'NVA.TO', 'BTE.TO', 'SRU_u.TO', 'SLF.TO', 'ATRL.TO', 'TECK.N', 'WPK.TO', 'GIBa.TO', 'MRC.TO', 'EMA.TO', 'EMPa.TO', 'MG.TO', 'SU.TO', 'IFP.TO', 'CP.TO', 'TIH.TO', 'QBRb.TO', 'CAS.TO', 'LNF.TO', 'CWB.TO', 'UNC.TO', 'MRE.TO', 'WFG.TO', 'HR_u.TO', 'MRT_u.TO', 'BSX.TO', 'APT.A', 'TFII.TO', 'SEA.TO', 'HRX.TO', 'COLUF.PK', 'BB.TO', 'T.TO', 'CLS.TO', 'CCA.TO', 'D_u.TO', 'CSWa.TO', 'ELF.TO', 'FTS.TO', 'CSU.TO', 'DPM.TO', 'RSI.TO', 'KBL.TO', 'CIGI.TO', 'ASM.TO', 'LGFa.N', 'APTO.OQ', 'RBA.TO', 'GSY.TO', 'SSRM.OQ', 'SOY.TO', 'TKO.TO', 'TWC.TO', 'TRX.A', 'CJRb.TO', 'BIP.N', 'ET.TO', 'LIF.TO', 'ONEX.TO', 'PHX.TO', 'IGM.TO', 'FFH.TO', 'INE.TO', 'CUPu.TO', 'CVG.TO', 'MDI.TO', 'GCGa.TO', 'SIS.TO', 'MAG.TO', 'WJX.TO', 'RCH.TO', 'WEF.TO', 'LASa.TO', 'CMG.TO', 'CHR.TO', 'ACQ.TO', 'CG.TO', 'FC.TO', 'AG.TO', 'NG.A', 'PSI.TO', 'TCW.TO', 'FM.TO', 'LULU.OQ', 'WPRT.TO', 'BCM.V', 'NDM.TO', 'UUUU.A', 'MLKKF.PK', 'CAR_u.TO', 'III.TO', 'RPI_u.TO', 'BPF_u.TO', 'ALS.TO', 'LUG.TO', 'KEY.TO', 'MFC.TO', 'ELR.TO', 'GAU.TO', 'ADEN.TO', 'BRE.TO', 'DBM.TO', 'HWX.TO', 'FENC.OQ', '2099.HK', 'TD.TO', 'NA.TO', 'AX_u.TO', 'TRP.TO', 'WRN.TO', 'IFC.TO', 'WPM.TO', 'SRSCQ.PK', 'CSH_u.TO', 'WILD.TO', 'SGQ.V', 'FNV.TO', 'CFW.TO', 'BTO.TO', 'DML.TO', 'BDT.TO', 'SVM.TO', 'FVI.TO', 'AC.TO', 'HLF.N', 'CWCO.OQ', 'CAP.SN', 'MASISA.SN', 'CCU.SN', 'VAPORES.SN', 'ENTEL.SN', 'COPEC.SN', 'CMPC.SN', 'COLBUN.SN', 'ENELGXCH.SN', 'ENELAM.SN', 'ANTARCHILE.SN', 'PARAUCO.SN', 'SQMA.SN', 'CONCHATORO.SN', 'SALFACORP.SN', 'BESALCO.SN', 'SECURITY.SN', 'ECL.SN', 'ANDINAB.SN', 'QUINENCO.SN', 'FALABELLA.SN', 'AGUASA.SN', 'BSANTANDER.SN', 'LTM.SN', 'RIPLEY.SN', 'EMBONORB.SN', 'ITAUCL.SN', 'CALICHERAA.SN', 'GASCO.SN', 'BCI.SN', 'SK.SN', 'FORUS.SN', 'CENCOSUD.SN', 'IAM.SN', 'CHILE.SN', 'SONDA.SN', 'ABC.SN', 'OROBLANCO.SN', 'BBO.CN', 'PMG.CN', 'SIS.CN', 'GAA.CN', 'COFIDCC1.LM', 'NCH.CN', 'BGA.CN', 'BIC.CN', 'ARG.CN', 'IMI.CN', 'CEL.CN', 'CFV.CN', 'ISA.CN', 'NMPGY.PK', 'HKLD.SI', 'FLUT.N', 'MNOV.OQ', 'ICHB.MX', 'VITROA.MX', 'KUOB.MX', 'CMOCTEZ.MX', 'GCARSOA1.MX', 'POSADASA.MX', 'KOFUBL.MX', 'PINFRA.MX', 'BEVIDESB.MX', 'ALFAA.MX', 'PEOLES.MX', 'IDEALB1.MX', 'CULTIBAB.MX', 'GFINBURO.MX', 'SIMECB.MX', 'HERDEZ.MX', 'TLEVISACPO.MX', 'GNP.MX', 'CIEB.MX', 'ORBIA.MX', 'GMEXICOB.MX', 'GCC.MX', 'OMAB.MX', 'ASURB.MX', 'ACCELSAB.MX', 'AMXB.MX', 'GPROFUT.MX', 'AC.MX', 'LAMOSA.MX', 'KIMBERA.MX', 'LIVEPOLC1.MX', 'GAPB.MX', 'GIGANTE.MX', 'AZTECACPO.MX', 'ELEKTRA.MX', 'GPH1.MX', 'GRUMAB.MX', 'ALSEA.MX', 'GFNORTEO.MX', 'POCHTECB.MX', 'SORIANAB.MX', 'BAFARB.MX', 'FRAGUAB.MX', 'FINAMEXO.MX', 'VALUEGFO.MX', 'CHDRAUIB.MX', 'GFAMSAA.MX', 'MEDICAB.MX', 'COLLADO.MX', 'MEGACPO.MX', 'CYDSASAA.MX', 'HOMEX.MX', 'CABLECPO.MX', 'AUTLANB.MX', 'BIMBOA.MX', 'FEMSAUBD.MX', 'AXTELCPO.MX', 'GBMO.MX', 'WALMEX.MX', 'GLNG.OQ', 'BWO.OL', 'CREDITC1.LM', 'UNACEMC1.LM', 'LUSURC1.LM', 'INTERBC1.LM', 'VOLCABC1.LM', 'PLUZENC1.LM', 'ORYGENC1.LM', 'CASAGRC1.LM', 'CVERDEC1.LM', 'MOROCOC2.LM', 'ENGIEC1.LM', 'CORAREC1.LM', 'CPACASC1.LM', 'BBVAC1.LM', 'FERREYC1.LM', 'ATACOAC1.LM', 'BUENAVC1.LM', 'ALICORC1.LM', 'AUSTRAC1.LM', 'AENZAC1.LM', 'SIDERC1.LM', 'MINSURI1.LM', 'BROCALC1.LM', 'NEXAPEC1.LM', 'FLEX.OQ', 'HSX.L', 'SIG.N', 'GNK.N', 'SMBK.N', 'NXST.OQ', 'OTRK.OQ', 'CTRN.OQ', 'ENSG.OQ', 'DRH.N', 'ATLO.OQ', 'GPRE.OQ', 'FLR.N', 'LINC.OQ', 'CBRE.N', 'PNNT.N', 'PKBK.OQ', 'TZOO.OQ', 'HTH.N', 'AVNW.OQ', 'EVR.N', 'CUBE.N', 'RVP.A', 'BGFV.OQ', 'BATL.A', 'BCLI.OQ', 'SAFT.OQ', 'SNX.N', 'AHT.N', 'AIZ.N', 'AVXL.OQ', 'MASI.OQ', 'KOP.N', 'DPZ.OQ', 'ATRC.OQ', 'KW.N', 'NGS.N', 'DYAI.OQ', 'BFIN.OQ', 'MEIP.OQ', 'PIPR.N', 'CME.OQ', 'ABR.N', 'CQP.N', 'EPM.A', 'BTU.N', 'INFU.A', 'CE.N', 'CORT.OQ', 'PSIX.OQ', 'BXC.N', 'EXPE.OQ', 'MDGL.OQ', 'UBFO.OQ', 'AXON.OQ', 'FBIZ.OQ', 'MDLZ.OQ', 'KIRK.OQ', 'ALGT.OQ', 'CDXC.OQ', 'BLKB.OQ', 'FIS.N', 'POR.N', 'KRG.N', 'LKQ.OQ', 'GYRE.OQ', 'CYTK.OQ', 'NYMT.OQ', 'WMG.OQ', 'EHTH.OQ', 'VNDA.OQ', 'AMP.N', 'DLB.N', 'TXRH.OQ', 'NNI.N', 'WLK.N', 'ARAY.OQ', 'LDOS.N', 'HRI.N', 'WEX.N', 'MDXG.OQ', 'CMG.N', 'INVA.OQ', 'NCMI.OQ', 'EFSC.OQ', 'LEVI.N', 'TMUS.OQ', 'ALGN.OQ', 'FTK.N', 'HAFC.OQ', 'SNFCA.OQ', 'ABG.N', 'EXAS.OQ', 'GPN.N', 'OIS.N', 'QRTEA.OQ', 'MARK.PK', 'IPGP.OQ', 'ACTG.OQ', 'GLDD.OQ', 'DZSI.PK', 'BRKR.OQ', 'ILMN.OQ', 'ESPR.OQ', 'EQIX.OQ', 'MPX.N', 'ACR.N', 'WW.OQ', 'SOCGP.PK', 'TXMD.OQ', 'ALLY.N', 'SPTN.OQ', 'TLS.OQ', 'UELMO.PK', 'DLAPQ.PK', 'RCL.N', 'CRL.N', 'UNB.OQ', 'BHLB.N', 'INO.OQ', 'ATEC.OQ', 'DOX.OQ', 'CVGW.OQ', 'DHC.OQ', 'BLX.N', 'CNK.N', 'CDR_pb.N', 'SWBI.OQ', 'MFIN.OQ', 'EPD.N', 'TTMI.OQ', 'DRRX.OQ', 'AWH.OQ', 'ITGR.N', 'TPR.N', 'EVC.N', 'PFIS.OQ', 'OSUR.OQ', 'EA.OQ', 'PLPC.OQ', 'BFC.OQ', 'CWBC.OQ', 'PFG.OQ', 'CCRN.OQ', 'ELV.N', 'AAP.N', 'AYI.N', 'CNC.N', 'WNEB.OQ', 'NWBO.PK', 'PRU.N', 'SYNA.OQ', 'APLD.OQ', 'L.N', 'GME.N', 'DSHK.PK', 'JBLU.OQ', 'NPO.N', 'VRNT.OQ', 'DXCM.OQ', 'BYON.N', 'TBRG.OQ', 'WTBA.OQ', 'ANIP.OQ', 'NFLX.OQ', 'NFBK.OQ', 'AMTX.OQ', 'HBIO.OQ', 'BYRN.OQ', 'INFN.OQ', 'AQB.OQ', 'ICFI.OQ', 'CPRX.OQ', 'BKD.N', 'XPO.N', 'MMLP.OQ', 'CONNQ.PK', 'DBI.N', 'HROW.OQ', 'MFIC.OQ', 'CYRX.OQ', 'PRTS.OQ', 'FRST.OQ', 'WLDN.OQ', 'GSAT.A', 'HBI.N', 'ORA.N', 'ALT.OQ', 'MFA.N', 'MA.N', 'CROX.OQ', 'AMSF.OQ', 'III.OQ', 'FOLD.OQ', 'CLNE.OQ', 'RLGT.A', 'EXR.N', 'TCRT.OQ', 'BR.N', 'CVGI.OQ', 'PNFP.OQ', 'RBC.N', 'PDFS.OQ', 'STXS.A', 'NOG.N', 'PRIM.N', 'BX.N', 'DFS.N', 'CVCO.OQ', 'PODD.OQ', 'BANC.N', 'ALTO.OQ', 'ENS.N', 'RGP.OQ', 'EGIOQ.PK', 'MPWR.OQ', 'OFLX.OQ', 'PDM.N', 'BLDR.N', 'FORM.OQ', 'GENN.PK', 'GORO.A', 'ARLP.OQ', 'ATSG.OQ', 'OMCL.OQ', 'ZBH.N', 'B.N', 'X.N', 'COP.N', 'K.N', 'O.N', 'R.N', 'FL.N', 'F.N', 'BA.N', 'AGCO.N', 'MTRN.N', 'AP.N', 'BC.N', 'KO.N', 'CAR.OQ', 'AE.A', 'ED.N', 'DE.N', 'ATGE.N', 'CTA_pa.N', 'CL.N', 'KODK.N', 'DY.N', 'LCII.N', 'M.N', 'DG.N', 'GT.OQ', 'GE.N', 'FE.N', 'GD.N', 'FC.N', 'FR.N', 'NEU.N', 'HD.N', 'HE.N', 'HL.N', 'HP.N', 'IT.N', 'SR.N', 'EL.N', 'MU.OQ', 'IP.N', 'CW.N', 'KR.N', 'MLSS.A', 'NPKI.N', 'ES.N', 'PH.N', 'MO.N', 'NL.N', 'EXC.OQ', 'RL.N', 'NX.N', 'NC.N', 'SO.N', 'TG.N', 'PG.N', 'SF.N', 'AAL.OQ', 'WY.N', 'TR.N', 'AFL.N', 'PLD.N', 'ABT.N', 'ACU.A', 'AKR.N', 'AIR.N', 'NSP.N', 'AES.N', 'APD.N', 'ANF.N', 'ADC.N', 'AMD.OQ', 'AMG.N', 'THG.N', 'ALB.N', 'ALX.N', 'ALK.N', 'AMBC.N', 'ALL.N', 'AIN.N', 'AXP.N', 'ALG.N', 'AEE.N', 'AEP.OQ', 'AFG.N', 'ARE.N', 'AIG.N', 'IEP.OQ', 'AWR.N', 'AWK.N', 'HES.N', 'APH.N', 'ARL.N', 'CNS.N', 'AVD.N', 'COR.N', 'TFSL.OQ', 'AME.N', 'AMT.N', 'ADI.OQ', 'ACAD.OQ', 'ACM.N', 'DVAX.OQ', 'MNKD.OQ', 'ACIC.OQ', 'NLY.N', 'EIG.N', 'AIV.N', 'CF.N', 'ADM.N', 'ATR.N', 'EPAC.N', 'ARW.N', 'AVB.N', 'AIT.N', 'ATO.N', 'AZO.N', 'AVY.N', 'AZZ.N', 'BCPC.OQ', 'AVT.OQ', 'TFC.N', 'ATNI.OQ', 'ADP.OQ', 'BMI.N', 'BALL.N', 'BHB.A', 'BAX.N', 'BDX.N', 'BDC.N', 'BHE.N', 'BZH.N', 'BBY.N', 'BBDC.N', 'HRB.N', 'LCTX.A', 'BXP.N', 'BSX.N', 'SAM.N', 'BYD.N', 'BDN.N', 'BRC.N', 'BWA.N', 'BKE.N', 'EAT.N', 'BMY.N', 'CVS.N', 'CBT.N', 'CTS.N', 'CNA.N', 'CAC.OQ', 'CVBF.OQ', 'CMS.N', 'CDNS.OQ', 'CWT.N', 'CPB.OQ', 'SNDA.N', 'CAH.N', 'COF.N', 'CSL.N', 'CTRA.N', 'CSX.OQ', 'MODG.N', 'CRS.N', 'CSV.N', 'CVM.A', 'CAT.N', 'CPT.N', 'CCL.N', 'EXP.N', 'EXE.OQ', 'KMX.N', 'CHE.N', 'CHD.N', 'CPK.N', 'CHH.N', 'IHRT.OQ', 'CIA.N', 'CLX.N', 'GTLS.N', 'CVX.N', 'CMA.N', 'FIX.N', 'CBU.N', 'CLF.N', 'CMC.N', 'NNN.N', 'CAG.N', 'CRK.N', 'COO.OQ', 'CIX.A', 'CNO.N', 'CDP.N', 'GLW.N', 'CMT.A', 'CUZ.N', 'TAP.N', 'BIGGQ.PK', 'INGR.N', 'CCK.N', 'DAR.N', 'AORT.N', 'CFR.N', 'CMI.N', 'DAN.N', 'CULP.N', 'DHR.N', 'DRS.OQ', 'DRI.N', 'DAL.N', 'DTE.N', 'DHI.N', 'SITC.N', 'DDS.N', 'DCI.N', 'DOV.N', 'INVX.N', 'DLX.N', 'EGP.N', 'DCO.N', 'EMN.N', 'DUK.N', 'EML.OQ', 'EIX.N', 'ECL.N', 'EMR.N', 'EBF.N', 'ETR.N', 'EPR.N', 'ENZ.N', 'EFX.N', 'EQT.N', 'EQR.N', 'ESS.N', 'ETD.N', 'NEE.N', 'FMC.N', 'FCN.N', 'FICO.N', 'FDS.N', 'FMCC.PK', 'FNMA.PK', 'FSS.N', 'AGM.N', 'FHI.N', 'FCF.N', 'FBP.N', 'SSB.N', 'FHN.N', 'FLS.N', 'FLO.N', 'BEN.N', 'GATX.N', 'AJG.N', 'TGNA.N', 'GAP.N', 'GCO.N', 'GIS.N', 'GEL.N', 'AVNT.N', 'GPC.N', 'GTY.N', 'MAGN.N', 'GRC.N', 'GGG.N', 'GVA.N', 'GHM.N', 'GWW.N', 'GTN.N', 'GPI.N', 'GFF.N', 'EQC.N', 'HAE.N', 'HAL.N', 'HALL.PK', 'HOG.N', 'NVRI.N', 'HIG.N', 'HAS.OQ', 'EHC.N', 'HVT.N', 'HEI.N', 'HIW.N', 'HXL.N', 'HPQ.N', 'HSY.N', 'PK.N', 'HNI.N', 'HRL.N', 'HMN.N', 'SVC.OQ', 'HOV.N', 'HUM.N', 'DIN.N', 'ESINQ.PK', 'IDA.N', 'ITW.N', 'IEX.N', 'IMPM.PK', 'IIIN.N', 'WTI.N', 'ANGO.OQ', 'FDBC.OQ', 'UCTT.OQ', 'CUTR.OQ', 'IESC.OQ', 'CCRD.N', 'IBM.N', 'IFF.N', 'IPG.N', 'JBL.N', 'JNJ.N', 'KALU.OQ', 'KBH.N', 'KMT.N', 'KMB.N', 'KEY.N', 'KRC.N', 'KEX.N', 'KSS.N', 'LTC.N', 'TBI.N', 'LZB.N', 'LEA.N', 'LEG.N', 'LEE.OQ', 'JEF.N', 'LEN.N', 'LLY.N', 'LNN.N', 'LNC.N', 'BBWI.N', 'LPX.N', 'LOW.N', 'LAD.N', 'MTB.N', 'MBI.N', 'MTG.N', 'MGM.N', 'MHO.N', 'MSM.N', 'VRE.N', 'MAC.N', 'MTW.N', 'ELS.N', 'MAN.N', 'MKL.N', 'MCS.N', 'MMC.N', 'MAS.N', 'MLM.N', 'MAR.OQ', 'MTZ.N', 'MLP.N', 'MAT.OQ', 'MMS.N', 'MKC.N', 'SPGI.N', 'MCD.N', 'PRA.N', 'MCY.N', 'MTH.N', 'MAA.N', 'MTD.N', 'MPB.OQ', 'MLR.N', 'MTX.N', 'ALE.N', 'MHK.N', 'MS.N', 'MLI.N', 'MSI.N', 'MUR.N', 'MYE.N', 'VYX.N', 'NVR.N', 'ERNA.OQ', 'FIZZ.OQ', 'NBR.N', 'NFG.N', 'NHC.A', 'NOV.N', 'NHI.N', 'NPK.N', 'NYT.N', 'NJR.N', 'JWN.N', 'NEM.N', 'NSC.N', 'NKE.N', 'XEL.OQ', 'NVAX.OQ', 'NUE.N', 'NUS.N', 'OGE.N', 'IOSP.OQ', 'OXY.N', 'ONIT.N', 'OII.N', 'OLN.N', 'OMC.N', 'ODC.N', 'ORI.N', 'OHI.N', 'OLP.N', 'OKE.N', 'OFG.N', 'OMI.N', 'PAR.N', 'PCG.N', 'PNC.N', 'PRK.A', 'PPG.N', 'BOH.N', 'PKE.N', 'OC.N', 'PMIR.PK', 'OXM.N', 'RNST.N', 'PEP.OQ', 'TPC.N', 'PFE.N', 'WTRG.N', 'PVH.N', 'PNW.N', 'PPC.OQ', 'BCO.N', 'PBI.N', 'PAA.OQ', 'PCH.OQ', 'PGR.N', 'TXNM.N', 'PHM.N', 'PEG.N', 'KWR.N', 'PWR.N', 'DGX.N', 'RPM.N', 'RES.N', 'RRC.N', 'RYN.N', 'RWT.N', 'RRX.N', 'REG.OQ', 'RGA.N', 'RSG.N', 'RMD.N', 'REX.N', 'RHI.N', 'ROK.N', 'ROG.N', 'ROP.OQ', 'ROL.N', 'T.N', 'SLG.N', 'SYBT.OQ', 'SFES.PK', 'JOE.N', 'TRV.N', 'BFS.N', 'MRK.N', 'SLB.N', 'MATV.N', 'SMG.N', 'SEB.A', 'SSP.OQ', 'SEE.N', 'SRE.N', 'SCI.N', 'SHW.N', 'SSD.N', 'SPG.N', 'SKY.N', 'SNA.N', 'SON.N', 'SAH.N', 'LUV.N', 'SCCO.N', 'SMP.N', 'SXI.N', 'SWK.N', 'STT.N', 'SCL.N', 'SCS.N', 'STC.N', 'SRI.N', 'SYK.N', 'RGR.N', 'SUI.N', 'SGC.OQ', 'SNV.N', 'SYY.N', 'TJX.N', 'TISI.N', 'SKT.N', 'TRC.N', 'TFX.N', 'THC.N', 'TDS.N', 'TEX.N', 'TER.OQ', 'TTI.N', 'TXN.OQ', 'TXT.N', 'TMO.N', 'KAI.N', 'THO.N', 'TDW.N', 'TKR.N', 'TOL.N', 'TMP.A', 'GL.N', 'TTC.N', 'DVA.N', 'TCI.N', 'YUM.N', 'WEN.OQ', 'TRN.N', 'TUPBQ.PK', 'TGI.N', 'TWIN.OQ', 'EAF.N', 'UAL.OQ', 'TSN.N', 'LEU.A', 'UFI.N', 'UNF.N', 'UNP.N', 'UIS.N', 'UNTC.PK', 'UDR.N', 'PAG.N', 'UMH.N', 'URI.N', 'RTX.N', 'USM.N', 'UVV.N', 'UTL.N', 'UHT.N', 'SXT.N', 'VHI.N', 'VFC.N', 'VLO.N', 'UHS.N', 'VLY.OQ', 'MTN.N', 'PRSU.N', 'VTR.N', 'VSH.N', 'LMAT.OQ', 'RRGB.OQ', 'NLST.PK', 'GSIT.OQ', 'PSEC.OQ', 'VNO.N', 'VMC.N', 'WNC.N', 'WMT.N', 'GHC.N', 'WAT.N', 'ELME.N', 'WM.N', 'WSO.N', 'WTS.N', 'WMK.N', 'WDC.OQ', 'WHR.N', 'WMB.N', 'WSM.N', 'WEC.N', 'WGO.N', 'WOR.N', 'WWW.N', 'WKC.N', 'SPB.N', 'RNR.N', 'ABCB.N', 'AAON.OQ', 'SRPT.OQ', 'VCEL.OQ', 'ADBE.OQ', 'HRTX.OQ', 'DENN.OQ', 'AEHR.OQ', 'AEIS.OQ', 'ALCO.OQ', 'SWKS.OQ', 'AMZN.OQ', 'AMED.OQ', 'UHAL.N', 'AMSC.OQ', 'AMWD.OQ', 'AXTI.OQ', 'AMGN.OQ', 'AMKR.OQ', 'CFNB.PK', 'ANDE.OQ', 'ANSS.OQ', 'HCKT.OQ', 'APOG.OQ', 'AAPL.OQ', 'AMAT.OQ', 'ARCB.OQ', 'AROW.OQ', 'ASB.N', 'ASTE.OQ', 'ALOT.OQ', 'ATRO.OQ', 'VOXX.OQ', 'ADSK.OQ', 'BANF.OQ', 'BOKF.OQ', 'BBSI.OQ', 'BSET.OQ', 'WRB.N', 'NOTV.OQ', 'BCRX.OQ', 'APYX.OQ', 'BRID.OQ', 'BRKL.OQ', 'AZTA.OQ', 'BFb.N', 'CFFI.OQ', 'CDW.OQ', 'CHRW.OQ', 'CPF.N', 'CCNE.OQ', 'CSGS.OQ', 'CACI.N', 'CDZI.OQ', 'HLX.N', 'CALM.OQ', 'PRDO.OQ', 'CASY.OQ', 'CWST.OQ', 'CASS.OQ', 'CATY.OQ', 'DMKPQ.PK', 'CNOB.OQ', 'CENX.OQ', 'CBZ.N', 'CENT.OQ', 'CERS.OQ', 'CRAI.OQ', 'CAKE.OQ', 'PLCE.OQ', 'CHDN.OQ', 'CIEN.N', 'CINF.OQ', 'CRUS.OQ', 'CTAS.OQ', 'CHCO.OQ', 'CLAR.OQ', 'CLH.N', 'CGNX.OQ', 'CTSH.OQ', 'COHU.OQ', 'COKE.OQ', 'CBAN.N', 'COLB.OQ', 'CMCO.OQ', 'COLM.OQ', 'CBSH.OQ', 'CTBI.OQ', 'CMTL.OQ', 'CPSS.OQ', 'CPRT.OQ', 'CRVL.OQ', 'CVLG.N', 'CRIS.OQ', 'CACC.OQ', 'WOLF.N', 'DXPE.OQ', 'DAKT.OQ', 'ELA.A', 'DECK.N', 'DXLG.OQ', 'XRAY.OQ', 'DGII.OQ', 'DIOD.OQ', 'DLTR.OQ', 'BOOM.OQ', 'EXCE.PK', 'EBAY.OQ', 'EDMCQ.PK', 'EMKR.OQ', 'EME.N', 'CEVA.OQ', 'MODV.OQ', 'MSCI.N', 'KBR.N', 'ERIE.OQ', 'ESCA.OQ', 'EEFT.OQ', 'EXPD.N', 'EXPO.OQ', 'EZPW.OQ', 'FARM.OQ', 'FARO.OQ', 'FAST.OQ', 'VERU.OQ', 'FITB.OQ', 'PFC.OQ', 'FFBC.OQ', 'FFIN.OQ', 'THFF.OQ', 'FRME.OQ', 'CASH.OQ', 'FUNC.OQ', 'BANR.OQ', 'FI.N', 'FLXS.OQ', 'FFIC.OQ', 'FORR.OQ', 'FONR.OQ', 'FWRD.OQ', 'FOSL.OQ', 'FSTR.OQ', 'FELE.OQ', 'FUL.N', 'FULT.OQ', 'GIII.OQ', 'GNTX.OQ', 'GENC.A', 'GABC.OQ', 'GERN.OQ', 'ROCK.OQ', 'GILD.OQ', 'GBCI.N', 'GSBC.OQ', 'WINA.OQ', 'HNRG.OQ', 'HWC.OQ', 'GIFI.OQ', 'HLIT.OQ', 'ALNT.OQ', 'HWKN.OQ', 'HCSG.OQ', 'HTLD.OQ', 'HTBK.OQ', 'JKHY.OQ', 'HFWA.OQ', 'HIFS.OQ', 'HOLX.OQ', 'ICAD.OQ', 'HUBG.OQ', 'JBHT.OQ', 'HBAN.OQ', 'HURC.OQ', 'ICUI.OQ', 'BIIB.OQ', 'IDT.N', 'IDXX.OQ', 'INCY.OQ', 'IBCP.OQ', 'INDB.OQ', 'WT.N', 'TCX.OQ', 'NKTR.OQ', 'INOD.OQ', 'NSIT.OQ', 'IART.OQ', 'INTC.OQ', 'IBOC.OQ', 'IVAC.OQ', 'INTU.OQ', 'INTT.A', 'ITIC.OQ', 'IONS.OQ', 'ITRI.OQ', 'JJSF.OQ', 'JAKK.OQ', 'KLAC.OQ', 'KCLI.PK', 'KVHI.OQ', 'KOPN.OQ', 'LXU.N', 'LYTS.OQ', 'LKFN.OQ', 'LAKE.OQ', 'LRCX.OQ', 'LANC.OQ', 'LFCR.OQ', 'LARK.OQ', 'LSTR.OQ', 'LSCC.OQ', 'AXGN.OQ', 'DSGR.OQ', 'LCUT.OQ', 'PPBI.OQ', 'LGND.OQ', 'LECO.OQ', 'LFUS.OQ', 'OPCH.OQ', 'SHOO.OQ', 'MGEE.OQ', 'MANH.OQ', 'MRTN.OQ', 'MTRX.OQ', 'MATW.OQ', 'MGRC.OQ', 'MRCY.OQ', 'MLAB.OQ', 'MESA.OQ', 'MMSI.OQ', 'MSFT.OQ', 'MVIS.OQ', 'MCHP.OQ', 'MSTR.OQ', 'MIDD.OQ', 'MSEX.OQ', 'MLKN.OQ', 'MOD.N', 'MCRI.OQ', 'MNRO.OQ', 'MYGN.OQ', 'MOV.N', 'VXRT.OQ', 'NBTB.OQ', 'ONTO.N', 'HOPE.OQ', 'NSSC.OQ', 'NATH.OQ', 'NATR.OQ', 'FF.N', 'EXLS.OQ', 'PBH.N', 'MBCN.OQ', 'LUNA.PK', 'TRAK.N', 'WBD.OQ', 'FFNW.OQ', 'SMCI.OQ', 'NTGR.OQ', 'BBW.N', 'NEOG.OQ', 'NTAP.OQ', 'NBIX.OQ', 'NDSN.OQ', 'NTRS.OQ', 'NRIM.OQ', 'NWFL.OQ', 'NWPX.OQ', 'OSIS.OQ', 'OVBC.OQ', 'ODFL.OQ', 'ONB.OQ', 'OSBC.OQ', 'ZEUS.OQ', 'ASGN.N', 'ORCL.N', 'ORLY.OQ', 'OTTR.OQ', 'GTBP.OQ', 'CNXN.OQ', 'PCAR.OQ', 'CODA.OQ', 'PZZA.OQ', 'PTC.OQ', 'PKOH.OQ', 'PTEN.OQ', 'PAYX.OQ', 'PATK.OQ', 'PDCO.OQ', 'PEGA.OQ', 'PENN.OQ', 'PEBO.OQ', 'AGYS.OQ', 'PLAB.OQ', 'PLXS.OQ', 'BPOP.OQ', 'POWL.OQ', 'POWI.OQ', 'NVEC.OQ', 'TROW.OQ', 'PSMT.OQ', 'VATE.N', 'CLMB.OQ', 'PROV.OQ', 'QCOM.OQ', 'FLG.N', 'DORM.OQ', 'RMBS.OQ', 'RF.N', 'REGN.OQ', 'RGS.OQ', 'RGEN.OQ', 'FRBKQ.PK', 'REFR.OQ', 'RELL.OQ', 'RVSB.OQ', 'RCKY.OQ', 'ROST.OQ', 'RGLD.OQ', 'RUSHA.OQ', 'STBA.OQ', 'INVE.OQ', 'POOL.OQ', 'SEIC.OQ', 'SM.N', 'SASR.OQ', 'SANM.OQ', 'SCSC.OQ', 'HSIC.OQ', 'RDUS.OQ', 'SEAC.PK', 'SIGI.OQ', 'SMTC.OQ', 'SCVL.OQ', 'SLGN.N', 'SMSI.OQ', 'SKYW.OQ', 'SMBC.OQ', 'SBSI.N', 'SHYF.OQ', 'STAA.OQ', 'SBUX.OQ', 'STLD.OQ', 'STRS.OQ', 'STRA.OQ', 'STRT.OQ', 'HLIO.N', 'SRDX.OQ', 'GEN.OQ', 'ACNT.OQ', 'LAUR.OQ', 'SNPS.OQ', 'TECH.OQ', 'TTEC.OQ', 'TTEK.OQ', 'TNC.N', 'TSBK.OQ', 'TSCO.OQ', 'TRNS.OQ', 'ACIW.OQ', 'TCBK.OQ', 'TRST.OQ', 'TRMK.OQ', 'COHR.N', 'UFPT.OQ', 'UMBF.OQ', 'USNA.N', 'MTCH.OQ', 'USPH.N', 'ULBI.OQ', 'AUB.N', 'UBSI.OQ', 'USLM.OQ', 'UNFI.N', 'KMPR.N', 'UNTY.OQ', 'UEIC.OQ', 'UFPI.OQ', 'USAP.OQ', 'URBN.OQ', 'UTMD.OQ', 'VSEC.OQ', 'VMI.N', 'VALU.OQ', 'VECO.OQ', 'VRSN.OQ', 'VSAT.OQ', 'VRTX.OQ', 'VICR.OQ', 'WDFC.OQ', 'WSFS.OQ', 'MTEM.PK', 'GALT.OQ', 'PETS.OQ', 'LVS.N', 'SAIA.OQ', 'MCHX.OQ', 'WASH.OQ', 'WBS.N', 'WAFD.OQ', 'WERN.OQ', 'WSBC.OQ', 'WABC.OQ', 'WEYS.OQ', 'WLFC.OQ', 'WLY.N', 'WRLD.OQ', 'WTFC.OQ', 'WWD.OQ', 'YELLQ.PK', 'ZBRA.OQ', 'HELE.OQ', 'ARTNA.OQ', 'LGTY.OQ', 'BELFA.OQ', 'BRKa.N', 'BIO.N', 'STZ.N', 'CATO.N', 'CRDb.N', 'CMCSA.OQ', 'FCX.N', 'IMKTA.OQ', 'TILE.OQ', 'GEF.N', 'HUBB.N', 'KELYA.OQ', 'MERC.OQ', 'MEI.N', 'MOGa.N', 'SBCF.OQ', 'RBCAA.OQ', 'SENEA.OQ', 'AOS.N', 'SFNC.OQ', 'SJM.N', 'PARA.OQ', 'A.N', 'VLGEA.OQ', 'ACCO.N', 'CRI.N', 'GRBK.N', 'TIPT.OQ', 'ULH.OQ', 'BECN.OQ', 'KRO.N', 'DEI.N', 'DLR.N', 'CYH.N', 'CMS_pb.N', 'EBTC.OQ', 'FBMS.N', 'CODI.N', 'IRBT.OQ', 'FSLR.OQ', 'TPX.N', 'BWEN.OQ', 'GPK.N', 'HBIA.PK', 'MYRG.OQ', 'PPWLO.PK', 'ARCC.OQ', 'MORN.OQ', 'THS.N', 'EBS.N', 'SBH.N', 'MPW.N', 'MOH.N', 'SHO.N', 'GLP.N', 'TRS.OQ', 'AB.N', 'AN.N', 'DX.N', 'BXMT.N', 'GS.N', 'EW.N', 'NI.N', 'VZ.N', 'EG.N', 'VC.OQ', 'ABEO.OQ', 'ATI.N', 'LNT.OQ', 'AXL.N', 'AWI.N', 'AVA.N', 'CAL.N', 'BRO.N', 'LUMN.N', 'CIZN.PK', 'CNX.N', 'FNF.N', 'ROIC.OQ', 'ZNOG.PK', 'DVN.N', 'EVI.A', 'NRP.N', 'EOG.N', 'EPC.N', 'XOM.N', 'ESE.N', 'FDX.N', 'HST.OQ', 'IDN.OQ', 'JACK.OQ', 'JLL.N', 'SAFE.N', 'KFY.N', 'LII.N', 'MCK.N', 'MET.N', 'MHGU.PK', 'NRG.N', 'NWL.OQ', 'PPL.N', 'PKG.N', 'RVTY.N', 'PTN.A', 'QMCO.OQ', 'CXW.N', 'CNP.N', 'RDN.N', 'SKX.N', 'TGT.N', 'GIC.N', 'REPX.A', 'TDY.N', 'SGU.N', 'TREX.N', 'TYL.N', 'UNM.N', 'WST.N', 'UPS.N', 'WAB.N', 'WTM.N', 'FDP.N', 'SRCE.OQ', 'FLWS.OQ', 'DDD.N', 'EGHT.OQ', 'ACNB.OQ', 'AKAM.OQ', 'MDRX.PK', 'NEO.OQ', 'AEO.N', 'GNSS.OQ', 'THRM.OQ', 'ASYS.OQ', 'ACGL.OQ', 'AGEN.OQ', 'CLFD.OQ', 'EP.A', 'VIRX.OQ', 'AUBN.OQ', 'TGTX.OQ', 'DHIL.OQ', 'BSRR.OQ', 'BBGI.OQ', 'ERINQ.PK', 'BMRN.OQ', 'DCOM.OQ', 'CBRL.OQ', 'CNTY.OQ', 'CSBR.OQ', 'CHMG.OQ', 'LNG.N', 'BJRI.OQ', 'COFS.OQ', 'CZNC.OQ', 'CCB.OQ', 'CSGP.OQ', 'COST.OQ', 'CRMT.OQ', 'BLFS.OQ', 'WHG.N', 'PFBC.OQ', 'HOMB.N', 'AX.N', 'TIOG.PK', 'ZUMZ.OQ', 'TDG.N', 'KINS.OQ', 'PARR.N', 'DGICA.OQ', 'EGBN.OQ', 'EWBC.OQ', 'APPS.OQ', 'EGAN.OQ', 'PLUS.OQ', 'HTGC.N', 'EVBN.A', 'HWBK.OQ', 'EXTR.OQ', 'FFIV.OQ', 'FISI.OQ', 'FCAP.OQ', 'BUSE.OQ', 'FCCO.OQ', 'CIVB.OQ', 'FMBH.OQ', 'FNLC.OQ', 'FLIC.OQ', 'FRAF.OQ', 'FLL.OQ', 'FCEL.OQ', 'SFST.OQ', 'GCBC.OQ', 'GNTY.N', 'HAIN.OQ', 'HSTM.OQ', 'HTLF.OQ', 'MED.N', 'HSII.OQ', 'HBNC.OQ', 'IMMR.OQ', 'IPAR.OQ', 'AREN.A', 'ARWR.OQ', 'DKS.N', 'SNEX.OQ', 'FA.OQ', 'TBBK.OQ', 'HEES.OQ', 'CVI.N', 'AVAV.OQ', 'MVBF.OQ', 'ISRG.OQ', 'VIAV.OQ', 'JOUT.OQ', 'JNPR.N', 'SWKH.OQ', 'KFRC.N', 'GDEN.OQ', 'LXRX.OQ', 'LPSN.OQ', 'LWAY.OQ', 'SMTI.OQ', 'ECPG.OQ', 'G.N', 'MBWM.OQ', 'MITK.OQ', 'MKSI.OQ', 'NKSH.OQ', 'NTCT.OQ', 'SEELQ.PK', 'NNBR.OQ', 'STRL.OQ', 'NVDA.OQ', 'OCFC.OQ', 'OPOF.OQ', 'ORRF.OQ', 'PGC.OQ', 'PEBK.OQ', 'PWOD.OQ', 'PLUG.OQ', 'BKNG.OQ', 'RDNT.OQ', 'PNRG.OQ', 'PB.N', 'PCYO.OQ', 'AGX.N', 'QCRH.OQ', 'UONEK.OQ', 'UPBD.OQ', 'RICK.OQ', 'RGCO.OQ', 'UAA.N', 'WYNN.OQ', 'HUN.N', 'BCBP.OQ', 'PLBC.OQ', 'GOOD.OQ', 'ET.N', 'LQDT.OQ', 'FSP.A', 'SBFG.OQ', 'SALM.PK', 'SGMO.OQ', 'SNBR.OQ', 'ELOX.PK', 'SHBI.OQ', 'SHEN.OQ', 'SIEB.OQ', 'SIGA.OQ', 'SLP.OQ', 'SLAB.OQ', 'SBSAA.PK', 'RGTPQ.PK', 'TTWO.OQ', 'DLHC.OQ', 'CDMO.OQ', 'DWSN.OQ', 'TUEMQ.PK', 'OLED.OQ', 'UTHR.OQ', 'UVE.N', 'UVSP.OQ', 'CTLP.OQ', 'EGY.N', 'OSPN.OQ', 'SCWO.OQ', 'KTOS.OQ', 'YORW.OQ', 'WU.N', 'RDI.OQ', 'PHX.N', 'CSR.N', 'MKTX.OQ', 'KRNY.OQ', 'SCOR.OQ', 'PRO.N', 'CRM.N', 'TCBI.OQ', 'RAIL.OQ', 'HALO.OQ', 'TOWN.OQ', 'SVRA.OQ', 'ACLS.OQ', 'BGS.N', 'CECO.OQ', 'ENTG.OQ', 'EXEL.OQ', 'GAIA.OQ', 'INFIQ.PK', 'HOFT.OQ', 'INSM.OQ', 'LCNB.OQ', 'CVLT.OQ', 'SAVA.OQ', 'PXLW.OQ', 'RIGL.OQ', 'ASTH.OQ', 'ON.OQ', 'HURN.OQ', 'CMP.N', 'SNCR.OQ', 'CNVS.OQ', 'UTI.N', 'ALNY.OQ', 'PFS.N', 'SD.N', 'TNL.N', 'APEI.OQ', 'CXB.TO', 'SKE.TO', 'NB.OQ', 'MTY.TO', 'TXG.TO', 'UAVS.A', 'BURL.N', 'FTNT.OQ', 'GWRE.N', 'IRWD.OQ', 'FN.N', 'BCC.N', 'PLTR.OQ', 'OBT.OQ', 'CABO.N', 'PBPB.OQ', 'IMUX.OQ', 'SPSC.OQ', 'ZION.OQ', 'PANW.OQ', 'CALX.N', 'ABOS.OQ', 'PTCT.OQ', 'AAOI.OQ', 'GLYC.OQ', 'TLG.TO', 'EIT_u.TO', 'NUAG.TO', 'WMS.N', 'SLQT.N', 'RIMSEGC1.LM', 'TARA.OQ', 'ECO.TO', 'CL.CD', 'CLSK.OQ', 'GMED.N', 'XPEL.OQ', 'ACMR.OQ', 'RRBI.OQ', 'TRIP.OQ', 'USAC.N', 'CDNA.OQ', 'NDLS.OQ', 'XGN.OQ', 'FSBC.OQ', 'FMAO.OQ', 'MGLU3.SA', 'DLTH.OQ', 'LRMR.OQ', 'JBSS.OQ', 'CBOE.Z', 'BNGO.OQ', 'SITM.OQ', 'FSK.N', 'OB.OQ', 'ANG_pa.N', 'XXII.OQ', 'AAT.N', 'EFC.N', 'ALUP11.SA', 'WDAY.OQ', 'HUBS.N', 'NGNE.OQ', 'KLTR.OQ', 'VTGN.OQ', 'FATE.OQ', 'IRTC.OQ', 'RVNC.OQ', 'AMRC.N', 'PCOR.N', 'PHR.N', 'LXFR.N', 'GLO.TO', 'CPAY.N', 'EBMT.OQ', 'ELAN.N', 'H.N', 'MRBK.OQ', 'VERX.OQ', 'SGRCF.PK', 'CIC.CN', 'NEPTF.PK', 'CRFB3.SA', 'ONTF.N', 'TVE.TO', 'CREAL.MX', 'FNWB.OQ', 'CHRD.OQ', 'PML.V', 'CFG.N', 'EIF.TO', 'NTB.N', 'TVTX.OQ', 'CRDF.OQ', 'PAYS.OQ', 'AUSOm.BA', 'BRBS.A', 'SNDR.N', 'LAB.OQ', 'CBFV.OQ', 'OPRX.OQ', 'EGRX.PK', 'LWLG.OQ', 'OZK.OQ', 'EXSR.PK', 'ALRS.OQ', 'SSYS.OQ', 'LMNR.OQ', 'OVLY.OQ', 'PCG_pa.A', 'SRNE.PK', 'IDEXQ.PK', 'FGEN.OQ', 'TRUL.CD', 'LOMAm.BA', 'VOYA.N', 'CNXC.OQ', 'TNET.N', 'ACTINVRB.MX', 'CKPT.OQ', 'HON.OQ', 'RICHm.BA', 'TCMD.OQ', 'KXS.TO', 'CBAV3.SA', 'BBAJIOO.MX', 'CARS.N', 'PHLT.OQ', 'MGNX.OQ', 'QNST.OQ', 'SSNC.OQ', 'GEB.CN', 'GKOS.N', 'SICPQ.PK', 'INVCENC1.LM', 'AZZA3.SA', 'MEC.N', 'CAML3.SA', 'QUAD.N', 'HY.N', 'SAVEQ.PK', 'IRBR3.SA', 'UNIFINA.MX', 'RNG.N', 'VALOm.BA', 'LASR.OQ', 'ALPEKA.MX', 'AR.N', 'SONM.OQ', 'BSY.OQ', 'POMALCC1.LM', 'CZFS.OQ', 'NPCE.OQ', 'ENTA.OQ', 'FIVE.OQ', 'PI.OQ', 'MRNS.OQ', 'NIC.N', 'EQBK.N', 'STIM.OQ', 'RXST.OQ', 'QLYS.OQ', 'CMRX.OQ', 'LUC.TO', 'ETB.CN', 'PACB.OQ', 'DVI_p.CN', 'OOMA.N', 'PCB.OQ', 'SONO.OQ', 'MXCT.OQ', 'HQY.OQ', 'ASPN.N', 'EPAM.N', 'ARMK.N', 'CVRX.OQ', 'INGN.OQ', 'RCKT.OQ', 'CDXS.OQ', 'ECO.CN', 'MEG.TO', 'AMBA.OQ', 'MLNK.N', 'CLDX.OQ', 'OMER.OQ', 'SSTI.OQ', 'LZ.OQ', 'MXL.OQ', 'LNSR.OQ', 'FIVN.OQ', 'VESTA.MX', 'MTDR.N', 'DOCU.OQ', 'SLNO.OQ', 'REEMF.PK', 'INBK.OQ', 'CELH.OQ', 'CPA.N', 'NEMAKA.MX', 'BCOV.OQ', 'ATEX.OQ', 'PEN.N', 'CHTR.OQ', 'FN.TO', 'ATH.TO', 'IRMD.OQ', 'CARA.OQ', 'RBLX.N', 'MWA.N', 'NXDT.N', 'META.OQ', 'DRT.TO', 'CSNA3.SA', 'OCGN.OQ', 'KALV.OQ', 'VERO.OQ', 'RDFN.OQ', 'ALDX.OQ', 'SUPN.OQ', 'YELP.N', 'AMAL.OQ', 'FBIN.N', 'BTB_u.TO', 'AIRG.OQ', 'ALOS3.SA', 'HR.N', 'VRNS.OQ', 'FLRY3.SA', 'TLPH.OQ', 'ADMA.OQ', 'ANAB.OQ', 'ETSY.OQ', 'TDOC.N', 'CCS.N', 'TRUP.OQ', 'EVFM.PK', 'GOGO.OQ', 'GCDIm.BA', 'AOSL.OQ', 'HOUS.N', 'CCLD.OQ', 'SFM.OQ', 'CHGG.N', 'EQX.A', 'KANT.PK', 'LPLA.OQ', 'GDOT.N', 'BERY.N', 'BNL.N', 'ZUO.N', 'APLE.N', 'RPID.OQ', 'ECN.TO', 'PBYI.OQ', 'TPL.CN', 'BLMN.OQ', 'SSBK.OQ', 'PGEN.OQ', 'ACB.TO', 'WKHS.OQ', 'VCYT.OQ', 'SNDX.OQ', 'CNVI.V', 'GEVO.OQ', 'RVYL.OQ', 'SMU.SN', 'MTBLY.PK', 'STKS.OQ', 'IFS.N', 'FET.N', 'OCUL.OQ', 'LEGH.OQ', 'AMWL.N', 'UAN.N', 'SGMT.OQ', 'APPF.OQ', 'BE.N', 'IMVIF.PK', 'PCRX.OQ', 'LC.N', 'AOI.TO', 'RNTX.OQ', 'TRVN.PK', 'SDGR.OQ', 'AKBA.OQ', 'FBRX.OQ', 'ECOR3.SA', 'QTWO.N', 'SFBS.N', 'ROKU.OQ', 'LWSA3.SA', 'SII.TO', 'OLO.N', 'TNXP.OQ', 'ATOM.OQ', 'DIRR3.SA', 'HYPE3.SA', 'OBE.TO', 'PUBM.OQ', 'ACCD.OQ', 'CIU_pa.TO', 'AMPH.OQ', 'BAND.OQ', 'PMTS.OQ', 'TPIC.OQ', 'APPN.OQ', 'RILY.OQ', 'FRES.L', 'BOLSAA.MX', 'BVC.CN', 'GBVLAC1.LM', 'MTRm.BA', 'UEC.A', 'BLNK.OQ', 'TITN.OQ', 'KAR.N', 'MAIN.N', 'ALSN.N', 'FOR.N', 'CPS.N', 'SEM.N', 'ENOV.N', 'TPB.N', 'MCFT.OQ', 'LRN.N', 'KOD.OQ', 'WSR.N', 'TDC.N', 'DHT.N', 'IPI.N', 'KDP.OQ', 'VHC.N', 'SEAL_pb.N', 'GWRS.OQ', 'MRC.N', 'CTIOm.BA', 'LGO.TO', 'AGNC.OQ', 'OESX.OQ', 'ORN.N', 'PM.N', 'V.N', 'SATS.OQ', 'HI.N', 'OCSL.OQ', 'CIM.N', 'ATHXQ.PK', 'MG.N', 'ARDX.OQ', 'AYA.TO', 'LABB.MX', 'MYFW.OQ', 'TNDM.OQ', 'AGIO.OQ', 'BDSX.OQ', 'LAAC.N', 'ZWS.N', 'IVZ.N', 'VRSK.OQ', 'JBTM.N', 'MHLD.OQ', 'REI.A', 'ERII.OQ', 'OPK.OQ', 'HCI.N', 'AILLP.PK', 'HBCP.OQ', 'IBIO.A', 'TREE.OQ', 'ZYXI.OQ', 'LOPE.OQ', 'PRLB.N', 'CYB.TO', 'FURY.TO', 'NVRO.N', 'ENPH.OQ', 'TRUE.OQ', 'MRSN.OQ', 'B3SA3.SA', 'WK.N', 'CFB.OQ', 'FSFG.OQ', 'MHH.A', 'CDLX.OQ', 'IVR.N', 'LMND.N', 'BEEM.OQ', 'MMATQ.PK', 'TWLO.N', 'ESNT.N', 'QUAL3.SA', 'CLW.N', 'PMET.TO', 'RNAC.OQ', 'ATXS.OQ', 'ARR.N', 'USFD.N', 'RECV3.SA', 'ALEATIC.MX', 'OPI.OQ', 'ASAI3.SA', 'OBK.N', 'EFX.TO', 'ORGS.PK', 'LE.OQ', 'HMST.OQ', 'KMI.N', 'VASCONI.MX', 'SPFI.OQ', 'MGNI.OQ', 'CPX.TO', 'INSP.N', 'BASE.OQ', 'TNTRQ.PK', 'BOKU.L', 'ENJOY.SN', 'BFAM.N', 'ADUS.OQ', 'GM.N', 'PBH.TO', 'LUN.TO', 'BK.N', 'CCBG.OQ', 'PRAA.OQ', 'VRTS.N', 'EVRI.N', 'BFH.N', 'GGBR4.SA', 'SPH.N', 'ABM.N', 'DHX.N', 'SPOK.OQ', 'SPR.N', 'JPM.N', 'MUX.N', 'CBL.N', 'RAIZ4.SA', 'MIC_pa.TO', 'BAP.N', 'AROC.N', 'SCHL.OQ', 'BACKUAC1.LM', 'SBNY.PK', 'EQH.N', 'ESSA.OQ', 'MPAA.OQ', 'RRTS.PK', 'UCB.N', 'BWFG.OQ', 'FMNB.OQ', 'SCOTIAC1.LM', 'TBNK.OQ', 'STWD.N', 'BPAC11.SA', 'GLRE.OQ', 'CCB.CN', 'VTNRQ.PK', 'UGI.N', 'PMT.N', 'CLBK.OQ', 'CEMEXCPO.MX', 'HDSN.OQ', 'FGBI.OQ', 'MOFG.OQ', 'CTC.SN', 'JRVR.OQ', 'SANB3.SA', 'HBT.OQ', 'IBKR.OQ', 'CNGO.PK', 'GTII.CD', 'DMRC.OQ', 'MD.N', 'TWO.N', 'AMN.N', 'COTY.N', 'BWB.OQ', 'INVEXA.MX', 'LMT.N', 'BPAN4.SA', 'MMM.N', 'NBN.OQ', 'ASUR.OQ', 'FBNC.OQ', 'NWC.TO', 'LXP.N', 'AMC.N', 'GFMULTIO.MX', 'ARI.N', 'VUZI.OQ', 'CGX.TO', 'LHX.N', 'MRVL.OQ', 'VTDRF.PK', 'SXC.N', 'FOXF.OQ', 'STEL.N', 'INSW.N', 'TIMS3.SA', 'BLK.N', 'MOS.N', 'DOW.N', 'CMIN3.SA', 'ARES.N', 'SIBN.OQ', 'LOB.N', 'CIX.TO', 'ATLC.OQ', 'CFLT.OQ', 'ALRM.OQ', 'NOW.N', 'ELMD.A', 'BWXT.N', 'IRT.N', 'IRDM.OQ', 'JAVER.MX', 'YEXT.N', 'GNLX.OQ', 'BOX.N', 'CSTL.OQ', 'AUPH.OQ', 'LYRA.OQ', 'RUN.OQ', 'XNCR.OQ', 'IMNM.OQ', 'OCX.OQ', 'GNRC.N', 'NWBI.OQ', 'PDSB.OQ', 'BGSF.N', 'CLDT.N', 'AQN.TO', 'PSTG.N', 'NBHC.N', 'BMRC.OQ', 'EB.N', 'PRI.N', 'MCB.N', 'TRNO.N', 'GBDC.OQ', 'NET.N', 'LIF.OQ', 'ASAN.N', 'CRMD.OQ', 'GGD.TO', 'PEB.N', 'XYZ.N', 'ABEV3.SA', 'CLPT.OQ', 'NUTX.OQ', 'IRSAm.BA', 'KALA.OQ', 'ADN.TO', 'BDI.TO', 'SG.N', 'AI.N', 'CVCB3.SA', 'EXFY.OQ', 'PECO.OQ', 'ADPT.OQ', 'GIL.TO', 'WEED.TO', 'RYI.N', 'LYV.N', 'PLOW.N', 'CADE.N', 'COCO.OQ', 'VNRX.A', 'SIA.TO', 'NWH_u.TO', 'FRBA.OQ', 'CARM.OQ', 'HPP.N', 'OM.OQ', 'FUBO.N', 'BRBI11.SA', 'FAF.N', 'TDUP.OQ', 'AMPE.PK', 'AIP.OQ', 'SUPVm.BA', 'GFS.OQ', 'ATOS.OQ', 'THR.N', 'VBTX.OQ', 'EU.V', 'VPG.N', 'GRPN.OQ', 'PII.N', 'APLS.OQ', 'AL.N', 'TTOO.OQ', 'PD.TO', 'NVO.TO', 'BAH.N', 'SYBX.OQ', 'CALA.PK', 'VTOL.N', 'PNCa.TO', 'OKTA.OQ', 'KNX.N', 'UI.N', 'CPNG.N', 'STAG.N', 'PRIO3.SA', 'NCLH.N', 'BJ.N', 'IGMS.OQ', 'RBB.OQ', 'FHB.OQ', 'CURY3.SA', 'LOGG3.SA', 'ILC.SN', 'ROAD.OQ', 'QTRX.OQ', 'VBNK.TO', 'FCNCA.OQ', 'TSQ.N', 'CFRXQ.PK', 'FBK.N', 'TRGP.N', 'ASMB.OQ', 'WD.N', 'INN.N', 'AURA.OQ', 'SBRA.OQ', 'OFED.PK', 'VRA.OQ', 'TBCH.OQ', 'SPORTS.MX', 'CTGO.A', 'MALLPLAZA.SN', 'MNMD.OQ', '3665.TW', 'KNT.TO', 'BKU.N', 'WRBY.N', 'HCA.N', 'HII.N', 'GENTERA.MX', 'REVG.N', 'DOMO.OQ', 'CTXR.OQ', 'FRCB.PK', 'INVJm.BA', 'CFFN.OQ', 'DJCO.OQ', 'RDOR3.SA', 'DOO.TO', 'FRPT.OQ', 'MFRISCOA1.MX', 'BOC.N', 'SIMH3.SA', 'REAL.TO', 'EXPI.OQ', 'IAC.OQ', 'XFOR.OQ', 'FLYW.OQ', 'UPLD.OQ', 'FND.N', 'SHOP.N', 'MARA.OQ', 'SERA.OQ', 'BKH.N', 'D.N', 'ABNB.OQ', 'KPTI.OQ', 'VSTM.OQ', 'MPC.N', 'RLJ.N', 'GOLD.TO', 'NGL.N', 'RCIb.TO', 'HWM.N', 'HLLY.N', 'TKNO.OQ', 'SST.N', 'EWCZ.OQ', 'ENFN.N', 'COMM.OQ', 'MCW.OQ', 'JELD.N', 'STRO.OQ', 'TRML.OQ', 'SKYX.OQ', 'KIDS.OQ', 'ECOR.OQ', 'UDMY.OQ', 'BRCC.N', 'PET.TO', 'MITT.N', 'TCPC.OQ', 'PBFS.OQ', 'GOOGL.OQ', 'CLMT.OQ', 'RVLV.N', 'SBGI.OQ', 'RPRX.OQ', 'AQST.OQ', 'GPRO.OQ', 'NTRA.OQ', 'RMR.OQ', 'FUNO11.MX', 'LIND.OQ', 'GEOS.OQ', 'PAHC.OQ', 'MBIN.OQ', 'ARCO.N', 'SPLP.N', 'DOCS.N', 'TSLX.N', 'BRX.N', 'SPT.OQ', 'LGD.TO', 'ORC.N', 'AIHC1.LM', 'CHEF.OQ', 'ARQ.OQ', 'VRDN.OQ', 'OTLK.OQ', 'ADVM.OQ', 'BPMC.OQ', 'NTNX.OQ', 'HBSA3.SA', 'CTE.CN', 'PENG.OQ', 'CRBG.N', 'GEI.TO', 'X.TO', 'BALY.N', 'PPTA.TO', 'ATKR.N', 'AMCX.OQ', 'CIVI.N', 'VRAYQ.PK', 'RARE.OQ', 'TLYS.N', 'NOC.N', 'ACHC.OQ', 'FBIO.OQ', 'PLAY.OQ', 'MTSI.OQ', 'PLYM.N', 'AFIB.PK', 'CON.CN', 'MGPI.OQ', 'VTRS.OQ', 'BLUE.OQ', 'NERV.OQ', 'RGNX.OQ', 'DTIL.OQ', 'IMRX.OQ', 'SMLR.OQ', 'VTLE.N', 'CG.OQ', '4966.TWO', 'RH.N', 'XYL.N', 'PLNT.N', 'ACRE.N', 'GNE.N', 'ENVA.N', 'CBNK.OQ', 'ALKT.OQ', 'VAC.N', 'FSBW.OQ', 'POST.N', 'PINS.N', 'GNPX.OQ', 'AVTX.OQ', 'CHE_u.TO', 'VITL.OQ', 'ILLM.TO', 'SGHT.OQ', 'DIBS.OQ', 'NARI.OQ', 'BSGM.OQ', 'SND.OQ', 'RENB.OQ', 'SAGE.OQ', 'PBF.N', 'LYFT.OQ', 'ZS.OQ', 'AMPY.N', 'OGI.TO', 'TGLS.N', 'TOON.A', 'BEP_u.TO', 'CUBI.N', 'NEO.TO', 'GNL.N', 'DRIO.OQ', 'NVEE.OQ', 'JYNT.OQ', '4958.TW', 'EFN.TO', 'GLSI.OQ', 'PSX.N', 'HTBI.OQ', 'XOMA.OQ', 'CRWD.OQ', 'CLSD.OQ', 'LOCO.OQ', 'SPNT.N', 'PRGS.OQ', 'UFCS.OQ', 'MSBI.OQ', 'ESRT.N', 'SWIM.OQ', 'FANG.OQ', 'SMSAAM.SN', 'RIVN.OQ', 'TRU.N', 'ASPU.PK', '0US7.L', 'MRG_u.TO', 'PV.MX', 'MATX.N', 'HNST.OQ', 'RM.N', 'MASS.OQ', 'CWBR.PK', 'COUR.N', 'CHRS.OQ', 'OPRT.OQ', 'WELL.TO', 'UBER.N', 'PPL.TO', 'SSTK.N', 'CRNX.OQ', 'LOGC.OQ', 'ABBV.N', 'Q.MX', 'LSPD.TO', 'BRZE.OQ', 'WPC.N', 'NGVC.N', 'NMIH.OQ', 'NMTRQ.PK', 'DDOG.OQ', 'SUN.N', 'TFIN.OQ', 'MPLX.N', 'TTSH.OQ', 'BFST.OQ', 'EXE.TO', 'SBT.OQ', 'ESBA.P', 'NRIX.OQ', 'DKL.N', 'MCRB.OQ', 'AIF.TO', 'ESQ.OQ', 'WCC.N', 'PSNL.OQ', 'ZTS.N', 'INRETC1.LM', 'DIR_u.TO', 'ORMP.OQ', 'PD.N', 'SFIX.OQ', 'SYRS.OQ', 'AI.TO', 'ACRS.OQ', 'IVN.TO', 'FROG.OQ', 'TPHS.PK', 'SAMG.OQ', 'DOL.TO', 'RHP.N', 'MULN.OQ', 'NHIQ.PK', 'BYND.OQ', 'FWONA.OQ', 'TNON.OQ', 'PTGX.OQ', 'AMPL.OQ', 'TXG.OQ', 'RPD.OQ', 'KPT.TO', 'EARN.N', 'ATRA.OQ', 'LAND.OQ', 'GBX.N', 'APAM.N', 'WES.N', 'RPR_u.TO', 'RLMD.OQ', 'AMBP3.SA', 'FINN13.MX', 'GRTX.PK', 'AMH.N', 'FIBRAMQ12.MX', 'ALLK.OQ', 'COMP.N', 'TELA.OQ', 'TNT_u.TO', 'TRVI.OQ', 'PRKS.N', 'AXDX.OQ', 'PZA.TO', 'THRY.OQ', 'GRT_u.TO', 'RC.N', 'HOT_u.TO', 'VYNE.OQ', 'RITM.N', 'FBRT.N', 'ZM.OQ', 'BBSE3.SA', 'SNAP.N', 'WH.N', 'KEL.TO', 'INO_u.TO', 'KOS.N', 'ACT.OQ', 'GNW.N', 'VEEV.N', 'ANNX.OQ', 'EOLS.OQ', 'NINE.N', 'PRV_u.TO', 'KHC.OQ', 'BPYPP.OQ', 'ICE.N', 'SYRE.OQ', 'ITCI.OQ', 'AHH.N', 'TXP.TO', 'EVTC.N', 'ALGM.OQ', 'PEY.TO', 'INTA.OQ', 'GSBD.N', 'VCTR.OQ', 'VABK.OQ', 'SAIC.OQ', 'ATNM.A', 'ZAMP3.SA', 'SFTC.TO', 'WDO.TO', 'CHMI.N', 'MUSA.N', 'XPRO.N', 'AXSM.OQ', 'CDE.N', 'ESI.N', 'REXR.N', 'DRM.TO', 'CHP_u.TO', 'NWSA.OQ', 'FRSH.OQ', 'CWENa.N', 'WATT.OQ', 'CART.OQ', '1316.HK', 'VOLARA.MX', 'GLPI.OQ', 'OUT.N', 'ASC.N', 'BHR.N', 'PPX.V', 'FIHO12.MX', 'LGIH.OQ', 'BIVI.OQ', 'IR.N', 'VNCE.N', 'TWST.OQ', 'DELL.N', 'EVH.N', 'FUVV.PK', 'CODX.OQ', 'PAGP.OQ', 'CEAB3.SA', 'LPG.N', 'MTNB.A', 'TBPH.OQ', 'S.N', 'OMF.N', 'RMAX.N', 'PVLA.OQ', 'AZUL.N', 'SEER3.SA', 'PINC.OQ', 'CRT_u.TO', 'OSCR.N', 'HLT.N', 'KN.N', 'XMTR.OQ', 'MMI.N', 'TCSGQ.PK', 'NEWT.OQ', 'DANHOS13.MX', 'OGS.N', 'CTRE.N', 'GEO.N', 'ENLC.N', 'EDIT.OQ', 'GCI.N', 'NAVI.OQ', 'AVIR.OQ', 'MBUU.OQ', 'ULCC.OQ', 'BLBD.OQ', 'LADR.N', 'FCUV.OQ', 'WSP.TO', 'SLDB.OQ', 'CIO.N', 'PCVX.OQ', 'IBP.N', 'CRBP.OQ', 'FPI.N', 'CTMX.OQ', 'GMRE.N', 'PAYC.N', 'SABR.OQ', 'RMBL.OQ', 'DBX.OQ', 'PCTY.OQ', 'RYAM.N', 'RNA.OQ', 'VYGR.OQ', 'MTUS.N', 'ABSI.OQ', 'WSBF.OQ', 'GH.OQ', 'DNOW.N', 'PMVP.OQ', 'ELF.N', 'GMS.N', 'FIGS.N', 'ZI.OQ', 'MC.N', 'KEYS.N', 'BHC.TO', 'PKST.N', 'VVX.N', 'GAMI.PK', 'MSA.N', 'VIRT.OQ', 'ATEN.N', 'SYF.N', 'SPWH.OQ', 'MEDP.OQ', 'MDB.OQ', 'LTRPA.PK', 'AMRK.OQ', 'SCPH.OQ', 'NMRD.PK', 'ANET.N', 'RXRX.OQ', 'UBX.OQ', 'SPRO.OQ', 'GRWG.OQ', 'SQZB.PK', 'QRVO.OQ', 'PSK.TO', 'CDTX.OQ', 'SGR_u.TO', 'HRTG.N', 'CMCT.OQ', 'LMB.OQ', 'PANL.OQ', 'AVNS.N', 'VNOM.OQ', 'ISTR.OQ', 'SLM.OQ', 'NEP.N', 'VKTX.OQ', 'CCOI.OQ', 'MPCT_u.TO', 'OR.TO', 'MEG.N', 'FIBRAPL14.MX', 'GDDY.N', 'TLN.OQ', 'TOY.TO', 'QNCX.OQ', 'ZD.OQ', 'WFRD.OQ', 'WEAV.N', 'CRC.N', 'LNTH.OQ', 'UE.N', 'NEXT.OQ', 'COGT.OQ', 'FSLY.N', 'LBRDA.OQ', 'RPT.N', 'PRN.TO', 'VERI.OQ', 'OPTN.OQ', 'CCI.N', 'W.N', 'INSE.OQ', 'AXTA.N', 'BOOT.N', 'RYTM.OQ', 'LUNG.OQ', 'PFGC.N', 'WBA.OQ', 'QSR.N', 'LAMR.OQ', 'XHR.N', 'CHWY.N', 'CRBU.OQ', 'XELA.PK', 'NXRT.N', 'AMLX.OQ', 'UPWK.OQ', 'AAMI.N', 'Z.OQ', 'SNOW.N', 'AQMS.OQ', 'BV.N', 'INDV.L', 'NTLA.OQ', 'PAVM.OQ', 'STOK.OQ', 'DAY.N', 'LAW.N', 'FIHu.TO', 'PGRE.N', 'ZVRA.OQ', 'CSWI.OQ', 'BDTX.OQ', 'FRPH.OQ', 'FMTY14.MX', 'ALVR.OQ', 'SUM.N', 'DEA.N', 'BIORQ.PK', 'SHAK.N', 'EHAB.N', 'CRVS.OQ', 'BIGC.OQ', 'IRM.N', 'KURA.OQ', 'KREF.N', 'XBIT.OQ', 'MNST.OQ', 'RVMD.OQ', 'HAPV3.SA', 'UNIT.OQ', 'BLND.N', 'NSA.N', 'ATXI.OQ', 'VRCA.OQ', 'BNED.N', 'NRDS.OQ', 'MBIO.OQ', 'HZO.N', 'SMHI.N', 'HCMC.PK', 'FOOD.TO', 'DASH.OQ', 'SRG.N', 'OKUR.OQ', 'CHCT.N', 'CURA.TO', 'FLNT.OQ', 'OVID.OQ', 'MTVA.OQ', 'EYEN.OQ', 'AM.N', 'CC.N', 'PYPL.OQ', 'BW.N', 'BLD.N', 'PTON.OQ', 'PJT.N', 'CJ.TO', 'CAVA.N', 'LITE.OQ', 'EVER.OQ', 'MDNA.TO', 'WINSF.PK', 'COLL.OQ', 'WING.OQ', 'ENR.N', 'SHC.OQ', 'AC.N', 'COOP.OQ', 'DNLI.OQ', 'OLLI.OQ', 'FSV.TO', 'PRTC.L', 'RS.N', 'LTH.N', 'IBRX.OQ', 'SMFT3.SA', 'RAPT.OQ', '688235.SS', 'AYTU.OQ', 'APR_u.TO', 'MSGS.N', 'PRPL.OQ', 'ALBT.OQ', 'VTVT.OQ', 'KZR.OQ', 'DEC.L', 'TWI.N', 'UNH.N', 'WAL.N', 'IDYA.OQ', 'QTTB.OQ', 'TPH.N', 'ACI.N', 'YCBD.A', 'ONDS.OQ', 'HPE.N', 'MATR.TO', 'UPST.OQ', 'CUE.OQ', 'PBL.TO', 'GUD.TO', 'SITES1A1.MX', 'LPTV.PK', 'CLPR.N', 'FCPT.N', 'HLI.N', 'FDMT.OQ', 'WSC.OQ', 'SGRY.OQ', 'SITE.N', 'CXSE3.SA', 'MOVI3.SA', 'H.TO', 'GTLB.OQ', 'XAIR.OQ', 'NGVT.N', 'PLRX.OQ', 'GRTSQ.PK', 'SIL.TO', 'AFIIQ.PK', 'BCSF.N', 'RRR.OQ', 'BBU_u.TO', 'OBDC.N', 'TERRA13.MX', 'TMCI.OQ', 'ELC.N', 'EVLO.PK', 'ELUT.OQ', 'ORIC.OQ', 'FTV.N', 'SENS.A', 'SCWX.OQ', 'HTZ.OQ', 'TRTN_pa.N', 'CMLS.OQ', 'SJ.TO', 'LACOMERUBC.MX', 'CCCC.OQ', 'NXTC.OQ', 'KMP_u.TO', 'ABCL.OQ', 'PR.N', 'CMTG.N', 'OSK.N', 'TECX.OQ', 'HCAT.OQ', 'LFVN.OQ', 'CBUS.OQ', 'PGNY.OQ', 'HCC.N', 'ENELCHILE.SN', 'MCBS.OQ', 'NEUE.N', 'FHTX.OQ', 'SPRY.OQ', 'ADT.N', 'ELVN.OQ', 'DD.N', 'SUP.N', 'SPRB.OQ', 'ASAPQ.PK', 'KROS.OQ', 'AGUA.MX', 'ITT.N', 'TLIS.PK', 'WCN.TO', 'ASH.N', 'SEZL.OQ', 'DFIN.N', 'CAE.TO', 'CWH.N', 'CTKB.OQ', 'GOLF.N', 'AA.N', 'SNDL.OQ', 'PLX.A', 'YETI.N', 'KNSL.N', 'TF.TO', 'FLGT.OQ', 'MNSB.OQ', 'BVS.OQ', 'FULC.OQ', 'VREX.OQ', 'KLDO.PK', 'YMAB.OQ', 'INDI.OQ', 'AMR.N', 'HGV.N', 'STTK.OQ', 'LW.N', 'ASIX.N', 'RCUS.N', 'PRTH.OQ', 'TUSK.OQ', 'ATZ.TO', 'TENB.OQ', 'RLAY.OQ', 'DBRG.N', 'TRTX.N', 'ACVA.OQ', 'VVV.N', 'ODTCD.PK', 'CNDT.OQ', 'AVTR.N', 'BL.OQ', 'IQV.N', 'SBAC.OQ', 'VST.N', 'BHF.OQ', 'TRMB.OQ', 'NODK.OQ', 'ACLX.OQ', 'BYSI.OQ', 'IIPR.N', 'ATUS.N', 'HFFG.OQ', 'FNB.N', 'INSG.OQ', 'TRDA.OQ', 'PRAX.OQ', 'ICHR.OQ', 'SJW.OQ', 'DNTH.OQ', 'IVQu.TO', 'TWM.TO', 'CRON.TO', 'TNYA.OQ', 'VRRM.OQ', 'NU.N', 'BCML.OQ', 'KEGX.PK', 'BORR.N', 'NYMXF.PK', 'IMXI.OQ', 'METC.OQ', 'FOSSALC1.LM', 'SWX.N', 'BBIO.OQ', 'CALC.OQ', 'INZY.OQ', 'VEL.N', 'KRRO.OQ', 'INVH.N', 'VIR.OQ', 'JBGS.N', 'PUMP.N', 'TIXT.TO', 'ULTA.OQ', 'HLNE.OQ', 'ESTA.OQ', 'QEC.TO', 'APLT.OQ', 'JILL.N', 'REKR.OQ', 'TSU.TO', 'GOOS.TO', 'OPBK.OQ', 'LBRT.N', 'WTTR.N', 'DK.N', 'CSSEQ.PK', 'JMSB.OQ', 'KNTK.N', 'GTE.A', 'DXC.N', 'BYMAm.BA', 'SEI.N', 'AADI.OQ', 'BRT.N', 'XCUR.OQ', 'IPCOR.ST', 'IMAB.OQ', 'FEC.TO', 'NCSM.OQ', 'WOW.N', 'BKR.OQ', 'CRDL.TO', 'NESR.OQ', 'EEX.N', 'CVNA.N', 'SER.A', 'SMPL.OQ', 'LOVE.OQ', 'ARIS.TO', 'PDLB.OQ', 'TOT.TO', 'WRAP.OQ', 'MGY.N', 'HYFM.OQ', 'PATH.N', 'ACET.OQ', 'CEU.TO', 'GPMT.N', 'BRY.OQ', 'VISTAA.MX', 'GMXT.MX', 'ECVT.N', 'MOLAm.BA', 'QTRH.TO', 'REAL.OQ', 'IOVA.OQ', 'LEGN.OQ', 'TI.TO', 'BY.N', 'ELTP.PK', 'ROOT.TO', 'AKTSQ.PK', 'CVHm.BA', 'CNR.N', 'MDA.TO', 'CTOS.N', 'NFE.OQ', 'AKYA.OQ', 'RNGR.N', 'SES.TO', 'STNE.OQ', 'CNQ.TO', 'ANGI.OQ', 'SWI.N', 'HBB.N', 'COIN.OQ', 'KRYS.OQ', 'RBBN.OQ', 'DESP.N', 'CNNE.N', 'VICI.N', 'ALEX.N', 'DOCN.N', 'SPCE.N', 'LVO.OQ', 'ERO.TO', 'CELC.OQ', 'FVCB.OQ', 'CARG.OQ', 'TBIO.PK', 'RIOT.OQ', 'TRAXIONA.MX', 'EVRG.OQ', 'EYE.OQ', 'ST.N', 'FNKO.OQ', 'ATHA.OQ', 'RMNI.OQ', 'CTSO.OQ', 'LABS.TO', 'PRLD.OQ', 'AIM.TO', 'NMRK.OQ', 'OLA.TO', 'FNCH.PK', 'RFL.N', 'HYREQ.PK', 'ALTR.OQ', 'JAMF.OQ', 'GFL.TO', 'LILA.OQ', 'BRSP.N', 'ILPT.OQ', 'GORV.OQ', 'BTAI.OQ', 'OMIC.OQ', 'RXO.N', 'IKNA.OQ', 'ORA.TO', 'AGS.N', 'LEVGQ.PK', 'VMD.OQ', 'BTBT.OQ', 'BHa.N', 'NTR.TO', 'PAGS.N', 'GTES.N', 'AMTB.N', 'TALO.N', 'AHCO.OQ', 'RDVT.OQ', 'FTCI.OQ', 'SEER.OQ', 'GOSS.OQ', 'GBIO.OQ', 'SRRK.OQ', 'LNW.OQ', 'WHD.N', 'HYMC.OQ', 'CUERVO.MX', 'ALHC.OQ', 'JOBY.N', 'CTS.TO', 'AVGO.OQ', 'TLRY.OQ', 'IBEX.OQ', 'NKTX.OQ', 'AURE3.SA', 'DCBO.TO', 'NWN.N', 'HLS.TO', 'ALYA.TO', 'SVI.TO', 'ANTX.OQ', 'ABUS.OQ', 'ARGH.V', 'BEAM.OQ', 'CHX.OQ', 'GSHD.OQ', 'ALLO.OQ', 'CVEO.N', 'VRT.N', 'DXT.TO', 'RA.MX', 'POU.TO', 'HOMu.TO', 'NKLA.OQ', 'ACRV.OQ', 'RLYB.OQ', 'QIPT.TO', 'MGTX.OQ', 'ACA.N', 'CI.N', 'LSEA.OQ', 'KRON.OQ', 'MI_u.TO', 'EPRT.N', 'IIIV.OQ', 'CMPX.OQ', 'VET.TO', 'VAMO3.SA', 'FFWM.N', 'ERE_u.TO', 'PLSE.OQ', 'REPL.OQ', 'DIS.N', 'AKRO.OQ', 'CWEB.TO', 'ARLO.N', 'BLDP.TO', 'SSL.TO', 'TTCFQ.PK', 'JWEL.TO', 'HLTHQ.PK', 'BXSL.N', 'OTEX.TO', 'KLXE.OQ', 'OFIX.OQ', 'FTDR.OQ', 'PFSI.N', 'NXE.TO', 'ETON.OQ', 'U.N', 'CBST.NLB', 'REZI.N', 'NGD.TO', 'BSVN.OQ', 'PXT.TO', 'GDYN.OQ', 'RVLPQ.PK', 'EQ.OQ', 'IMG.TO', 'ADWa.TO', 'VNP.TO', 'PASG.OQ', 'ALGS.OQ', 'EGLX.TO', 'RAMP.N', 'FCBC.OQ', 'KLDI.PK', 'MRKR.OQ', 'CTVA.N', 'ADN.OQ', 'TMHC.N', 'FTHM.OQ', 'MRNA.OQ', 'ALEC.OQ', 'GRYP.OQ', 'MIRM.OQ', 'CABA.OQ', 'OSW.OQ', 'SNCY.OQ', 'JANX.OQ', 'ABX.TO', 'LAZR.OQ', 'ERAS.OQ', 'KRUS.OQ', 'KRT.OQ', 'PHUN.OQ', 'IAG.TO', 'FOXA.OQ', 'SANA.OQ', 'MDU.N', 'TSBX.OQ', 'NRDE.PK', 'CYCN.OQ', 'RLI.N', 'MVST.OQ', 'AOMR.N', 'HYLN.A', 'HOOK.OQ', 'GOEV.OQ', 'KFS.N', 'ORGO.OQ', 'HOWL.OQ', 'PSA.N', 'CLYM.OQ', 'DM.N', 'KTB.N', 'ANIK.OQ', 'TW.OQ', 'HONE.OQ', 'RMBI.OQ', 'ALTG.N', 'AIOT.OQ', 'PLMR.OQ', 'XRX.OQ', 'ARTG.V', 'ARCT.OQ', 'TH.OQ', 'GNLN.OQ', 'OTIS.N', 'TROX.N', 'PSTL.N', 'DYN.OQ', 'CCO.N', 'TMDX.OQ', 'SWTX.OQ', 'PSN.N', 'IMVT.OQ', 'PNTG.OQ', 'LYEL.OQ', 'CELU.OQ', 'FLNG.OL', 'ATER.OQ', 'GO.OQ', 'PHAT.OQ', 'NTCO3.SA', 'SPRU.N', 'CISO.OQ', 'CMBM.OQ', 'PACK.N', 'SKIL.N', 'SCPS.PK', 'TARS.OQ', 'AFYA.OQ', 'MOGO.TO', 'NOVA.N', 'ADV.OQ', 'IPSC.OQ', 'CENCOMALLS.SN', 'CHPT.N', 'RELY.OQ', 'RPAY.OQ', 'ONEW.OQ', 'NVST.N', 'DT.N', 'AVPT.OQ', 'CRNC.OQ', 'BLDE.OQ', 'ASTS.OQ', 'EWTX.OQ', 'DMTKQ.PK', 'APRE.OQ', 'BSBK.OQ', 'HQI.OQ', 'OMGA.OQ', 'BWIN.OQ', 'ROOT.OQ', 'MNTS.OQ', 'AVAH.OQ', 'TIL.OQ', 'CURI.OQ', 'ETNB.OQ', 'BKSY.N', 'DSGN.OQ', 'RMTI.OQ', 'NUVL.OQ', 'ARQT.OQ', 'PINE.N', 'BXRXQ.PK', 'PRCH.OQ', 'TFPM.TO', 'BEPC.N', 'HOFV.OQ', 'BILL.N', 'PRVA.OQ', 'REYN.OQ', 'XP.OQ', 'ACEL.N', 'AND.TO', 'HPK.OQ', 'UWMC.N', 'NREF.N', 'OI.N', 'PVBC.OQ', 'BYD.TO', 'ZNTL.OQ', 'CGEM.OQ', 'AEVA.N', 'THRD.OQ', 'TYRA.OQ', 'BGXX.PK', 'AVTE.OQ', 'OVV.N', 'CRSR.OQ', 'NTST.N', 'TCBX.OQ', 'MPLN.N', 'RSI.N', 'ASLE.OQ', 'CARR.N', 'AZEK.N', 'SKLZ.N', 'VNT.N', 'VINC.OQ', 'TOI.OQ', 'ZEVY.PK', 'DNMR.PK', 'SPHR.N', 'MP.N', 'FWRG.OQ', 'WEST.OQ', 'LPRO.OQ', 'GAN.OQ', 'ADEA.OQ', 'WOOF.OQ', 'BAM.N', 'HSDT.OQ', 'CRDO.OQ', 'ASRT.OQ', 'ITOS.OQ', 'PRG.N', 'AUR.OQ', 'FCR_u.TO', 'AVO.OQ', 'MIR.N', 'EOSE.OQ', 'APG.N', 'TSHA.OQ', 'BFLY.N', 'PLBY.OQ', 'FOUR.N', 'SGA.OQ', 'TRAD3.SA', 'FTAI.OQ', 'ML.N', 'QS.N', 'DNB.N', 'GLSHQ.PK', 'VERV.OQ', 'NUVB.N', 'EBC.OQ', 'CNMD.N', 'RCEL.OQ', 'ALXO.OQ', 'DND.TO', 'CMAXQ.PK', 'FFIE.OQ', 'ODP.OQ', 'AOUT.OQ', 'VSTA.OQ', 'RKT.N', 'GOCO.OQ', 'RXT.OQ', 'KYMR.OQ', 'LENZ.OQ', 'EVLV.OQ', 'LCID.OQ', 'BCAB.OQ', 'DMSLQ.PK', 'WWOK.PK', 'IVVD.OQ', 'CZR.OQ', 'ATIP.PK', 'SPIR.N', 'IRON.OQ', 'HRMY.OQ', 'IDCC.OQ', 'STEM.N', 'TNGX.OQ', 'QSI.OQ', 'GCMG.OQ', 'HUMA.OQ', 'LQDA.OQ', 'VTYX.OQ', 'CASH3.SA', 'WGS.OQ', 'OWLT.N', 'STEP.OQ', 'CIFR.OQ', 'BBCP.OQ', 'OPFI.N', 'AFCG.OQ', 'GBLI.N', 'UTZ.N', 'ARX.TO', 'LSF.A', 'AD_u.TO', 'IAS.OQ', 'CTO.N', 'PWP.OQ', 'ALA.TO', 'ASO.OQ', 'SKIN.OQ', 'EVGO.OQ', 'CLNN.OQ', 'ARKO.OQ', 'SEAT.OQ', 'XPER.N', 'BBLNF.PK', 'HLMN.OQ', 'SMMT.OQ', 'HYZN.OQ', 'BDVC.PK', 'GLUE.OQ', 'OPAD.N', 'DCGO.OQ', 'BARK.N', 'BBXIA.PK', 'GHLD.N', 'VIGL.OQ', 'ACHR.N', 'MTTR.OQ', 'CMPO.OQ', 'MAX.N', 'LIDR.OQ', 'LESL.OQ', 'BODI.N', 'PYYX.PK', 'ISPO.OQ', 'IONQ.N', 'EVEX.N', 'FORA.OQ', 'SFC.TO', 'MRVI.OQ', 'EBET.PK', 'NVTS.OQ', 'FOA.N', 'AMPS.N', 'CERT.OQ', 'ENVX.OQ', 'XPOF.N', 'VERA.OQ', 'PCT.OQ', 'CARE.OQ', 'RSVR.OQ', 'PRST.PK', 'PRME.OQ', 'DRVN.OQ', 'PEPG.OQ', 'MNTK.OQ', 'TPL.N', 'BHIL.OQ', 'PDYN.OQ', 'OPEN.OQ', 'DFH.N', 'SOND.OQ', 'VMEO.OQ', 'IAU.TO', 'PAX.OQ', 'SHLS.OQ', 'SEMR.N', 'BMEA.OQ', 'CLOV.OQ', 'TERN.OQ', 'UHG.OQ', 'PIII.OQ', 'GROY.A', 'NRGV.N', 'BMBL.OQ', 'SNSE.OQ', 'SMRT.N', 'HIMS.N', 'BBAI.N', 'HLGN.PK', 'ETWO.N', 'STGW.OQ', 'JBI.N', 'INNV.OQ', 'GREE.OQ', 'BTMD.OQ', 'SLDP.OQ', 'KIND.N', 'SOUN.OQ', 'LNZA.OQ', 'DAWN.OQ', 'PAYO.OQ', 'PL.N', 'HAYW.N', 'CRCT.OQ', 'MLYS.OQ', 'FNA.N', 'CSCO.OQ', 'VGAS.OQ', 'APA.OQ', 'OGN.N', 'AGL.N', 'OUST.OQ', 'DV.N', 'TSPH.PK', 'SKYT.OQ', 'VAXX.PK', 'BWMN.OQ', 'FYBR.OQ', 'LFST.OQ', 'LICY.N', 'BLFY.OQ', 'KORE.N', 'PYCR.OQ', 'ZETA.N', 'VAL.N', 'DTM.N', 'SVV.N', 'OWL.N', 'POWW.OQ', 'CNM.N', 'ZURA.OQ', 'ASTL.OQ', 'TEGA3.SA', 'HSHP.OL', 'PLL.OQ', 'EVCM.OQ', 'DNUT.OQ', 'PROK.OQ', 'YOU.N', 'CURV.N', 'CRGY.N', 'GXO.N', 'NAUT.OQ', 'NN.OQ', 'VWESQ.PK', 'SOFI.OQ', 'MAPS.OQ', 'ME.OQ', 'APO.N', 'JXN.N', 'ZVIA.N', 'ORGN.OQ', 'VSCO.N', 'TSAT.OQ', 'TSVT.OQ', 'MDVLQ.PK', 'XERS.OQ', 'COOK.N', 'ALIT.N', 'NABL.N', 'CDRE.N', 'UP.N', 'MKFG.N', 'U_u.TO', 'GPOR.N', 'OCEA.OQ', 'MKTW.OQ', 'BBUC.TO', 'SLVM.N', 'HIPO.N', 'HLVX.OQ', 'CCCS.OQ', 'IE.A', 'CCSI.OQ', 'PRCT.OQ', 'WN.TO', 'BROS.N', 'DH.OQ', 'XOS.OQ', 'AKA.N', 'RKLB.OQ', 'OLPX.OQ', 'ADTN.OQ', 'NVCT.OQ', 'DFY.TO', 'RDW.N', 'PTLO.OQ', 'SDE.TO', 'DNA.N', 'AVDX.OQ', 'NRDY.N', 'RBOT.N', 'VCSA.OQ', 'ARIS.N', 'PX.N', 'FLNC.OQ', 'VLDX.PK', 'KD.N', 'INFA.N', 'ONL.N', 'DTC.N', 'ARRY.OQ', 'AIRS.OQ', 'NMRA.OQ', 'DKNG.OQ', 'LVLU.OQ', 'BKKT.N', 'GDI.TO', 'REFI.OQ', 'SLND.A', 'TSE.N', 'DOUG.N', 'OCTO.OQ', 'ESAB.N', 'ZIMV.OQ', 'ACDC.OQ', 'LOCL.N', 'CTV.N', 'CEG.OQ', 'LVWR.N', 'LUCK.N', 'TPG.OQ', 'FCFS.OQ', 'BRBR.N', 'EMBC.OQ', 'USCB.OQ', 'PMZ_u.TO', 'CVE.TO', 'EE.N', 'SDRL.N', 'AZPN.OQ', 'NCNO.OQ', 'QDEL.OQ', 'SES.N', 'FEAM.OQ', 'GWH.N', 'RGTI.OQ', 'BPRN.OQ', 'BHM.A', 'DINO.N', 'CXT.N', 'FRT.N', 'FRGE.N', 'CS.TO', 'S.TO', 'DRRu.TO', 'FIP.OQ', 'SMR.N', 'GRNT.N', 'COLD.N', 'KKR.N', 'SYM.OQ', 'GBTG.N', 'ZYME.OQ', 'MOND.PK', 'PIF.TO', 'NOTE.N', 'BHVN.N', 'SPXC.N', 'WELL.N', 'J.N', 'AMPX.N', 'LASITEB1.MX', 'NE.N', 'AEI.OQ', 'STR.N', 'GEHC.OQ', 'PKI.TO', 'DIV.TO', 'MBC.N', 'OABI.OQ', 'KVUE.N', 'GRND.N', 'TCS.TO', 'BATRA.OQ', 'ALTI.OQ', 'KIM.N', 'OBIO.OQ', 'ISPR.OQ', 'STHO.OQ', 'NXT.OQ', 'DOC.N', 'MSGE.N', 'CRGX.OQ', 'RAYa.TO', 'KNF.N', 'HUT.OQ', 'CR.N', 'DFLI.OQ', 'BGC.OQ', 'CLB.N', 'LBTYA.OQ', 'FTRE.OQ', 'KRMD.OQ', 'PHIN.N', 'FIBK.OQ', 'TKO.N', 'BG.N', 'APGE.OQ', 'SN.N', 'GRAM.NLB', 'DSG.TO', 'KLG.N', 'VLTO.N', 'NWE.OQ', 'HHH.N', 'VFF.OQ', 'Y.TO', 'XAU.TO', 'FREY.N', 'TA.TO', 'FUN.N', 'IXHL.OQ', 'ARGO_pa.N', 'PVF_u.V', 'AMRX.OQ', 'LAZ.N', 'ATH_pa.N', 'INBX.OQ', 'MPVD.TO', 'SIRI.OQ', 'TOU.TO', 'CENN.OQ', 'AEM.TO', 'FERG.N', 'AFN.TO', 'CSIQ.OQ', 'SGY.TO', 'K.TO', 'LH.N', 'IMAX.N', 'TSLA.OQ', 'TPZ.TO', 'WCP.TO', 'HASI.N', 'NPI.TO', 'TK.N', 'NRC.OQ', 'TNK.N', 'PRMB.N', 'PAMT.OQ', 'NEUP.OQ', 'CNE.TO', 'TTD.OQ', 'AGI.TO', 'DC.A', 'BIPC.N', 'HBM.TO', 'NDAQ.OQ', 'WFC.N', 'USB.N', 'RY.TO', 'RJF.N', 'MCO.N', 'C.N', 'SCHW.N', 'CF.TO', 'BAC.N']
asianRICs = ['OMH.AX', 'SALAM.BH', 'BEYON.BH', 'BISB.BH', 'NATB.BH', 'GHG.BH', 'BCFC.BH', 'ITHMR.BH', 'DUTY.BH', 'INOV.BH', 'BKIC.BH', 'BNHB.BH', 'GFHB.BH', 'EICB.BH', 'UGIC.BH', 'BBKB.BH', '0085.HK', '3918.HK', '002036.SZ', '002138.SZ', '600000.SS', '600017.SS', '000883.SZ', '600166.SS', '002080.SZ', '000718.SZ', '600588.SS', '600428.SS', '600061.SS', '002151.SZ', '000543.SZ', '000553.SZ', '600690.SS', '000887.SZ', '0817.HK', '600705.SS', '002081.SZ', '601939.SS', '600743.SS', '0425.HK', '601988.SS', '600879.SS', '000066.SZ', '000738.SZ', '0867.HK', '600783.SS', '002127.SZ', '600089.SS', '600688.SS', '600312.SS', '600546.SS', '600266.SS', '600556.SS', '000636.SZ', '002044.SZ', '600827.SS', '600095.SS', '000598.SZ', '600886.SS', '601398.SS', '000739.SZ', '600196.SS', '600881.SS', '600497.SS', '600177.SS', '600460.SS', '000550.SZ', '600271.SS', '600352.SS', '000813.SZ', '000685.SZ', '002092.SZ', '600236.SS', '600036.SS', '000959.SZ', '000933.SZ', '002128.SZ', '600655.SS', '600129.SS', '000009.SZ', '000002.SZ', '601088.SS', '600606.SS', '600864.SS', '600009.SS', '600604.SS', '000733.SZ', '000758.SZ', '600481.SS', '000617.SZ', '000877.SZ', '600885.SS', '002129.SZ', '002217.SZ', '600346.SS', '601009.SS', '000012.SZ', '600390.SS', '600030.SS', '000810.SZ', '002048.SZ', '600315.SS', '600612.SS', '000501.SZ', '600623.SS', '600118.SS', '000031.SZ', '000688.SZ', '600718.SS', '000970.SZ', '000627.SZ', '600155.SS', '600233.SS', '600273.SS', '600153.SS', '002041.SZ', '600478.SS', '600859.SS', '002010.SZ', '600010.SS', '600649.SS', '600640.SS', '600673.SS', '600643.SS', 'SVA.OQ', '600675.SS', '000513.SZ', '600674.SS', '001872.SZ', '000028.SZ', '000408.SZ', '002191.SZ', '600600.SS', '000800.SZ', 'YNLG.SI', '600648.SS', '000807.SZ', '600309.SS', '600663.SS', '600150.SS', '1530.HK', '000785.SZ', '600372.SS', '600350.SS', '600737.SS', '002142.SZ', '601975.SS', '601808.SS', '600252.SS', '002032.SZ', '002192.SZ', '000400.SZ', '600869.SS', '600064.SS', '1666.HK', '3900.HK', '600803.SS', '000723.SZ', '600741.SS', '000538.SZ', '601666.SS', '0848.HK', '600418.SS', '600015.SS', '600519.SS', '600378.SS', '600839.SS', '000998.SZ', '600863.SS', '002123.SZ', '000039.SZ', '000061.SZ', '600158.SS', '600801.SS', '600820.SS', '000425.SZ', '002124.SZ', '000987.SZ', '000539.SZ', '600398.SS', '000977.SZ', '600348.SS', '600056.SS', '600845.SS', '600012.SS', '600776.SS', '601333.SS', '3333.HK', '600754.SS', '000921.SZ', '000581.SZ', '000596.SZ', '000630.SZ', '000625.SZ', '000089.SZ', '2382.HK', '600094.SS', '600157.SS', '000878.SZ', '601727.SS', '600562.SS', '000988.SZ', '000776.SZ', '3998.HK', '600020.SS', '000338.SZ', '600583.SS', '000301.SZ', '600332.SS', '002215.SZ', '601991.SS', '600764.SS', '600169.SS', '1393.HK', '600221.SS', '600887.SS', '600320.SS', '600548.SS', '900948.SS', '600115.SS', '600362.SS', '000898.SZ', '600029.SS', '002176.SZ', '002216.SZ', '000623.SZ', '600516.SS', '600039.SS', '600276.SS', '0576.HK', '601699.SS', '600585.SS', '600377.SS', '000488.SZ', '600812.SS', '002203.SZ', '600808.SS', 'MLCO.OQ', '0696.HK', '002202.SZ', '600016.SS', '600699.SS', '1133.HK', '600026.SS', '0073.HK', '000839.SZ', '601186.SS', '000792.SZ', '600048.SS', '600639.SS', '2233.HK', '601600.SS', '000062.SZ', '2007.HK', '600098.SS', '000683.SZ', '600835.SS', '600862.SS', '600837.SS', '3377.HK', '1882.HK', '601800.SS', '600755.SS', '0656.HK', '601633.SS', '600006.SS', '000963.SZ', '000778.SZ', '002065.SZ', '600848.SS', '600028.SS', '000826.SZ', 'NTES.OQ', '601328.SS', '600711.SS', '000157.SZ', '000401.SZ', '600188.SS', '600707.SS', '000027.SZ', '600635.SS', '0981.HK', '002064.SZ', '2689.HK', '000893.SZ', '000021.SZ', '0813.HK', '000960.SZ', '000661.SZ', '600027.SS', '600298.SS', '000717.SZ', '600395.SS', '600498.SS', '600567.SS', '002056.SZ', '600563.SS', '600875.SS', '0337.HK', '600871.SS', 'AIFU.OQ', '000725.SZ', '600415.SS', '000799.SZ', '600765.SS', '601005.SS', '600295.SS', '600329.SS', '002025.SZ', '001914.SZ', '002185.SZ', '600970.SS', '000825.SZ', '000001.SZ', '3808.HK', '600120.SS', '000750.SZ', '603993.SS', '600072.SS', '000078.SZ', '000519.SZ', '688385.SS', '002008.SZ', '2777.HK', '600446.SS', '000999.SZ', '0581.HK', '000983.SZ', '002179.SZ', '000690.SZ', '601601.SS', '600637.SS', '000537.SZ', '600436.SS', '600884.SS', '601168.SS', '600201.SS', '601166.SS', '601998.SS', '000990.SZ', '601811.SS', '600566.SS', '600038.SS', '601318.SS', '000032.SZ', '600549.SS', '601099.SS', '600816.SS', '3323.HK', '600176.SS', '600515.SS', '600011.SS', '601628.SS', '600388.SS', '600483.SS', '000703.SZ', '600022.SS', '000869.SZ', '600435.SS', '600760.SS', '000997.SZ', '600259.SS', '000828.SZ', '600167.SS', 'BIDU.OQ', '000555.SZ', '601003.SS', '600507.SS', '600521.SS', '000712.SZ', '000768.SZ', '600109.SS', '600018.SS', '000902.SZ', '600426.SS', '601857.SS', '600143.SS', '600216.SS', '600633.SS', '000708.SZ', '000895.SZ', '600582.SS', '0546.HK', '0819.HK', '600804.SS', '600160.SS', '601880.SS', '601866.SS', '002085.SZ', '600131.SS', '000930.SZ', '002028.SZ', '600406.SS', '600141.SS', '600486.SS', '600416.SS', '600031.SS', '600809.SS', '600873.SS', '600550.SS', '600399.SS', '600511.SS', '600739.SS', '002001.SZ', '000600.SZ', '000517.SZ', '600895.SS', '002024.SZ', '002110.SZ', '600522.SS', '0658.HK', '1066.HK', '600019.SS', '600299.SS', '000967.SZ', '002074.SZ', '002049.SZ', '000423.SZ', '600062.SS', '600517.SS', '600021.SS', '1813.HK', '600667.SS', '601898.SS', '600456.SS', '002212.SZ', '600528.SS', '600642.SS', '0151.HK', '600307.SS', '000156.SZ', '002004.SZ', '600380.SS', '000783.SZ', '002078.SZ', '600645.SS', '000728.SZ', '600007.SS', '0700.HK', '0489.HK', '000069.SZ', '600170.SS', 'YAZG.SI', '002174.SZ', '1818.HK', '600536.SS', '000059.SZ', '600728.SS', '002152.SZ', '600316.SS', '000981.SZ', '000786.SZ', '3983.HK', '600535.SS', '000767.SZ', '002120.SZ', '600745.SS', '600580.SS', '601390.SS', '600717.SS', '000860.SZ', '000563.SZ', '600985.SS', 'EDU.N', '600409.SS', 'WUMSY.PK', '600369.SS', '0410.HK', '002145.SZ', '002594.SZ', '601006.SS', '1888.HK', 'CEWL.SI', '600258.SS', '600219.SS', '002030.SZ', '600995.SS', '002153.SZ', '600132.SS', '600795.SS', '0013.HK', '600657.SS', '600732.SS', '600339.SS', '000568.SZ', '0189.HK', '600733.SS', '600008.SS', '000063.SZ', '000937.SZ', '600383.SS', '000686.SZ', '3818.HK', '3883.HK', '600126.SS', '002146.SZ', '600050.SS', '0460.HK', '600529.SS', '002050.SZ', '600161.SS', '002075.SZ', '000681.SZ', '601169.SS', '600729.SS', '600256.SS', '2328.HK', '601607.SS', '600811.SS', '600575.SS', '600066.SS', '0272.HK', '000050.SZ', '600037.SS', '000090.SZ', '002155.SZ', '0552.HK', '600373.SS', '000547.SZ', '000656.SZ', '600704.SS', '600085.SS', '3383.HK', '600060.SS', '600410.SS', '000100.SZ', '002156.SZ', '002019.SZ', '600267.SS', '600782.SS', '000088.SZ', '3899.HK', '600338.SS', '3339.HK', '600572.SS', '600988.SS', '000559.SZ', '000709.SZ', '600872.SS', '2005.HK', '000060.SZ', '3888.HK', '000629.SZ', '600282.SS', '002157.SZ', '600559.SS', '601598.SS', '000158.SZ', '9961.HK', '000402.SZ', '600195.SS', '600867.SS', '000876.SZ', '600578.SS', '600004.SS', '600208.SS', '600325.SS', '000831.SZ', '600104.SS', '600598.SS', '601899.SS', '000932.SZ', '002007.SZ', '000729.SZ', '002195.SZ', '002038.SZ', '600787.SS', '000958.SZ', '600171.SS', '600779.SS', '600703.SS', '600100.SS', '600900.SS', '000651.SZ', '600482.SS', '600893.SS', '601872.SS', '600183.SS', '600518.SS', '000591.SZ', '601728.SS', '000528.SZ', '600487.SS', '0220.HK', '600763.SS', '601111.SS', '600096.SS', '600570.SS', '002221.SZ', '2313.HK', '600777.SS', '000830.SZ', '002099.SZ', '600489.SS', 'KEDAq.S', '600392.SS', '600111.SS', '000975.SZ', '000155.SZ', '3337.HK', '000858.SZ', '688187.SS', '600284.SS', '002131.SZ', '600584.SS', '600702.SS', '601919.SS', '3368.HK', '600151.SS', '0694.HK', '600340.SS', '000415.SZ', '600079.SS', '600376.SS', 'CAAS.OQ', '600967.SS', '600597.SS', '600438.SS', '600547.SS', '000938.SZ', '600660.SS', '2688.HK', '0685.HK', '1083.HK', '0127.HK', '0868.HK', '2388.HK', '0083.HK', '2356.HK', '2357.HK', '0267.HK', '0709.HK', '0506.HK', '1199.HK', '0285.HK', '0682.HK', '1093.HK', '0440.HK', '0291.HK', '0345.HK', '0570.HK', '0123.HK', '0200.HK', '0551.HK', '1177.HK', '1038.HK', '0322.HK', '0148.HK', '1193.HK', '0603.HK', '1060.HK', '0683.HK', '0257.HK', '0363.HK', '1052.HK', '0210.HK', '0604.HK', '0135.HK', '0823.HK', '0330.HK', '0027.HK', '0276.HK', '0031.HK', '0241.HK', '0268.HK', '0152.HK', '0242.HK', '0175.HK', '0032.HK', '0014.HK', '0003.HK', '0659.HK', '0012.HK', '0185.HK', '0081.HK', '0179.HK', '0035.HK', '0751.HK', '0270.HK', '0686.HK', '8083.HK', '0966.HK', '0053.HK', '0308.HK', '0144.HK', '0522.HK', '0992.HK', '0069.HK', '0688.HK', '0612.HK', '0812.HK', '0119.HK', '0142.HK', '2314.HK', '0341.HK', '0293.HK', '600941.SS', 'MOIL.SI', 'DFIR.SI', '0002.HK', '1109.HK', '0316.HK', '0010.HK', '0004.HK', '1114.HK', 'JARD.SI', '0006.HK', '0101.HK', '0392.HK', '0011.HK', '0016.HK', '0019.HK', '0178.HK', '0017.HK', '0165.HK', '0855.HK', '0511.HK', '0055.HK', '0639.HK', '600938.SS', '0384.HK', '1044.HK', '0754.HK', '0371.HK', '1050.HK', '1125.HK', '0136.HK', '0336.HK', '0467.HK', '1224.HK', '0934.HK', '0978.HK', '0737.HK', '0297.HK', '2343.HK', '0303.HK', '3800.HK', '0354.HK', '2018.HK', '3311.HK', '0008.HK', '2778.HK', '688396.SS', '2331.HK', '1999.HK', '0762.HK', '0066.HK', '3389.HK', '1313.HK', '2319.HK', '1068.HK', '2877.HK', '0836.HK', '2380.HK', '0023.HK', '2038.HK', '0607.HK', '1883.HK', 'SPRC.NS', 'ARTI.NS', 'SAMD.NS', 'KAMH.BO', 'NATP.NS', 'JKBK.NS', 'VNTI.NS', 'GRWL.NS', 'UBBW.NS', 'CYIE.NS', 'NEFI.NS', 'TIME.NS', 'INED.NS', 'SAEM.NS', 'HIMD.NS', 'BASF.NS', 'FOHE.NS', 'IONX.NS', 'GMMP.NS', 'UNBK.NS', 'KENI.BO', 'BECK.BO', 'BRIT.NS', 'BAYE.NS', 'UNTE.NS', 'NEST.NS', 'GDFR.NS', 'GLAX.NS', 'GHCH.NS', 'AKZO.NS', 'TVSM.NS', 'SANO.NS', 'MNFL.NS', 'SKFB.NS', 'JINW.NS', 'SGRC.NS', 'HGSL.NS', 'SUNT.NS', 'PVRL.NS', 'PAFO.NS', 'HLEG.NS', 'INBA.NS', 'OLEC.NS', 'BORO.NS', 'RAID.NS', 'SRES.NS', 'HEGL.NS', 'APLA.NS', 'STWH.NS', 'ACLL.NS', 'KTKM.NS', 'GRPL.NS', 'LART.NS', 'ACC.NS', 'TISC.NS', 'TREI.NS', 'IGLY.NS', 'ITC.NS', 'HEID.NS', 'JKLC.NS', 'BFRG.NS', 'ARVN.NS', 'CAST.NS', 'TMIN.NS', 'GMDC.NS', 'DIVI.NS', 'SADH.NS', 'HBLO.NS', 'ARBN.NS', 'GODI.NS', 'BPCL.NS', 'BRGR.NS', 'NIRL.BO', 'HPCL.NS', 'MRPL.NS', 'TATA.NS', 'PROR.NS', 'ERDY.NS', 'ALKY.NS', 'AISG.NS', 'GRPH.NS', 'UFLX.NS', 'NIIT.NS', 'NCCL.NS', 'VENK.NS', 'PAUS.BO', 'CBI.NS', 'ADEL.NS', 'CHLA.NS', 'TOPO.NS', 'KAPT.NS', 'ORCL.NS', 'PRFO.NS', 'BRSN.NS', 'NLCI.NS', 'HZNC.NS', 'IGAS.NS', 'JCHA.NS', 'BHEL.NS', 'BJFN.NS', 'FCTL.NS', 'NALU.NS', 'KBNK.NS', 'SAIL.NS', 'CERA.NS', 'JAIA.NS', 'CRSL.NS', 'CNBK.NS', 'JYOI.NS', 'IOC.NS', 'HROM.NS', 'TORP.NS', 'VOLT.NS', 'THMX.NS', 'MAHM.NS', 'TTCH.NS', 'ZEE.NS', 'NOCI.NS', 'CHPC.NS', 'EPLI.NS', 'SNFS.NS', 'INFY.NS', 'CIPL.NS', 'ONGC.NS', 'FED.NS', 'JIND.NS', 'MAST.NS', 'CHMB.NS', 'LIND.NS', 'PIRA.NS', 'LMVL.NS', 'TAMO.NS', 'IDBI.NS', 'PLMD.NS', 'PROC.NS', 'BEML.NS', 'TCS.NS', 'DSTV.NS', 'VST.NS', 'SUZL.NS', 'TVSH.NS', 'AVNT.NS', 'JAIC.NS', 'AHLU.NS', 'APCI.NS', 'PNBK.NS', 'SWAN.NS', 'GSPT.NS', 'TIPS.NS', 'CIEA.NS', 'ATOA.NS', 'IFBI.NS', 'HONE.NS', 'UPLL.NS', 'MRTI.NS', 'DPNT.NS', 'WLSP.NS', 'BACH.NS', 'SHOP.NS', 'CNFH.NS', 'ELGE.NS', 'BANN.NS', 'DHNP.NS', 'BLUS.NS', 'DCMS.NS', 'THOM.NS', 'BOB.NS', 'JAIR.NS', 'BOI.NS', 'PIDI.NS', 'EIDP.NS', 'SCHE.NS', 'FLTX.NS', 'REDY.NS', 'TNSL.NS', 'JSTL.NS', 'APLH.NS', 'GILE.NS', 'RLCP.NS', 'MALD.NS', 'HLL.NS', 'JBMA.NS', 'SEAM.NS', 'STYE.NS', 'DLF.NS', 'RYMD.NS', 'JKIN.NS', 'BRLC.NS', 'BATA.NS', 'GRAS.NS', 'THRM.NS', 'RELI.NS', 'MRF.NS', 'VDAN.NS', 'TTPW.NS', 'USBL.NS', 'SUBR.NS', 'AGRO.NS', 'MMFS.NS', 'BRTI.NS', 'PIVI.NS', 'SUTV.NS', 'DELT.NS', 'HDIL.NS', 'ICMN.NS', 'ECLE.NS', 'SDCH.NS', 'SIYR.NS', 'CGPO.NS', 'ASPN.NS', 'FNXC.NS', 'INGR.NS', 'CUMM.NS', 'NAVN.NS', 'SUPI.NS', 'GRVL.NS', 'ABB.NS', 'ITEL.NS', 'CNTP.NS', 'SPTL.NS', 'PTCI.NS', 'AXBK.NS', 'FDC.NS', 'CEAT.NS', 'CORF.NS', 'BOSH.NS', 'BJAT.NS', 'ENGI.NS', 'TACN.NS', 'GNFC.NS', 'JKPA.NS', 'ELST.NS', 'JNSW.NS', 'AITE.NS', 'HFCL.NS', 'SHME.NS', 'TRCE.NS', 'GAIL.NS', 'PFIZ.NS', 'PGRD.NS', 'GMRI.NS', 'TINV.NS', 'EXID.NS', 'COLG.NS', 'HDBK.NS', 'CESC.NS', 'IHTL.NS', 'JBCH.NS', 'ABOT.NS', 'PLYP.NS', 'TREN.NS', 'POON.NS', 'RSTC.NS', 'WHIR.NS', 'SOBH.NS', 'JUBA.NS', 'GRNN.NS', 'CSFL.NS', 'UNOI.NS', 'DPFE.NS', 'TITN.NS', 'BLKI.NS', 'GALK.NS', 'SRID.NS', 'APLO.NS', 'ABUJ.NS', 'MBFL.NS', 'DABU.NS', 'CRBR.NS', 'KANE.NS', 'DLMI.NS', 'ESAB.NS', 'JIST.NS', 'KSBL.NS', 'KIRP.NS', 'GOCP.NS', 'SUPE.NS', 'AEGS.NS', 'APSE.NS', 'APAR.NS', 'YESB.NS', 'WGSR.NS', 'DCBA.NS', 'RLCM.NS', 'CHOL.NS', 'GDYR.BO', 'VODA.NS', 'JKCE.NS', 'SRFL.NS', 'MMTC.NS', 'TEML.NS', 'CCLP.NS', 'RLXO.NS', 'ASTR.NS', 'PWFC.NS', 'BIRS.NS', 'LUPN.NS', 'LTOL.NS', 'EQUN.NS', 'WNS.N', 'CCRI.NS', 'SOFT.NS', 'INBK.NS', 'BRIH.NS', 'BAJE.NS', 'STTE.NS', 'JMSH.NS', 'KRBL.NS', 'IPCA.NS', 'WIPR.NS', 'WCKH.NS', 'HWKN.BO', 'SARE.NS', 'MRCO.NS', 'FISO.NS', 'SUN.NS', 'ICNT.NS', 'BAMN.NS', 'HCLT.NS', 'ICBK.NS', 'SIFY.OQ', 'SHMF.NS', 'HALC.NS', 'JMNA.NS', 'RLIN.NS', 'ESCO.NS', 'ZYDS.NS', 'RALL.NS', 'HVEL.NS', 'GSFC.NS', 'HAPL.NS', 'ASOK.NS', 'RECM.NS', 'SIEM.NS', 'CHPA.NS', 'GLEN.NS', 'NAAL.NS', 'KKCL.NS', 'BMBK.NS', 'PCBL.NS', 'EIHO.NS', 'PAGE.NS', 'ATLP.NS', 'KARU.NS', 'ZYDU.NS', 'ALOK.NS', 'JPLY.NS', 'COFO.NS', 'VART.NS', 'REDI.NS', 'TRIE.NS', 'SNFN.NS', 'TTKL.NS', 'NKML.NS', 'KRBR.NS', 'BJEL.NS', 'ACEL.NS', 'NAFT.NS', 'GESC.NS', 'MAXI.NS', 'SBI.NS', 'SOCE.NS', 'HIKA.NS', 'ADRG.NS', 'SLIN.NS', 'AMAR.NS', 'SPJT.BO', 'PRIS.NS', 'BION.NS', 'KNRL.NS', 'AJAN.NS', 'NSEN.NS', 'SEQU.NS', 'NAFL.NS', 'RADC.NS', 'HCPR.NS', 'DWAR.NS', 'INRM.NS', 'PLNG.NS', 'KRFI.BO', 'IIFL.NS', 'GRAN.NS', 'WSTC.NS', 'HMTL.NS', 'REXP.NS', 'GHFC.NS', 'RPOL.NS', 'ULTC.NS', 'AIAE.NS', 'MMFO.NS', 'LAOP.NS', 'NMDC.NS', 'VAKR.NS', 'SHCM.NS', 'RSYS.NS', 'IOBK.NS', 'TEIN.NS', 'KEIN.NS', 'TIMK.NS', 'RAJR.NS', 'JNSP.NS', 'VAIB.NS', 'GAME.NS', 'IRBI.NS', 'PHOE.NS', 'LICH.NS', 'TTEX.NS', 'MIRZ.NS', 'BLAC.NS', 'UNSP.NS', 'AJPH.NS', 'TCIL.NS', 'BRIG.NS', 'GETD.NS', 'NTPC.NS', 'MHSM.NS', 'ICRA.NS', 'RKFO.NS', 'HLLT.NS', 'EICH.NS', 'FINX.NS', 'HOEX.NS', 'RMT.NS', 'SCI.NS', 'MHSC.NS', 'VESU.NS', 'LYMT.NS', 'GDPI.NS', 'ELCN.NS', 'JAYN.NS', 'EDEL.NS', 'ZENT.NS', 'VSTI.NS', 'VOTL.NS', 'TABE.NS', 'CTBK.NS', 'ITMG.JK', 'BMRI.JK', 'BBRM.NS', 'BLDT.NS', 'MNCN.JK', 'ASII.JK', 'INTP.JK', 'KLBF.JK', 'SMGR.JK', 'HMSP.JK', 'INKP.JK', 'PGAS.JK', 'UNVR.JK', 'JPFA.JK', 'INCO.JK', 'UNTR.JK', 'TKIM.JK', 'BRPT.JK', 'ABDA.JK', 'BDMN.JK', 'AALI.JK', 'AKRA.JK', 'BMTR.JK', 'ASRI.JK', 'BBNI.JK', 'PTBA.JK', 'TLKM.JK', 'LPKR.JK', 'ASDM.JK', 'BUMI.JK', 'CPIN.JK', 'BNGA.JK', 'CTRA.JK', 'INDF.JK', 'BNII.JK', 'GGRM.JK', 'ABBA.JK', 'GJTL.JK', 'ISAT.JK', 'ANTM.JK', 'MYOR.JK', 'MTDL.JK', 'PNLF.JK', 'PWON.JK', 'LPPF.JK', 'SMRA.JK', 'MREI.JK', 'ADES.JK', 'NISP.JK', 'SCMA.JK', 'TPIA.JK', 'BBCA.JK', 'JSMR.JK', 'BBRI.JK', 'ADHI.JK', 'EXCL.JK', 'ACES.JK', 'ADMF.JK', 'KREN.JK', 'FRLD.SI', 'FREN.JK', 'BCOM.TA', 'BEZQ.TA', 'SHLTN.S', 'MGIC.OQ', 'ILCO.TA', 'ICL.TA', 'DLEKG.TA', 'ITRN.OQ', 'DSCT.TA', 'ESLT.TA', 'NICE.TA', 'MZTF.TA', 'NVMI.OQ', 'CRNT.OQ', 'CAMT.OQ', 'TEVA.TA', 'AMOT.TA', 'BVC.L', 'FIBI.TA', 'PTNR.TA', 'CHKP.OQ', 'CEL.TA', 'SILC.OQ', 'STRS.TA', 'TSEM.OQ', 'POLI.TA', 'ALLT.OQ', '8960.T', '2433.T', '9602.T', '4544.T', '3092.T', '6305.T', '4819.T', '9766.T', '2371.T', '8439.T', '3349.T', '4188.T', '8086.T', '6273.T', '8253.T', '8219.T', '6861.T', '7974.T', '7211.T', '6005.T', '1414.T', '1878.T', '9706.T', '9719.T', '8227.T', '9832.T', '5702.T', '7947.T', '4549.T', '8111.T', '5938.T', '1662.T', '9744.T', '1973.T', '2502.T', '6594.T', '9749.T', '3626.T', '8591.T', '6967.T', '3003.T', '5727.T', '7203.T', '5932.T', '6412.T', '1925.T', '5803.T', '1808.T', '4519.T', '8016.T', '5631.T', '4118.T', '7984.T', '7276.T', '1944.T', '4523.T', '1959.T', '4182.T', '6370.T', '5991.T', '6971.T', '5214.T', '7731.T', '4272.T', '2897.T', '6703.T', '8056.T', '7733.T', '1911.T', '6645.T', '6586.T', '4043.T', '5332.T', '4507.T', '1803.T', '6923.T', '6762.T', '1801.T', '4503.T', '7730.T', '5451.T', '1786.T', '8012.T', '8418.T', '9987.T', '9613.T', '7956.T', '8570.T', '6869.T', '2593.T', '9984.T', '9843.T', '6481.T', '6417.T', '8595.T', '8566.T', '2801.T', '6965.T', '9021.T', '4208.T', '9983.T', '2871.T', '7951.T', '4912.T', '6988.T', '4661.T', '7013.T', '7011.T', '8984.T', '5401.T', '7261.T', '6503.T', '5108.T', '7202.T', '6501.T', '3407.T', '9435.T', '7012.T', '3402.T', '4005.T', '7201.T', '5741.T', '1332.T', '7912.T', '5802.T', '5406.T', '5201.T', '4502.T', '6702.T', '6701.T', '6902.T', '4901.T', '5801.T', '6301.T', '7003.T', '6753.T', '7272.T', '7004.T', '7269.T', '6302.T', '4204.T', '3401.T', '6326.T', '5471.T', '7270.T', '7205.T', '2802.T', '2002.T', '5901.T', '4631.T', '3101.T', '3863.T', '7751.T', '5706.T', '5101.T', '6366.T', '7911.T', '4183.T', '3861.T', '2282.T', '5711.T', '4452.T', '4911.T', '4151.T', '5713.T', '3002.T', '4061.T', '6201.T', '2264.T', '4042.T', '3405.T', '7453.T', '6471.T', '4676.T', '6146.T', '3088.T', '9989.T', '7752.T', '9022.T', '9001.T', '3315.T', '4813.T', '3941.T', '4680.T', '6951.T', '8032.T', '8511.T', '6807.T', '8584.T', '3099.T', '9119.T', '9509.T', '4088.T', '3865.T', '4530.T', '9042.T', '9075.T', '5714.T', '9010.T', '6383.T', '9504.T', '4506.T', '7735.T', '8133.T', '3877.T', '5482.T', '2811.T', '6754.T', '6436.T', '7936.T', '9364.T', '9006.T', '9008.T', '9045.T', '9009.T', '6752.T', '9301.T', '9044.T', '2201.T', '2001.T', '5334.T', '7240.T', '6996.T', '4516.T', '4403.T', '6849.T', '4114.T', '9031.T', '4021.T', '1833.T', '9007.T', '6103.T', '4528.T', '5232.T', '6136.T', '5947.T', '9003.T', '4527.T', '6707.T', '4536.T', '9065.T', '8136.T', '5929.T', '9735.T', '3569.T', '2809.T', '9069.T', '9110.T', '6976.T', '4004.T', '6339.T', '9601.T', '2531.T', '8233.T', '7309.T', '9533.T', '1860.T', '4205.T', '6963.T', '9064.T', '7988.T', '2267.T', '6141.T', '8572.T', '8593.T', '4206.T', '5110.T', '6457.T', '6113.T', '8766.T', '2730.T', '3880.T', '6966.T', '7532.T', '2678.T', '4816.T', '8952.T', '8951.T', '4324.T', '8308.T', '4307.T', '9020.T', '8953.T', '8309.T', '6857.T', '8954.T', '8411.T', '6806.T', '3105.T', '6925.T', '6349.T', '5301.T', '1518.T', '8113.T', '8035.T', '4540.T', '6845.T', '7282.T', '2875.T', '5423.T', '8804.T', '7729.T', '4543.T', '4203.T', '6995.T', '8515.T', '5344.T', '3708.T', '9404.T', '4684.T', '4062.T', '6856.T', '6755.T', '5715.T', '4044.T', '8060.T', '4617.T', '3048.T', '8316.T', '9603.T', '9616.T', '4967.T', '4689.T', '9684.T', '4666.T', '8304.T', '4704.T', '6592.T', '8905.T', '3116.T', '3064.T', '5105.T', '2702.T', '7550.T', '4716.T', '4732.T', '5726.T', '3086.T', '9831.T', '5444.T', '7313.T', '1963.T', '3360.T', '8473.T', '4812.T', '6920.T', '7476.T', '4681.T', '8410.T', '6268.T', '2914.T', '2327.T', '7518.T', '8283.T', '8876.T', '6460.T', '9449.T', '4686.T', '8129.T', '9076.T', '7564.T', '6425.T', '7458.T', '6324.T', '6724.T', '6728.T', '9697.T', '7459.T', '9434.T', '9962.T', '1951.T', '1928.T', '2222.T', '4922.T', '8848.T', '3774.T', '4733.T', '8628.T', '4755.T', '8750.T', '2670.T', '4768.T', '7649.T', '2331.T', '6674.T', '3391.T', '2175.T', '8919.T', '6723.T', '8955.T', '6367.T', '8425.T', '8377.T', '6841.T', '7747.T', '1605.T', '3765.T', '9531.T', '2127.T', '5019.T', '9505.T', '2121.T', '3038.T', '3436.T', '9507.T', '2784.T', '6770.T', '8331.T', '3382.T', '9202.T', '6952.T', '6448.T', '8601.T', '6361.T', '9502.T', '8242.T', '8359.T', '8334.T', '8001.T', '2810.T', '8267.T', '1942.T', '8020.T', '9503.T', '1812.T', '4613.T', '7762.T', '8252.T', '9104.T', '2206.T', '6954.T', '8801.T', '8002.T', '9041.T', '8802.T', '8088.T', '9201.T', '5333.T', '7741.T', '6479.T', '8058.T', '8031.T', '6473.T', '9433.T', '9532.T', '9048.T', '8585.T', '4568.T', '9101.T', '8358.T', '5233.T', '1820.T', '9005.T', '9506.T', '9401.T', '8725.T', '8341.T', '1802.T', '1893.T', '3769.T', '9501.T', '3591.T', '2212.T', '4565.T', '9508.T', '6981.T', '4612.T', '4202.T', '9432.T', '1721.T', '8830.T', '6472.T', '4091.T', '7701.T', '8015.T', '9107.T', '4063.T', '3050.T', '6506.T', '5411.T', '5463.T', '7259.T', '8424.T', '9513.T', '3825.T', '3397.T', '2432.T', '8279.T', '2413.T', 'ARBK.AM', 'BOJX.AM', '3231.T', 'SIBK.AM', 'CABK.AM', 'AHLI.AM', 'APOT.AM', 'THBK.AM', 'JOIB.AM', '000810.KS', '001040.KS', '006800.KS', '001230.KS', '006260.KS', '032640.KS', '000240.KS', '071050.KS', '000080.KS', '005830.KS', '004020.KS', '009830.KS', '010620.KS', '000880.KS', '000120.KS', '003490.KS', '011780.KS', '002380.KS', '001570.KS', '034230.KS', '003550.KS', '004990.KS', '005300.KS', '060980.KS', '006360.KS', '000150.KS', '005940.KS', '004370.KS', '005070.KS', '006400.KS', '012450.KS', '005490.KS', '004170.KS', '009150.KS', '001800.KS', '010950.KS', '004800.KS', '041510.KQ', '017670.KS', '008770.KS', '011170.KS', '016360.KS', '012330.KS', '003670.KS', '007310.KS', '001450.KS', '005380.KS', '002790.KS', '000720.KS', '000100.KS', '012750.KS', '011200.KS', '015760.KS', '001740.KS', '028050.KS', '000660.KS', '084990.KQ', '012630.KS', '018880.KS', '093050.KS', '011790.KS', '012510.KS', '028300.KQ', '006280.KS', '019170.KS', '035720.KS', '042670.KS', '035760.KQ', '032500.KQ', '024110.KS', '036570.KS', '029780.KS', '034020.KS', '055550.KS', '066570.KS', '022100.KS', '105560.KS', '078930.KS', '010130.KS', '051910.KS', '010140.KS', '035250.KS', '010120.KS', '004000.KS', '021240.KS', '030200.KS', '051900.KS', '042660.KS', '000270.KS', '009540.KS', '066970.KS', '008930.KS', '020000.KS', '030000.KS', '036460.KS', '097950.KS', '033780.KS', '011070.KS', '035420.KS', '047050.KS', '047040.KS', '068760.KQ', '054950.KQ', '028670.KS', '003690.KS', '005930.KS', '010060.KS', '090430.KS', '056190.KQ', '009240.KS', '026960.KS', '086280.KS', '034220.KS', '035900.KQ', '046890.KQ', '023530.KS', '086790.KS', 'KIBK.KW', 'CBKK.KW', 'FTIK.KW', 'KFH.KW', 'ZAIN.KW', 'NBKK.KW', 'ASCK.KW', 'GBKK.KW', 'EQUI.KW', 'KAMC.KW', 'TAMK.KW', 'AGLT.KW', 'EKTT.KW', 'MABK.KW', 'FCI.AD', 'WETH.KW', 'ABKK.KW', 'MRCK.KW', 'BURG.KW', 'BOUK.KW', 'QIC.AD', 'SENE.KW', 'LAND.KL', 'UULI.KL', 'CJCE.KL', 'SLCN.KL', 'AIMD.KL', 'DUOP.KL', 'PMAS.KL', 'ETEX.KL', 'HTVE.KL', 'DUTA.KL', 'NESM.KL', 'TLMM.KL', 'IOIB.KL', 'MPIM.KL', 'FVCO.KL', 'TRCG.KL', 'ELSR.KL', 'TENA.KL', 'HENY.KL', 'ALDR.KL', 'MAHS.KL', 'AMMB.KL', 'AINM.KL', 'FMHB.KL', 'MALY.KL', 'FRAS.KL', 'MALA.KL', 'GENM.KL', 'MFMS.KL', 'GENT.KL', 'HEIN.KL', 'MUDA.KL', 'PANS.KL', 'SOPS.KL', 'STIK.KL', 'ECOW.KL', 'HLGC.KL', 'YTLS.KL', 'CMSM.KL', 'PRMS.KL', 'MEGA.KL', 'DATA.KL', 'ADVA.KL', 'DNEX.KL', 'TNTT.KL', 'ONEG.KL', 'YTLR.KL', 'DRBM.KL', 'MYRS.KL', 'ANCM.KL', 'TDMS.KL', 'GNCH.KL', 'IWCI.KL', 'STEE.KL', 'MYEG.KL', 'WTKH.KL', 'CNON.KL', 'HLBB.KL', 'DLEU.KL', 'GAMU.KL', 'EVGN.KL', 'MUHI.KL', 'MUIB.KL', 'ADVE.KL', 'PETR.KL', 'UTPS.KL', 'DOVT.KL', 'PBSB.KL', 'KLKK.KL', 'STAR.KL', 'PGAS.KL', 'MLTO.KL', 'KPJH.KL', 'INSS.KL', 'METR.KL', 'TAKA.KL', 'SAEG.KL', 'MBMR.KL', 'MNRB.KL', 'AEOM.KL', 'AXSR.KL', 'EKOV.KL', 'TANC.KL', 'MAYB.KL', 'YTLP.KL', 'KESM.KL', 'SETI.KL', 'SURI.KL', 'PARB.KL', 'INNO.KL', 'JTIA.KL', 'TCOM.KL', 'SPOR.KL', 'HLIB.KL', 'MAGM.KL', 'UEME.KL', 'GDEX.KL', 'HLCB.KL', 'LLBM.KL', 'QRES.KL', 'KIML.KL', 'HEXE.KL', 'PHUA.KL', 'HSIB.KL', 'WLOW.KL', 'ZHCO.KL', 'HCKC.KL', 'LHEN.KL', 'UCHI.KL', 'PTMR.KL', 'PIEN.KL', 'SCCM.KL', 'SUPM.KL', 'MATE.KL', 'PTRD.KL', 'PEPT.KL', 'NEXT.KL', 'HAPS.KL', 'COMF.KL', 'TPGC.KL', 'TWHB.KL', 'CITA.KL', 'LBSB.KL', 'RENE.KL', 'CBMS.KL', 'KSLH.KL', 'MISC.KL', 'BATO.KL', 'EGCM.KL', 'PRKN.KL', 'TASC.KL', 'HEVE.KL', 'WASC.KL', 'UNSM.KL', 'HONG.KL', 'PNTE.KL', 'KNMP.KL', 'WCEH.KL', 'ERHB.KL', 'AUMA.KL', 'CTAL.KL', 'CIMB.KL', 'AZRB.KL', 'SYML.KL', 'AJIY.KL', 'AJIN.KL', 'TWRK.KL', 'ANNJ.KL', 'VIZI.KL', 'BONI.KL', 'CBIP.KL', 'CWHB.KL', 'DELM.KL', 'CCKH.KL', 'ATAI.KL', 'BUAB.KL', 'FPIB.KL', 'CELC.KL', 'FIAM.KL', 'GADA.KL', 'GNIC.KL', 'FAJA.KL', 'HUMC.KL', 'DIAL.KL', 'JENT.KL', 'HOWA.KL', 'KKBE.KL', 'KRIB.KL', 'LAYH.KL', 'KUBM.KL', 'MSCB.KL', 'MAHB.KL', 'MITR.KL', 'PHMA.KL', 'RGTB.KL', 'PDNI.KL', 'PLSB.KL', 'REDI.KL', 'RAPD.KL', 'HEXR.KL', 'SCIB.KL', 'ROHA.KL', 'TAAN.KL', 'MILU.KL', 'KREJ.KL', 'TGIB.KL', 'TOMY.KL', 'THRB.KL', 'TEXC.KL', 'KOBA.KL', 'VSID.KL', 'YINS.KL', 'CABC.KL', 'TSHR.KL', 'KPSB.KL', 'SCIE.KL', 'HEXI.KL', 'HARR.KL', 'GKMS.KL', 'MPRM.KL', 'POWE.KL', 'HHCS.KL', 'TAFI.KL', 'MSWK.KL', 'HEKR.KL', 'IJMS.KL', 'SKPR.KL', 'NTPM.KL', 'STCM.L', 'TROP.KL', 'ENOB.KL', 'IMSP.KL', 'DEST.KL', 'POHK.KL', 'JFTB.KL', 'NCHB.KL', 'KWNF.KL', 'BPPL.KL', 'JAKS.KL', 'THEA.KL', 'LTER.KL', 'PERM.KL', 'VTRX.KL', 'GRNP.KL', 'CHIN.KL', 'YSPS.KL', 'HTHB.KL', 'WCAL.KL', 'OMES.KL', 'MBSB.KL', 'SLPB.KL', 'RGBI.KL', 'SCGB.KL', 'ABLE.KL', 'JHMC.KL', 'FAST.KL', 'FRKN.KL', 'LOTT.KL', 'NVTE.KL', 'HLEN.KL', 'ANCR.KL', 'SPLN.KL', 'PMBT.KL', 'TMCN.KL', 'MCRK.KL', 'RIMB.KL', 'MGBB.KL', 'AXIA.KL', 'IBHD.KL', 'NCTA.KL', 'ECOF.KL', 'SKOU.KL', 'PUBM.KL', 'DUFU.KL', 'EXFM.KL', 'THPB.KL', 'CSTH.KL', 'ANCO.KL', 'GENP.KL', 'SGNA.KL', 'MBBM.KL', 'DEHB.KL', 'AMZN.PSX', 'ATOR.PSX', 'BKEQ.PSX', 'HBL.PSX', 'FYBL.PSX', 'OGDC.PSX', 'LUKC.PSX', 'SBL.PSX', 'FAUF.PSX', 'BAFL.PSX', 'MCB.PSX', 'ACEN.PS', 'AP.PS', 'SM.PS', 'URC.PS', 'MEG.PS', 'RLC.PS', 'UBP.PS', 'JFC.PS', 'SECB.PS', 'JGS.PS', 'DMC.PS', 'SMPH.PS', 'BLOOM.PS', 'AEV.PS', 'AGI.PS', 'GLO.PS', 'AC.PS', 'MER.PS', 'SMC.PS', 'LTG.PS', 'MWC.PS', 'SCC.PS', 'BDO.PS', 'MBT.PS', 'ICT.PS', 'ALI.PS', 'PNB.PS', 'RCB.PS', 'UDCD.QA', 'SALM.QA', 'MARK.QA', 'MCGS.QA', 'COMB.QA', 'NLCS.QA', 'AABQ.QA', 'QEWC.QA', 'AHCS.QA', 'DICO.QA', 'ORDS.QA', 'QIIB.QA', 'QANC.QA', 'QIMC.QA', 'QGIR.QA', 'QCFD.QA', 'IQCD.QA', 'QGMS.QA', 'QGTS.QA', 'QNNC.QA', 'KINS.QA', 'WDAM.QA', 'QFLS.QA', 'ERES.QA', 'BRES.QA', 'DOBK.QA', 'QIIC.QA', 'QISB.QA', 'QNBK.QA', 'ZHCD.QA', 'GWCS.QA', 'DBIS.QA', 'QINS.QA', '1080.SE', '8030.SE', '2060.SE', '1060.SE', '2270.SE', '2290.SE', '3030.SE', '2190.SE', '2010.SE', '2380.SE', '1030.SE', '2020.SE', '4300.SE', '1050.SE', '2310.SE', '7020.SE', '3060.SE', '3050.SE', '4030.SE', '1120.SE', '1020.SE', '2250.SE', '3020.SE', '2050.SE', '3080.SE', '2280.SE', '4190.SE', '1010.SE', '5110.SE', '2330.SE', '8070.SE', '4210.SE', '7010.SE', '4220.SE', '2080.SE', '1140.SE', '8010.SE', '3040.SE', '2350.SE', 'SEAT.SI', 'WTHS.SI', 'HPPS.SI', 'SCIL.SI', 'RVHL.SI', 'CAPD.SI', 'BSES.SI', 'COSC.SI', 'AIMA.SI', 'AEM.SI', 'VENM.SI', 'UTOS.SI', 'FCRT.SI', 'WLIL.SI', 'CNAO.SI', 'GUOC.SI', 'SIND.SI', 'CYD.N', 'OVES.SI', '2186.HK', 'FRNM.SI', 'OCBC.SI', 'HPAR.SI', 'JCYC.SI', 'CTDM.SI', 'KPLM.SI', 'SIAL.SI', 'GAGR.SI', 'CSES.SI', 'HBEE.SI', 'RAFG.SI', 'SBVV.SI', 'KASA.SI', 'CAPC.SI', 'SIAE.SI', 'ESRE.SI', 'SATS.SI', 'SPOS.SI', 'CAPA.SI', 'CMDG.SI', 'THOS.SI', 'STAR.SI', 'GELA.SI', 'CDLT.SI', 'PWLR.SI', 'DBSM.SI', 'MAPL.SI', 'STHL.SI', 'DELF.SI', 'HLSF.SI', 'KEPL.SI', 'SUNT.SI', 'SPEN.CM', '2409.TW', '2412.TW', '2633.TW', '3045.TW', '2845.TW', '6244.TWO', '1102.TW', '2801.TW', '1314.TW', '2204.TW', '2903.TW', '2603.TW', '1103.TW', '1326.TW', '1216.TW', '1303.TW', '1440.TW', '1903.TW', '1605.TW', '2201.TW', '2606.TW', '2303.TW', '2227.TW', '2104.TW', '2891.TW', '6147.TWO', '6182.TWO', '2607.TW', '1909.TW', '3017.TW', '2601.TW', '2101.TW', '2915.TW', '2103.TW', '1476.TW', '9933.TW', '2015.TW', '2105.TW', '9910.TW', '2308.TW', '2324.TW', '2317.TW', '9904.TW', '9945.TW', '1227.TW', '2605.TW', '9917.TW', '1717.TW', '2609.TW', '2330.TW', '9921.TW', '1710.TW', '1565.TWO', 'HIMX.OQ', '2357.TW', '2337.TW', '2323.TW', '2344.TW', '2301.TW', '2615.TW', '2207.TW', '2352.TW', '2356.TW', '2912.TW', '2834.TW', '1477.TW', '2347.TW', '2383.TW', '2393.TW', '2439.TW', '5522.TW', '5371.TWO', '2707.TW', '2408.TW', '2492.TW', '3037.TW', '3008.TW', '2006.TW', '2385.TW', '2362.TW', '2360.TW', '2610.TW', '2884.TW', '6005.TW', '2885.TW', '2327.TW', '2882.TW', '2454.TW', '2881.TW', '2883.TW', '2498.TW', '2474.TW', '2886.TW', '4904.TW', '3034.TW', '2890.TW', '2887.TW', '2888.TW', '1722.TW', '2382.TW', '2049.TW', '2031.TW', '3189.TW', '6505.TW', '2376.TW', '2379.TW', '2392.TW', '8046.TW', '6116.TW', '6274.TWO', '3532.TW', '8936.TWO', '3481.TW', '1301.TW', '1504.TW', '1101.TW', '2449.TW', '1905.TW', '2880.TW', '8299.TWO', '2451.TW', '2353.TW', '1907.TW', '2371.TW', '2618.TW', '2002.TW', '1402.TW', '2345.TW', '6239.TW', '1802.TW', '2377.TW', '5347.TWO', '6121.TWO', '6176.TW', '8069.TWO', '2395.TW', '1434.TW', '3406.TW', 'SIMO.OQ', '3376.TW', '2542.TW', '3443.TW', '6214.TW', '2467.TW', '6282.TW', '6285.TW', '3227.TWO', '3231.TW', '9914.TW', '5483.TWO', '3023.TW', '5388.TW', '2892.TW', '3044.TW', 'LH.BK', 'INTUCH.BK', 'THANI.BK', 'TPIPL.BK', 'KTB.BK', 'BTS.BK', 'ADVANC.BK', 'BDMS.BK', 'SINGER.BK', 'SCGP.BK', 'RCL.BK', 'BCH.BK', 'BANPU.BK', 'NYVKt.MU', 'HTC.BK', 'KCE.BK', 'SNP.BK', 'SCC.BK', 'BJC.BK', 'CSC.BK', 'WACOAL.BK', 'CENTEL.BK', 'SYNTEC.BK', 'JAS.BK', 'CFRESH.BK', 'PSL.BK', 'IRPC.BK', 'EGCO.BK', 'IRC.BK', 'BCP.BK', 'TU.BK', 'CPN.BK', 'ILINK.BK', 'CK.BK', 'DELTA.BK', 'SPALI.BK', 'GFPT.BK', 'MBK.BK', 'TISCO.BK', 'BEC.BK', 'TCAP.BK', 'PTTEP.BK', 'TVO.BK', 'BH.BK', 'AJ.BK', 'TPCS.BK', 'KKP.BK', 'THCOM.BK', 'SMPC.BK', 'PT.BK', 'TASCO.BK', 'KBANK.BK', 'BBL.BK', 'THAI.BK', 'MINT.BK', 'AEONTS.BK', 'SCCC.BK', 'HMPRO.BK', 'TNL.BK', 'MAJOR.BK', 'FORTH.BK', 'EASTW.BK', 'CPALL.BK', 'AMATA.BK', 'BAFS.BK', 'CPF.BK', 'ASIAN.BK', 'HANA.BK', 'ERW.BK', 'M-CHAI.BK', 'LPN.BK', 'NOBLE.BK', 'MSC.BK', 'AP.BK', 'SSSC.BK', 'SIRI.BK', 'STA.BK', 'THIP.BK', 'SNJ.BK', 'DRT.BK', 'TTA.BK', 'PTT.BK', 'MFEC.BK', 'TIPCO.BK', 'SCG.BK', 'BWG.BK', 'TMT.BK', 'FPT.BK', 'AH.BK', 'RATCH.BK', 'QH.BK', 'SC.BK', 'BSRC.BK', 'S.BK', 'AOT.BK', 'TKS.BK', 'TOG.BK', 'KTC.BK', 'CIMBT.BK', 'TSC.BK', 'BAY.BK', 'SNC.BK', 'RS.BK', 'TSTH.BK', 'SAT.BK', 'EAND.AD', 'TTB.BK', 'TBEV.SI', 'TOP.BK', 'SUPER.BK', 'ADCB.AD', 'SCIDC.AD', 'RAKCEC.AD', 'ARMX.DU', 'ALDAR.AD', 'TABR.DU', 'GCEM.AD', 'NMDC.AD', 'DISB.DU', 'UAB.AD', 'EMAR.DU', 'DANA.AD', 'MASB.DU', 'DINV.DU', 'DU.DU', 'ADNH.AD', 'CBD.DU', 'FAB.AD', 'GLYHO.IS', 'AYGAZ.IS', 'ADIB.AD', 'ASELS.IS', 'TAVHL.IS', 'ARCLK.IS', 'BOS.AD', 'EREGL.IS', 'PETKM.IS', 'PETUN.IS', 'MGROS.IS', 'OYAKC.IS', 'HEKTS.IS', 'SARKY.IS', 'GARAN.IS', 'ECILC.IS', 'VESTL.IS', 'TOASO.IS', 'TSKB.IS', 'FROTO.IS', 'SISE.IS', 'KRDMD.IS', 'TUPRS.IS', 'YKBNK.IS', 'AKSA.IS', 'THYAO.IS', 'ENKAI.IS', 'NETAS.IS', 'YUNSA.IS', 'TATGD.IS', 'ANSGR.IS', 'AKGRT.IS', 'ISCTR.IS', 'BIMAS.IS', 'DOHOL.IS', 'ALGYO.IS', 'ISFIN.IS', 'ASUZU.IS', 'AEFES.IS', 'SAHOL.IS', 'SASA.IS', 'TURSG.IS', 'CCOLA.IS', 'ALARK.IS', 'AKBNK.IS', 'ZOREN.IS', 'KCHOL.IS', 'TCELL.IS', 'BRISA.IS', 'DOAS.IS', 'ECZYT.IS', 'AKCNS.IS', 'ULKER.IS', 'TKFEN.IS', 'DYOBY.IS', 'KORDS.IS', 'BRYAT.IS', 'OTKAR.IS', 'CIMSA.IS', 'HALKB.IS', 'KLMSN.IS', 'KERVT.IS', 'ALKIM.IS', 'AKENR.IS', 'TTRAK.IS', 'CRFSA.IS', 'KOZAA.IS', 'ANHYT.IS', 'VAKBN.IS', 'PNSUT.IS', 'ISGYO.IS', 'ALBRK.IS', 'SKBNK.IS', 'AGHOL.IS', 'NUHCM.IS', 'VESBE.IS', 'VKGYO.IS', 'SELEC.IS', 'AAZ.L', 'EMWPF.PK', 'FRHC.OQ', 'KLIC.OQ', 'NHTC.OQ', 'CORBF.PK', 'KAJR.NS', 'ENLT.TA', 'BILDCO.AD', 'ADNOCDIST.AD', 'HFDS.NS', 'HDFA.NS', 'CHAL.NS', 'JUST.NS', 'GRSE.NS', 'HDFL.NS', 'OPTI.NS', 'CAPL.NS', 'NBF.AD', 'LTIM.NS', 'VATE.NS', 'IHC.AD', 'WTON.JK', '601577.SS', 'HIAE.NS', 'INVESTB.AD', 'JULPHAR.AD', '601128.SS', 'HSBK.KZ', 'SAFA.NS', 'GENA.NS', '352820.KS', 'COMU.NS', 'SMMN.NS', 'ARVF.NS', '688728.SS', '4483.T', 'KOZAL.IS', 'AJBNK.DU', 'ASTL.NS', '601825.SS', 'UTIA.NS', 'LEMO.NS', 'SUMG.NS', 'OYAYO.IS', '601162.SS', 'NARY.NS', 'ENBD.DU', 'ADSB.AD', 'MHEB.KL', 'BASGZ.IS', 'ALAF.KW', 'NATI.BO', '6465.T', '7780.T', '2229.T', 'SYMP.NS', 'GODE.NS', 'PIIL.NS', '9142.T', 'PERS.NS', 'LOGO.IS', 'BLOM.BY', 'CSBB.NS', 'BUTT.NS', 'IRCN.NS', '069960.KS', 'ALLI.KL', 'DHAFRA.AD', '1180.SE', 'ADNIC.AD', 'PINSU.IS', 'PTG.BK', 'FPT.HM', 'COCH.NS', 'ADEN.NS', '1210.SE', '088350.KS', 'CHG.BK', 'EPG.BK', 'MAHH.NS', 'RUCL.NS', 'OILI.NS', 'ZIM.N', 'ALAIN.AD', '8150.TW', 'DODL.NS', 'GLAD.NS', '601869.SS', 'POWR.JK', 'BLA.BK', '601077.SS', '601608.SS', 'RAKNIC.AD', 'RAPCO.AD', 'WSKT.JK', 'INID.NS', 'AFIN.KL', 'AFNIC.AD', 'VNM.HM', '1320.SE', '020150.KS', '601990.SS', 'SINU.NS', 'NBCC.NS', '2381.SE', 'NUVO.NS', 'SPRC.BK', '601888.SS', '057050.KS', 'NHPC.NS', '002368.SZ', 'NOAH.N', 'ARAM.AD', 'SICO.AD', '0636.HK', '601788.SS', '002430.SZ', 'MOIL.NS', 'PISA.NS', '600958.SS', 'HBC.HNO', 'GPPL.NS', 'NYT.BK', '603888.SS', 'PNBH.NS', 'TKFL.AD', 'NCTH.AD', '601229.SS', 'RITS.NS', '600928.SS', '601238.SS', '6740.T', '300383.SZ', 'MTEL.JK', '300009.SZ', 'SSPW.OM', '601696.SS', 'ALKE.NS', 'ICIR.NS', 'UAC.BK', '300294.SZ', '1606.HK', '688321.SS', '3661.TW', '688521.SS', 'EIB.DU', '603986.SS', 'DESA.IS', 'SHEF.NS', 'SICO.BH', '9898.HK', 'GRMO.NS', '081660.KS', 'SFUNY.PK', '688012.SS', 'CENA.NS', '601963.SS', '002966.SZ', 'DRIVE.AD', '002705.SZ', 'AWNIC.AD', 'NAZA.NS', 'ADIE.NS', 'AAV.BK', 'RELG.NS', 'RATB.NS', 'BKSB.OM', 'AIRX.KL', 'KNNK.KL', 'ABDL.NS', 'PAYT.NS', 'NBQ.AD', '601021.SS', 'ROLE.NS', '688599.SS', 'FBI.AD', 'AVAP.L', 'KVRI.NS', 'DIXO.NS', 'BRIS.JK', 'MAZG.NS', '6446.TW', '7779.T', 'NIPF.NS', 'PREG.NS', '601233.SS', '300296.SZ', 'NEWG.NS', 'HH.AD', 'ABNIC.AD', 'WEST.NS', 'GMPC.AD', 'ADAN.NS', 'VIC.HM', 'GODR.NS', 'HERN.NS', 'SAB.HM', 'RAIT.NS', '3673.TW', 'KHALEEJI.BH', 'QUEC.NS', 'SJVN.NS', 'CRAF.NS', '1727.HK', '601717.SS', '002405.SZ', 'DATP.NS', 'VARB.NS', 'CEIF.NS', 'KPRM.NS', '7182.T', '300482.SZ', '002422.SZ', '601018.SS', 'MISR.NS', 'EPIG.NS', 'UNION.AD', 'PNBS.JK', '1777.HK', 'EURK.NS', 'SISI.NS', '8210.SE', 'VFQS.QA', '688202.SS', '6415.TW', 'INUS.NS', 'BSIM.JK', 'ISGE.NS', 'SEDG.OQ', '1150.SE', 'HPG.HM', 'MSC.N', 'JUWL.NS', '1918.HK', '601828.SS', '601801.SS', '1638.HK', 'TLI.BK', '002500.SZ', '601336.SS', 'DLPA.NS', '1211.SE', '601688.SS', '4979.TWO', 'JUBI.NS', 'THYO.NS', '601881.SS', '002223.SZ', '002230.SZ', '601958.SS', 'NBOB.OM', 'OTEL.OM', 'RCCI.OM', 'ABOB.OM', 'BKMB.OM', '8100.SE', '8060.SE', '7030.SE', 'SGXL.SI', '8697.T', 'MCEI.NS', '0388.HK', 'DFM.DU', 'BMYS.KL', 'BSEL.NS', 'SABINA.BK', 'TTW.BK', 'ADAVIATION.AD', 'SIB.AD', 'TAQA.AD', 'EMSTEEL.AD', 'WAHA.AD', '002233.SZ', '002180.SZ', 'BKDB.OM', 'OMVS.OM', 'QOIS.QA', 'AGTHIA.AD', 'ISMEN.IS', '0493.HK', '002240.SZ', 'PM.BK', 'SGP.BK', '1368.HK', 'TTKOM.IS', '0832.HK', 'VGUA.NS', 'BJFS.NS', '002236.SZ', '002241.SZ', '002244.SZ', '002242.SZ', 'KIRI.NS', '096770.KS', 'IHGS.QA', '6271.TW', '097230.KS', '051600.KS', '3622.TW', 'FH.AD', '3529.TWO', '9958.TW', 'EIC.AD', '098460.KQ', 'SYNEX.BK', 'JKIP.NS', 'RAKPROP.AD', '002183.SZ', 'METHAQ.AD', '002250.SZ', '0691.HK', '002249.SZ', '002252.SZ', '6190.HK', '8190.SE', '3698.HK', '8200.SE', 'IIAN.NS', '4001.SE', '7167.T', 'JSWE.NS', 'SDGU.KL', '002268.SZ', 'ICCI.NS', '300748.SZ', 'MASF.NS', 'MANI.NS', '0880.HK', 'BSDE.JK', 'ADRO.JK', 'LCBH.KL', 'UZMA.KL', 'MOFS.NS', '002262.SZ', 'XIN.N', '002237.SZ', 'BYAN.JK', '601766.SS', 'ASTE.NS', 'APEX.AD', 'JMART.BK', '002273.SZ', '002266.SZ', 'CBI.AD', '002271.SZ', 'VPB.HM', '002601.SZ', 'MAIT.NS', '7181.T', 'POLC.NS', 'KBC.HM', 'PBFI.NS', '600901.SS', '2181.T', 'EA.BK', '3141.T', 'UMSB.KL', 'TEL.PS', 'MAVI.IS', 'ZFCV.NS', '2269.T', '002415.SZ', '3632.T', 'ANELE.IS', 'TSCP.KL', '601718.SS', 'TOWR.JK', '601200.SS', '688520.SS', '4147.TWO', '0881.HK', 'AMRT.JK', '086900.KQ', 'MERS.QA', 'KIOC.NS', 'HAG.HM', '300408.SZ', '002938.SZ', '002416.SZ', '002294.SZ', '002498.SZ', '002506.SZ', '002440.SZ', '603456.SS', '603799.SS', '601997.SS', '601058.SS', '002423.SZ', '002920.SZ', '603486.SS', '002643.SZ', '603256.SS', '601163.SS', '002670.SZ', '002382.SZ', '300498.SZ', '002465.SZ', '601231.SS', '603699.SS', '601222.SS', '002396.SZ', '002419.SZ', '603379.SS', '002444.SZ', '002429.SZ', 'AMBE.NS', 'ANUY.NS', 'FINO.NS', 'INMR.NS', 'LAXR.NS', 'SYNN.NS', 'TLSV.NS', 'TEGA.NS', 'VARE.NS', 'TEJS.NS', '6532.T', '9090.T', '3659.T', 'RHBC.KL', 'ISDMR.IS', 'PGSUS.IS', 'AKSEN.IS', 'AGESA.IS', 'BIZIM.IS', 'KCAER.IS', 'POLHO.IS', 'SUNTK.IS', '688819.SS', '601360.SS', '001203.SZ', '002541.SZ', '002727.SZ', '002424.SZ', '300999.SZ', '600917.SS', '601799.SS', '002385.SZ', '601311.SS', '601658.SS', '002756.SZ', '002372.SZ', '002293.SZ', '1578.HK', 'MUTT.NS', 'SASE.NS', 'DEVY.NS', '3635.T', '5857.T', 'MINC.NS', '002463.SZ', '1972.HK', 'INGL.NS', 'GALX.NS', '300438.SZ', 'CAPG.NS', 'DIG.HM', '600919.SS', '601512.SS', '2222.SE', '6808.HK', '4927.T', 'ITDC.NS', '002602.SZ', '002326.SZ', 'LIFI.NS', 'NEOE.NS', 'HEAC.NS', 'GTCAP.PS', '601106.SS', 'ICIL.NS', 'AUFI.NS', '601966.SS', '002508.SZ', '603712.SS', 'LAUL.NS', 'VEILV.L', '1361.HK', '603228.SS', 'SAHB.KL', '032830.KS', 'TTCL.BK', '300888.SZ', '002414.SZ', 'CTD.HM', '601992.SS', 'JAPF.SI', 'TATV.NS', 'KECL.NS', 'COAL.NS', '8306.T', '5202.T', '8053.T', '7832.T', '8795.T', '2579.T', '4751.T', '028260.KS', '2503.T', '6758.T', 'BGRO.KL', 'BJTM.JK', 'RAKBANK.AD', '600999.SS', 'BNLI.JK', 'BPP.BK', 'CAPN.SI', '000333.SZ', 'MCCS.QA', '2799.HK', 'CMLT.SI', 'BBTN.JK', '3320.HK', 'OR.BK', 'JCYI.KL', '003470.KS', 'ORDS.OM', '601288.SS', '0845.HK', '6098.T', '002027.SZ', '601698.SS', 'DEWAA.DU', 'VCG.HM', '1333.T', '6178.T', 'MOLT.NS', 'GLOS.NS', 'WHAUP.BK', 'ALBH.BH', 'IDIP.NS', '002939.SZ', '011210.KS', '4743.TWO', '601668.SS', 'KZAP.KZ', '300115.SZ', '002583.SZ', 'TPIPP.BK', 'EMTK.JK', '6561.TWO', 'EMPOWER.DU', '601319.SS', '8354.T', '1359.HK', 'VSTE.KL', '601818.SS', 'CTG.HM', 'UOKH.SI', '3702.TW', 'KARSN.IS', 'GISS.QA', 'HAYAH.AD', 'THREL.BK', 'IVL.BK', '002946.SZ', '0669.HK', 'ZAIN.BH', '002926.SZ', '3396.HK', 'SIME.KL', '600926.SS', '034730.KS', 'STEG.SI', '2588.HK', 'HUDC.NS', 'RAKWCT.AD', '2223.SE', 'PNCI.NS', 'VRLL.NS', '1213.SE', 'TIDLOR.BK', 'TTML.NS', 'VIPI.NS', '1789.TW', 'STEL.SI', 'SIPR.KL', 'SBIL.NS', '601377.SS', 'UBL.PSX', '9024.T', '1810.SE', 'FIDELITYUNITED.AD', '7267.T', '018260.KS', '2768.T', '6504.T', 'GLOBAL.BK', 'LITm.BK', '9143.T', '8273.T', 'JAPR.NS', 'MSN.HM', '5876.TW', '601155.SS', 'QNBTR.IS', 'KMGZ.KZ', '600918.SS', '601211.SS', '601158.SS', '071840.KS', 'JIOF.NS', 'BAJA.NS', '007070.KS', 'BPI.PS', 'BABA.N', '0215.HK', 'VIJA.NS', '600025.SS', 'LUMI.TA', '000210.KS', '601916.SS', '2354.TW', '7164.T', '000166.SZ', 'PEERm.BK', '2501.T', '047810.KS', '7202.SE', '4578.T', '601117.SS', 'UOBH.SI', 'FRPL.SI', '6004.SE', '4902.T', '068270.KS', '002281.SZ', '111770.KS', '601098.SS', '300136.SZ', '601995.SS', '002287.SZ', 'TKNSA.IS', 'GUNKUL.BK', 'THEE.NS', '1966.HK', 'HEXT.KL', '601187.SS', '300788.SZ', '601066.SS', '2637.TW', '002292.SZ', '601618.SS', '8714.T', 'LHFG.BK', '1099.HK', 'SOHU.OQ', '603259.SS', '300760.SZ', 'AIRA.DU', '1508.HK', 'AZRG.TA', '002299.SZ', '300002.SZ', '300010.SZ', '002683.SZ', '2020.HK', '300001.SZ', '4938.TW', '300003.SZ', '1668.HK', '4002.SE', '1128.HK', '002518.SZ', '300024.SZ', '3360.HK', '300015.SZ', '300014.SZ', '002384.SZ', 'SUMH.NS', '1238.HK', '002311.SZ', '300027.SZ', '300026.SZ', '2587.T', '300017.SZ', '002302.SZ', '300012.SZ', '300168.SZ', '600998.SS', 'QMDT.SI', '002304.SZ', '3105.TWO', '1628.HK', '002314.SZ', 'KELG.KL', '300180.SZ', 'KICO.AD', 'QFBQ.QA', '002475.SZ', 'MXSC.KL', '5020.T', 'HAWY.NS', '601059.SS', 'FGVH.KL', '0960.HK', 'MPARK.IS', 'MACE.NS', 'FENG.N', '1928.HK', 'CCM.N', '002432.SZ', '603077.SS', '603806.SS', '002531.SZ', '603589.SS', '300376.SZ', '002841.SZ', '688122.SS', '603885.SS', '603606.SS', '002648.SZ', '002738.SZ', '300251.SZ', '002791.SZ', '301035.SZ', '603866.SS', '002472.SZ', '002773.SZ', '603868.SS', '002461.SZ', '601636.SS', '603707.SS', '603737.SS', '002595.SZ', '002698.SZ', '002409.SZ', '603688.SS', '002624.SZ', '300207.SZ', '002745.SZ', '300088.SZ', '603833.SS', '300212.SZ', '002555.SZ', '002916.SZ', '002709.SZ', '601100.SS', '601108.SS', '002497.SZ', '300618.SZ', '002635.SZ', '002493.SZ', '300869.SZ', '002563.SZ', '002557.SZ', '603567.SS', '603529.SS', '002701.SZ', '605358.SS', '603858.SS', '603026.SS', '603233.SS', '002532.SZ', '603345.SS', '300118.SZ', '002690.SZ', '002865.SZ', '002460.SZ', '002821.SZ', '002572.SZ', '002815.SZ', '603708.SS', '603195.SS', '002831.SZ', '002600.SZ', '603728.SS', '002603.SZ', '0631.HK', 'TEEC.NS', '002317.SZ', '0916.HK', 'VELE.KL', '601139.SS', 'KRII.NS', '112040.KQ', '600905.SS', '300034.SZ', '002371.SZ', '601179.SS', '601989.SS', '2066.HK', '300033.SZ', 'SBIC.NS', 'DXG.HM', '9668.HK', '300037.SZ', 'CART.NS', '601877.SS', 'DQ.N', '600927.SS', '002340.SZ', '002352.SZ', '002353.SZ', 'JKS.N', '688223.SS', 'EKGYO.IS', 'KDH.HM', '300058.SZ', 'HOME.KL', '002373.SZ', '300059.SZ', '8630.T', '002375.SZ', 'PSHL.KL', 'OEBO.NS', '027410.KS', 'VND.HM', 'TMILLm.BK', '300769.SZ', '002387.SZ', '002390.SZ', 'BANK.JK', '1179.HK', 'TRVT.NS', '300070.SZ', '6138.HK', '601567.SS', '601369.SS', 'STAU.NS', '300072.SZ', '300413.SZ', '300073.SZ', 'QIGD.QA', '4174.TWO', 'BACO.NS', '601212.SS', '002608.SZ', '002607.SZ', '603225.SS', '002399.SZ', '1555.HK', 'ICBP.JK', 'PUNA.NS', '002407.SZ', '002408.SZ', '601665.SS', 'PHOE.TA', '4915.TW', '300085.SZ', 'BAAC.BO', '002410.SZ', '272210.KS', '214320.KS', '002714.SZ', '002434.SZ', '603087.SS', 'SHIN.KL', '002439.SZ', '300271.SZ', '603927.SS', '601000.SS', '600500.SS', '4310.SE', '2128.HK', 'ENRG.TA', '300558.SZ', '1788.HK', 'PGEO.JK', '300133.SZ', 'FOPT.KL', '2723.TW', 'RSPO.NS', '002456.SZ', 'SUNW.KL', '1299.HK', 'TBIG.JK', '603355.SS', 'TOA.BK', '002458.SZ', '002459.SZ', '300144.SZ', '300373.SZ', '601969.SS', '300257.SZ', '128940.KS', 'PDR.HM', 'GOOD.JK', '0975.HK', '601136.SS', '601236.SS', '300113.SZ', '300114.SZ', '002948.SZ', '002466.SZ', 'GEX.HM', 'MMYT.OQ', 'MAXE.NS', '002468.SZ', 'IHHH.KL', '300568.SZ', 'BAM.BK', '300773.SZ', '002673.SZ', 'ZOMT.NS', '096530.KQ', '293490.KQ', 'GCAPm.BK', 'AKPm.BK', 'AYDEM.IS', '600909.SS', '601375.SS', '002945.SZ', '601555.SS', '601878.SS', '601216.SS', 'ENDU.NS', 'SHYE.NS', 'LINK.JK', '002736.SZ', '688009.SS', 'STC.KW', 'BTPS.JK', 'AKFGY.IS', 'ADAW.NS', '601118.SS', '601198.SS', '002936.SZ', 'AUTH.NS', 'FINE.NS', 'ESCB.KL', 'MRDS.QA', '4321.SE', 'DMT.BK', 'BGRIM.BK', 'OSP.BK', 'BARA.NS', 'MGAS.NS', '300122.SZ', '300124.SZ', 'CYPR.KL', 'ADPORTS.AD', 'PCGB.KL', '1308.HK', '1958.HK', '0853.HK', 'DUBK.QA', '002487.SZ', '300142.SZ', '600956.SS', 'AVAL.KL', '1733.HK', '2282.HK', '601615.SS', 'TAL.N', 'HVN.HM', 'OXHL.SI', '002511.SZ', '300253.SZ', '301236.SZ', '002517.SZ', 'LTFL.NS', '002507.SZ', '1101.HK', '1117.HK', '601933.SS', 'CARE.KL', '1728.HK', '300146.SZ', 'SWAY.KL', '1112.HK', 'PCJE.NS', '1590.TW', 'ONEW.NS', '601519.SS', '300182.SZ', '002558.SZ', '300770.SZ', 'KIRO.NS', '300166.SZ', 'MARA.KW', 'MONDE.PS', 'GRSEL.IS', '300223.SZ', '603000.SS', 'ESHRAQ.AD', 'MAPI.SI', 'JIYAD.KW', 'HLGYO.IS', '1378.HK', 'BJFO.KL', 'IH.AD', 'WCTE.KL', '137310.KS', '1182.SE', '002797.SZ', 'XPP.L', '002568.SZ', '9926.HK', '4928.T', '1623.HK', 'HPHT.SI', 'GINF.NS', '688777.SS', 'VHM.HM', '138930.KS', 'CRDE.NS', 'TIFA.JK', 'VNET.OQ', 'GENIL.IS', 'CMSN.NS', 'DIBL.NS', 'BMGR.KL', '600989.SS', '603156.SS', '601838.SS', '001227.SZ', '1916.HK', '601669.SS', '002839.SZ', '601901.SS', 'LIFE.JK', '138040.KS', '139480.KS', '002958.SZ', '600903.SS', '600908.SS', 'TVDHm.BK', '139130.KS', '2558.HK', 'MSMH.KL', 'ABMM.JK', 'HIBI.KL', '0968.HK', 'AWAN.KL', '8454.TW', 'XNET.OQ', 'BKNZ.OM', 'VGI.BK', '1301.SE', '5274.TWO', '0579.HK', 'ASTR.KL', 'AMAR.JK', 'SAWAD.BK', '300496.SZ', '601860.SS', 'INAR.KL', '3668.T', 'PGOLD.PS', '300244.SZ', '603260.SS', '5871.TW', 'CKP.BK', 'SHEN.SI', '5269.TW', '601127.SS', '601456.SS', 'ITEL.BK', 'SAEN.KL', 'SOKM.IS', '001965.SZ', '601012.SS', '688065.SS', 'BEMA.QA', 'BERA.KL', '207940.KS', '300274.SZ', '6770.TW', 'SHAD.NS', 'WHA.BK', '600032.SS', 'BESA.NS', '196170.KQ', '300357.SZ', '603392.SS', '300347.SZ', 'URGN.OQ', '688082.SS', '300363.SZ', '300630.SZ', '603658.SS', '688276.SS', '4587.T', 'KLKIM.IS', '688331.SS', 'JMT.BK', 'HEML.NS', '002625.SZ', 'ADAG.NS', '6414.TW', '603377.SS', '601225.SS', '300285.SZ', 'ALEM.NS', 'BARQ.NS', 'ATHM.N', 'PTTGC.BK', '600977.SS', 'RHIM.NS', 'EMI.PS', 'TBEI.NS', '601928.SS', '300601.SZ', '4003.SE', '688303.SS', '603899.SS', '002812.SZ', '300315.SZ', '002653.SZ', '601689.SS', 'MALO.NS', '3197.T', '1269.HK', '6823.HK', '9698.HK', '603501.SS', 'HAPP.NS', 'INWN.NS', '300433.SZ', 'CMCM.N', 'WPHB.KL', '326030.KS', '300604.SZ', 'SCAC.NS', '251270.KS', '601326.SS', 'CSTE.OQ', '601016.SS', '601228.SS', '5880.TW', 'ADTB.NS', '688348.SS', '1929.HK', '300765.SZ', '8436.TWO', '601568.SS', '601816.SS', '688772.SS', '601985.SS', '1201.SE', '001286.SZ', '601611.SS', 'ENJSA.IS', '600968.SS', '003035.SZ', '688347.SS', '300308.SZ', 'HOME.NS', '603719.SS', 'YTRA.OQ', 'EQUI.NS', '1030.HK', '300316.SZ', '603444.SS', 'VIPS.N', '300418.SZ', '002739.SZ', '603369.SS', '3697.T', '603288.SS', 'BUMI.SI', '601865.SS', '6488.TWO', '603185.SS', '603198.SS', '603019.SS', 'PEIN.KL', 'ICON.KL', 'FVRR.N', '688185.SS', '601068.SS', '6028.T', 'SPAD.NS', '603883.SS', 'GWIND.IS', '300395.SZ', 'TUNE.KL', '300529.SZ', '300390.SZ', 'FPIm.BK', 'SEIN.NS', 'DELH.NS', 'GAQR.KL', 'OCKG.KL', 'PPSm.BK', 'APTS.NS', 'VERUS.IS', 'VMAR.NS', 'RTNI.NS', '600959.SS', 'ERIS.NS', 'DSON.KL', 'FAEH.SI', 'ACST.JK', 'ASSA.JK', 'IGRE.KL', '2082.SE', 'TKN.BK', '688363.SS', '603217.SS', '2500.HK', 'PPP.BK', '161390.KS', 'DLLA.NS', 'PESO.KL', '300450.SZ', '4004.SE', 'YY.OQ', '300628.SZ', '0884.HK', '3281.T', '003022.SZ', '688690.SS', 'RAIB.NS', 'AKSGY.IS', '1908.HK', 'KC.OQ', '688295.SS', '603127.SS', '6409.TW', 'PLANB.BK', '1232.HK', 'AMATAV.BK', '002901.SZ', 'GGAS.NS', '3283.T', '0288.HK', 'ICHI.BK', 'AMIO.NS', 'TITI.KL', 'IOIP.KL', 'FRT.HM', '300783.SZ', 'GPSC.BK', 'KALN.NS', 'ODAS.IS', 'MWG.HM', '688063.SS', 'HTTBT.IS', '3380.HK', 'LITB.N', '603816.SS', 'RAIV.NS', '688271.SS', '263750.KQ', 'MACT.SI', '2386.HK', 'HGIN.NS', 'AVEU.NS', 'SPRL.NS', 'MATR.KL', 'LEON.KL', 'TARO.NS', '603056.SS', 'WIX.OQ', '688008.SS', '603983.SS', 'RRHI.PS', '1107.HK', 'SALU.KL', '175330.KS', 'PLUSP.L', 'ORCE.NS', 'PARA.SI', '300463.SZ', '3291.T', '3289.T', 'ADIA.NS', 'MEGA.BK', '300763.SZ', '688050.SS', 'PCSGH.BK', '181710.KS', 'ROSB.NS', '180640.KS', '3288.T', 'CANTE.IS', '1830.SE', 'MPHC.QA', 'SEVE.KL', '2638.HK', '300661.SZ', '600906.SS', '0799.HK', '1862.HK', '3994.T', 'KARE.KL', '1302.SE', '4013.SE', 'SWIF.KL', '6199.HK', 'METP.NS', '1268.HK', 'DIFu.BK', '215600.KQ', '1233.HK', '0586.HK', '600023.SS', '688779.SS', '1622.HK', 'LTEH.NS', 'OUEI.SI', '1810.HK', '9618.HK', 'WVE.OQ', 'SHCR.NS', '300724.SZ', '002761.SZ', '603650.SS', '603160.SS', '603515.SS', '002818.SZ', '2255.HK', '300502.SZ', 'TCTM.OQ', '1202.SE', '4164.SE', '603568.SS', '300474.SZ', '300832.SZ', '603027.SS', 'LEJUY.PK', '603338.SS', '603317.SS', '300442.SZ', 'GMS.L', '300458.SZ', '300487.SZ', '002925.SZ', 'TOUR.OQ', '4385.T', '3923.T', '241560.KS', 'ECOH.KL', 'NEXN.L', '601298.SS', '601778.SS', '601019.SS', 'FSCA.SI', '9468.T', '3639.HK', '300594.SZ', '300759.SZ', '9633.HK', '002850.SZ', '2329.HK', 'CYBR.OQ', '300676.SZ', 'TME.N', '0788.HK', '003816.SZ', 'CBG.BK', 'MTC.BK', 'AVAS.NS', '688696.SS', 'DMW.PS', 'FRHO.SI', '8464.TW', '603882.SS', '002867.SZ', 'TFG.BK', '204320.KS', 'FSNE.NS', 'CARM.KL', '1303.SE', '603517.SS', 'ANAA.NS', 'ECOD.KL', 'PDSL.NS', 'NVD.BK', 'IFIB.NS', 'FDV.AX', 'MOMO.OQ', '603939.SS', '603218.SS', 'ORI.BK', '300682.SZ', '3132.T', '603638.SS', 'ICLK.OQ', 'INEE.NS', '603863.SS', '300750.SZ', '688180.SS', 'PECC.KL', 'SE.N', '6060.HK', 'INIR.NS', 'KINE.KL', 'SOGP.OQ', 'LEOG.KL', 'GOFA.NS', '1113.HK', 'GUJL.NS', 'KRNT.OQ', 'KEPE.SI', 'MDKA.JK', 'DADA.OQ', '300957.SZ', 'OC.DI', 'GEEN.NS', '1310.HK', '0001.HK', 'HUIZ.OQ', '7203.SE', 'BIOA.KL', 'BCG.HM', 'COM7.BK', '601868.SS', 'BZUN.OQ', 'SJWD.BK', 'CROP.NS', '2269.HK', 'CHIU.KL', 'METB.NS', 'LUXI.NS', '2669.HK', '5021.T', '7180.T', '603826.SS', '6456.TW', '688289.SS', 'IIFS.NS', 'MANU.SI', '688538.SS', '300595.SZ', 'BANH.NS', 'ADAI.NS', 'BCPG.BK', 'AMRY.KL', 'RANH.KL', '603613.SS', 'MYNE.KL', 'MULTIPLY.AD', 'IDFB.NS', '3462.T', '1585.HK', '001979.SZ', 'AZREF.PK', 'TWPC.BK', '603605.SS', 'DIDIY.PK', '300699.SZ', '688208.SS', '688029.SS', '7186.T', '323410.KS', '1548.HK', 'DXS.HM', 'ARTO.JK', '688798.SS', '9666.HK', '3301.HK', 'ADNA.NS', '2869.HK', '2296.T', 'BEM.BK', 'BORS.NS', '688126.SS', '3690.HK', 'EASM.NS', 'BLSN.NS', 'CABN.KL', '6811.TWO', 'GULF.BK', 'DALB.NS', '1579.HK', '603305.SS', '1789.HK', '300677.SZ', 'ROUT.NS', '300672.SZ', 'COE.A', 'FRAE.SI', 'SAPI.NS', 'FOUN.KL', '603867.SS', '247540.KQ', '603659.SS', 'HSSE.KL', 'AMMN.JK', '603893.SS', 'MAII.NS', 'ZHEN.SI', 'BCMA.KL', '3549.T', 'LU.N', 'UCBK.NS', '688153.SS', '1357.HK', 'FRTL.NS', 'YUMC.N', '688425.SS', 'TCIE.NS', '300861.SZ', 'PERT.KL', '6169.HK', 'INDG.NS', '300776.SZ', '688390.SS', '3563.T', '2057.HK', '300726.SZ', 'KSPI.KZ', '603267.SS', 'PSH.BK', '601156.SS', 'IGTR.NS', 'EBRIT.BH', '300866.SZ', 'NVL.HM', 'KIPR.KL', 'PSPP.NS', 'IGRD.QA', 'PRM.BK', '4161.SE', 'AVIA.JK', 'WLCON.PS', 'YI.OQ', 'IRBN.NS', 'UJJI.NS', '603596.SS', '6049.HK', '267270.KS', 'III.BK', '267260.KS', '267250.KS', '9863.HK', 'ANGO.NS', 'BEDU.N', '601138.SS', 'ADBm.BK', '688052.SS', '0772.HK', '688561.SS', '688111.SS', '1821.HK', '6669.TW', '271560.KS', 'ZEN.BK', 'STAT.NS', '300741.SZ', 'BEST.N', 'VEDN.NS', '6088.HK', '300751.SZ', '688536.SS', '300896.SZ', 'PMET.KL', 'NETL.SI', 'BINC.KL', '688516.SS', '300454.SZ', 'ASIA.L', 'SECOY.PK', '0839.HK', '1997.HK', '9688.HK', '688256.SS', '3319.HK', 'EMAARDEV.DU', 'QD.N', '688114.SS', 'PRCE.NS', 'NAAS.OQ', '6158.HK', '377300.KS', 'FINV.N', 'TFMAMA.BK', 'STGO.TO', '688281.SS', '300850.SZ', 'KMPUR.IS', '603786.SS', '282330.KS', 'SASS.SI', 'LX.OQ', 'TQM.BK', 'STON.SI', '688707.SS', '285130.KS', 'SKYW.KL', 'NOVW.KL', 'CLEA.NS', 'ONTE.NS', 'QESG.KL', '301269.SZ', 'GDBH.KL', 'ZEPP.N', 'CAPI.KL', '1833.HK', '688567.SS', 'STG.N', '259960.KS', 'GHG.N', 'IQ.OQ', 'BILI.OQ', 'GOTO.JK', 'KPIE.NS', '301267.SZ', 'DOHOME.BK', 'RBF.BK', 'SICTm.BK', '6098.HK', '300782.SZ', 'CNVRG.PS', 'TECB.KL', '2798.HK', 'HUYA.N', 'AFFL.NS', 'ASW.BK', 'REVE.KL', '3711.TW', '1755.HK', 'BGC.BK', '2048.HK', 'ITES.NS', '294870.KS', '688301.SS', '6862.HK', 'MITE.KL', 'PR9.BK', '688036.SS', 'UXIN.OQ', 'GENS.SI', '0780.HK', 'CANG.N', 'JG.OQ', '302440.KS', 'GLBE.OQ', '6689.TW', 'CNF.N', '1797.HK', '1873.HK', '1995.HK', 'WEIDY.PK', 'NIO.N', '2013.HK', 'CTKYY.PK', 'QTTOY.PK', 'SISB.BK', '688006.SS', 'XYF.N', 'VIOT.OQ', '688220.SS', 'RESR.NS', '3692.HK', 'FARM.KL', 'NIU.OQ', 'JFU.OQ', '603290.SS', '1801.HK', 'QFIN.OQ', 'MOGU.N', 'QAMC.QA', 'GREA.KL', 'PLTK.OQ', 'STEN.NS', 'GREP.NS', 'ACE.BK', '0667.HK', 'JFIN.OQ', '002985.SZ', 'RATE.NS', '688088.SS', 'FUTU.OQ', 'QUAGR.IS', '316140.KS', 'HIIL.NS', 'UWCB.KL', 'ADNOCLS.AD', 'AMEE.KL', 'SUVH.NS', '1951.HK', 'TIGR.OQ', 'OCFT.N', 'SOLA.KL', '600925.SS', 'TUYA.N', '688778.SS', 'EMBA.NS', 'ENEY.BO', 'YJ.OQ', 'MRDI.KL', 'SY.OQ', '688188.SS', '688005.SS', '688116.SS', '688169.SS', '688002.SS', '688099.SS', 'DOYU.OQ', 'LKNCY.PK', 'GOTU.N', '1876.HK', '688072.SS', '361610.KS', '2015.HK', 'AWC.BK', '329180.KS', '688107.SS', '6110.HK', '605117.SS', 'PRIE.SI', '2400.HK', 'CTOS.KL', 'INNA.KL', 'NATEN.IS', '688200.SS', '9922.HK', 'ALIV.NS', '7071.T', '6969.HK', '003031.SZ', 'KRKR.OQ', 'DAO.N', 'PAIYY.PK', 'FERTIGLB.AD', 'DUO.OQ', '336370.KS', 'CRC.BK', 'HITN.NS', '6186.HK', 'SHR.BK', 'MAXN.OQ', 'EMAM.NS', 'BLDN.QA', 'MKDTY.PK', 'STGT.BK', 'KLCC.KL', 'CAPS.SI', 'MINS.NS', '1209.HK', 'NNOX.OQ', 'SAK.BK', 'NRF.BK', 'PRAF.NS', '6078.HK', 'OPTI.KL', 'UCL.OQ', '0873.HK', 'MEGI.NS', 'ECOR.KL', 'ASTOR.IS', '301029.SZ', '688608.SS', '605499.SS', '300919.SZ', '2423.HK', '688105.SS', '688234.SS', '688686.SS', 'BNR.OQ', '7337.T', '1024.HK', 'QH.OQ', 'API.OQ', 'ESEN.IS', '0909.HK', 'QLMI.QA', '1516.HK', 'KONTR.IS', '357780.KQ', 'XPEV.N', 'KEX.BK', 'BIOEN.IS', 'IH.N', 'LXEH.OQ', 'ITMA.KL', '6618.HK', '9896.HK', 'SAMA.KL', '689009.SS', '2281.SE', '688120.SS', 'YSG.N', '6993.HK', 'YQ.OQ', 'YMM.N', '9992.HK', '4887.T', '373220.KS', 'CGNT.OQ', '688032.SS', 'RLX.N', 'SONB.NS', '375500.KS', '3714.TW', '6699.HK', 'HAFNI.OL', 'JUBN.NS', 'BROF.NS', 'SREN.NS', '2618.HK', '6781.TW', 'BBGI.BK', 'NATG.NS', '300979.SZ', '688349.SS', '1111.SE', 'STX.OQ', 'MAGEN.IS', 'POWG.NS', 'GRAB.OQ', '6741.TWO', '383220.KS', 'BZ.OQ', '5076.T', '688248.SS', '9147.T', 'CONSE.IS', 'SENH.KL', 'CORA.KL', 'MKDM.QA', 'AURE.KL', 'ALPHADHABI.AD', 'LATN.NS', 'ADNOCDRILL.AD', '688297.SS', '688375.SS', '1321.SE', 'CMRY.JK', '688041.SS', '402340.KS', 'BESH.KL', '0020.HK', 'SMRTG.IS', 'SCB.BK', 'OLAG.SI', 'E7.AD', 'YANZ.SI', 'BOROUGE.AD', '2367.HK', '5831.T', '5832.T', '5830.T', 'AMER.KL', '9658.HK', 'SALIK.DU', 'BURJEEL.AD', 'SPACE42.AD', 'AMR.AD', 'PRESIGHT.AD', '2382.SE', 'CASI.OQ', 'ADNOCGAS.AD', '5844.T', 'PUREHEALTH.AD', 'PHX.AD', 'KNDI.OQ', 'BWLPG.OL', 'SPNS.OQ', '8604.T']
worldRICs = europeanRICs + africanRICs + oceanianRICs + americanRICs + asianRICs

In [5]:
def Climate_GHGCompanyTracking(debug=False):

    ld.open_session()
    
    tfaRICs = TextFieldAutosuggest(
        placeholder='RIC',
        custom_items=[
            {
                'label': name,
                'value': {
                'title': name,
                'vc': 'OP',
                'symbol': name,
                'subtitle': name}}
            for name in worldRICs],
        ignore_list=['APP', 'FX', 'OP', 'CMP', 'EQ', 'INDX', 'BOND', 'FND', 'COM', 'OPT', 'ECON', 'FI', 'PREF', 'FUT', 'OTH', 'NEWS'])
    # tfaRICsOut = Output()
    
    scopeSelectOut = widgets.Output()
    scopeChoice = Select(
        placeholder='Scope', width=300.0)
    scopeChoice.data = [
        {'value': i, 'label': i, 'items': []}
        for i in ["Scope 1", "Scope 2", "Scope 3", "Scope 1 and 2", "Scope 1 and 2 and 3"]]
    # display(scopeChoice, scopeSelectOut) # If you want to display each choice box one above the other, you can use this line. I chose `HBox`s instead.

    # Display both choice boxes (`RICChoice` & `ScopeChoice`) horisontally, next to eachother.
    display(widgets.HBox([tfaRICs, scopeChoice])) # This will display both choice boxes horisontally, next to eachother.

    termSelectOut = widgets.Output()
    termChoice = Select(
        placeholder='Term', width=300.0)
    termChoice.data = [
        {'value': i, 'label': i, 'items': []}
        for i in ["ST", "LT"]]
    # display(termChoice, termSelectOut)

    # Display both choice boxes (`RICChoice` & `ScopeChoice`) horisontally, next to eachother.
    # display(widgets.HBox([setChoice, termChoice])) # This will display both choice boxes horisontally, next to eachother.

    sourceSelectOut = widgets.Output()
    sourceChoice = Select(
        placeholder='Source', width=300.0)
    sourceChoice.data = [
        {'value': i, 'label': i, 'items': []}
        for i in ["LSEG", "CDP"]]
    # display(sourceChoice, sourceSelectOut)

    display(widgets.HBox([termChoice, sourceChoice]))

    yearStartSelectOut = widgets.Output()
    yearStartChoice = Select(
        placeholder='Fiscal Year Range Start', width=300.0)
    yearStartChoice.data = [
        {'value': str(i), 'label': str(i), 'items': []}
        for i in range(2010, datetime.now().year+1)] # The `+1` here is because the `range` function is exclusive with respect to its upper limit
    # display(yearStartChoice, yearStartSelectOut)

    yearEndSelectOut = widgets.Output()
    yearEndChoice = Select(
        placeholder='Fiscal Year Range End', width=300.0)
    yearEndChoice.data = [
        {'value': str(i), 'label': str(i), 'items': []}
        for i in range(2010, datetime.now().year+1)] # The `+1` here is because the `range` function is exclusive with respect to its upper limit
    display(widgets.HBox([yearStartChoice, yearEndChoice])) # This will display both choice boxes horisontally, next to eachother.

    # Create Widgets
    button = Button('Create/Update Graph')
    button_output = widgets.Output()

    loader = Loader(visible=False)
    loader.visible = not loader.visible

    # Create Click Handler
    def click_handler(_a):
        with button_output:
            IPython.display.clear_output(wait=True)

            if debug or yearStartChoice.value >= yearEndChoice.value:
                print("\n")
                print(f"tfaRICs.value: {tfaRICs.value}")
                print(f"scopeChoice.value: {scopeChoice.value}")
                print(f"termChoice.value: {termChoice.value}")
                print(f"sourceChoice.value: {sourceChoice.value}")
                print(f"yearStartChoice.value: {yearStartChoice.value}")
                print(f"yearEndChoice.value: {yearEndChoice.value}")

            if yearStartChoice.value >= yearEndChoice.value:
                raise Exception(
                    "Expected Exception: yearStartChoice >= yearEndChoice. Please make sure that yearStartChoice < yearEndChoice")

            resultsPerYearDict = {fsclyr: [] for fsclyr in range(int(yearStartChoice.value), int(yearEndChoice.value)+1)}
            yearsThatWork, yearsThatDoNotWork = {i: [] for i in range(1,7)}, []

            # In the following `for` loop, we catch the Fiscal Years where data is available.
            for fsclyr in range(int(yearStartChoice.value), int(yearEndChoice.value)+1):
                ghg = GHGEmissionsBaseVsTargetUsingLSEGWorkspaceClass(
                    printMessage=False,
                    keepMessageObject=True,
                    scope=scopeChoice.value,
                    RIC=tfaRICs.value,
                    fiscalYear=int(fsclyr),
                    term=termChoice.value,
                    source=sourceChoice.value,
                    suppressWarnings=True)
                try:
                    setDictOfLists, dfOfSets, _dfOfSets = ghg.get_list_of_sets_that_contain_our_scope(
                        fiscalYear=fsclyr, RIC=ghg.RIC, term=ghg.term, scope=ghg.scope)
                    if debug:
                        print(f"setDictOfLists: {setDictOfLists}")
                        print(f"setDictOfLists[fsclyr]: {setDictOfLists[fsclyr]}")
                    for _set in setDictOfLists[fsclyr]:
                        try: # It is possible that not all the data required for the calculation for this _set exist, in which case we'd need this `try` loop.
                            result = ghg.get_data(
                                warning=False, fiscalYear=int(fsclyr), returnElements=True, set=_set)
                            if debug:
                                result.get_fig().fig.show()
                                print(f"_set: {_set}")
                                print(f"yearsThatWork: {yearsThatWork}")
                            yearsThatWork[int(_set)].append(fsclyr)
                            resultsPerYearDict[fsclyr].append([_set, result])
                        except:
                            yearsThatDoNotWork.append(fsclyr)
                except:
                    yearsThatDoNotWork.append(fsclyr)
            _yearsThatWork = [
                int(y) for y in resultsPerYearDict.keys()
                if len(resultsPerYearDict[y]) > 0]  # This `if` statement is needed just in case the set exists but the Starting Target Date or Ending Taget Date doesn't. This can happen if a company announces new target details, but no datapoint to assign to them yet.

            if debug:
                print("\n")
                print(f"yearsThatWork: {yearsThatWork}")
                print(f"yearsThatDoNotWork: {yearsThatDoNotWork}")
                print(f"setDictOfLists: {setDictOfLists}")
                print(f"resultsPerYearDict: {resultsPerYearDict}")

            resultsPerYearDictOnlyForSuccesfullYears = {
                y: resultsPerYearDict[y] for y in resultsPerYearDict.keys()
                if len(resultsPerYearDict[y]) > 0} # This `if` statement is needed just in case the set exists but the Starting Target Date or Ending Taget Date doesn't. This can happen if a company announces new target details, but no datapoint to assign to them yet.
            setDict = {} # Remember that a Set is a permutation of Starting Target Date, Ending Taget Date and Target (in Co2 Tons)
            for year, entries in resultsPerYearDictOnlyForSuccesfullYears.items():
                for entry in entries:
                    key = int(entry[0])
                    if key not in setDict:
                        setDict[key] = []
                    setDict[key].append(year)
            if debug:
                print(f"setDict: {setDict}")

            tabBar1OutDict = {
                _set : widgets.Output() for _set in setDict.keys()}
            tabBar1 = widgets.Tab(children=[
                tabBar1OutDict[_set] for _set in setDict.keys()])
            for i, _set in enumerate(tabBar1OutDict.keys()):
                tabBar1.set_title(i, f"Set{_set}")
            display(tabBar1)
            if debug:
                print(f"tabBar1OutDict: {tabBar1OutDict}")
                print("[tabBar1OutDict[_set] for _set in setDict.keys()]): " +
                      f"{[tabBar1OutDict[_set] for _set in setDict.keys()]}")
                print("\n")

            tabBar2OutDict = {
                _set: [
                    widgets.Output()
                    for i in yearsThatWork[_set]]
                for _set in setDict.keys()}
            tabBar2Dict = {
                _set: widgets.Tab(children=tabBar2OutDict[_set])
                for _set in setDict.keys()}
            for _set in tabBar1OutDict.keys():
                for i, yearThatWorks in enumerate(yearsThatWork[_set]):
                    tabBar2Dict[_set].set_title(i, f"Y{yearThatWorks}")
            if debug:
                print(f"tabBar2OutDict: {tabBar2OutDict}")
                print("\n")

            # creating tab bars
            for _set in tabBar2OutDict.keys():
                resultsPerYearDictForSet = {}
                for y in resultsPerYearDict.keys():
                    if len(resultsPerYearDict[y]) > 0:
                        for entry in resultsPerYearDict[y]:
                            set_key = str(entry[0])  # Ensure set_key is a string
                            if set_key not in resultsPerYearDictForSet:
                                resultsPerYearDictForSet[set_key] = {}
                            # Create a new instance of GHGEmissionsBaseVsTargetUsingLSEGWorkspaceClass for each set and year
                            resultsPerYearDictForSet[set_key][y] = GHGEmissionsBaseVsTargetUsingLSEGWorkspaceClass(
                                printMessage=False,
                                keepMessageObject=True,
                                scope=scopeChoice.value,
                                RIC=tfaRICs.value,
                                fiscalYear=int(y),
                                term=termChoice.value,
                                source=sourceChoice.value,
                                suppressWarnings=True)
                            resultsPerYearDictForSet[set_key][y].get_data(
                                warning=False, fiscalYear=int(y), returnElements=True, set=int(set_key))
                if debug:
                    print(f"resultsPerYearDictForSet: `{resultsPerYearDictForSet}`")
                with tabBar1OutDict[int(_set)]:
                    display(tabBar2Dict[int(_set)])
                    for i, y in enumerate(yearsThatWork[int(_set)]):
                        with tabBar2OutDict[int(_set)][i]:
                            if debug:
                                print(f"resultsPerYearDictForSet: {resultsPerYearDictForSet}")
                                print(f"[type(i) for i in resultsPerYearDictForSet.keys()]: {[type(i) for i in resultsPerYearDictForSet.keys()]}")
                                print(f"resultsPerYearDictForSet[str(_set)]: {resultsPerYearDictForSet[str(_set)]}")
                                print(f"Checking data for year {y} and set {int(_set)}")
                            if str(_set) in resultsPerYearDictForSet and y in resultsPerYearDictForSet[str(_set)]:
                                print(f"Displaying data for year {y} and set {int(_set)}")
                                print(resultsPerYearDictForSet[str(_set)][y].messageObject)
                                resultsPerYearDictForSet[str(_set)][y].get_fig().fig.show()
                            else:
                                print(f"No data for year {y} and set {int(_set)}")
            print("End Of App Run. If nothing is shown, no data for available Sets were retrievable.")

    # refister click handler for button
    button.on_click(click_handler)
    display(button)

    # display our widgets
    display(button_output)

In [6]:
Climate_GHGCompanyTracking(debug=False)

Button(height=0.0, tooltip='', value='Create/Update Graph', width=0.0)

Output()